# Recalcule Creance pour toute les entreprises Comparaison Calcule vs Non Calcule (2021)

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Charger les données
df = pd.read_excel("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//new_millesime_CIR_2021_corrected_sans_doublon.xlsx")

def convertir_en_nombre(valeur, defaut=0):
    """Convertit une valeur en nombre de façon sécurisée"""
    if pd.isna(valeur) or valeur == '' or valeur is None:
        return defaut
    try:
        return float(valeur)
    except:
        return defaut

def comparer_cir_declare_recalcule_2021():
    """Compare le CIR déclaré et le CIR recalculé pour 2021
    Note: Toutes les quote-parts sont mises à 0 (pas de prise en compte des sociétés de personnes)"""
    print(f"Nombre total d'entreprises: {len(df):,}")

    # Colonnes nécessaires pour recalculer le CIR 2021
    colonnes = [
        'siren_declarant', 'siren_deposant', 'DESIGN', 'COMPLT_DESIGN',
        'MT_NET_DEP_RD', 'MT_NET_DEP_RD_DOM', 'MT_TOT_CIR_CI_COLL_CII',
        'MT_CIR_RECH_YC_QP', 'MT_CI_COLL_APRS_MINIMI', 'MT_CII_YC_QP',
        'DOT_AMORT_IMMO', 'DOT_AMORT_IMMO_SINISTR', 'DEP_CHERCH_TECH', 'REM_SAL_INV',
        'DEP_JD', 'OTR_DEP_FONCT', 'FRAIS_BREV_COV', 'DEP_MAINT_BREV_COV',
        'DOT_AMORT_BREV', 'DEP_NORMALI', 'PRIM_COTIZ', 'DEP_VEIL_TECHNO',
        'MT_DEP_FONCT_TOT', 'MT_TOT_RD_1',
        # Sous-traitance organismes publics
        'DEP_ST_ORG_PUB_LIE_FR', 'DEP_ST_ORG_PUB_LIE_ETR', 
        'DEP_ST_ORG_PUB_NON_LIE_FR', 'DEP_ST_ORG_PUB_NON_LIE_ETR',
        'MT_TOT_ST_ORG_PUB',
        # Sous-traitance organismes privés
        'DEP_ST_ORG_PRIV_LIE_FR', 'DEP_ST_ORG_PRIV_LIE_ETR',
        'DEP_ST_ORG_PRIV_NON_LIE_FR', 'DEP_ST_ORG_PRIV_NON_LIE_ETR',
        'MT_TOT_ST_ORG_PRIV', 'PLAF_ST_ORG_PRIV', 'MT_TOT_ST', 
        'PLAF_ST_ORG_LIE', 'MT_ST_ORG_NON_LIE_PLAF', 'PLAF_GNRL_ST', 'MT_TOT_ST_PLAF',
        'MT_TOT_RD_2',
        'MT_AID_SUBV', 'MT_ENC_PRESTA', 'MT_DEP_CONSEILS_CIR', 'REMBST_SUBV',
        'FRAIS_COLL', 'FRAIS_DEF_DESSIN', 'MT_TOT_DEP_COLL', 'MT_AID_SUBV_COLL',
        'MT_DEP_CONSEILS_CIR_COLL', 'REMBST_SUBV_COLL', 'MT_NET_DEP_COLL', 'MT_NET_DEP_COLL_DOM',
        'DOT_AMORT_IMMO_INO', 'DEP_PERSONEL_INO', 'OTR_DEP_FONCT_INO',
        'FRAIS_BREV_COV_INO', 'FRAIS_DEF_BREV_INO', 'OP_INOV_EXT', 
        'MT_TOT_DEP_INO', 'MT_TOT_DEP_INO_PLAF',
        'MT_AID_SUBV_INO', 'MT_ENC_PRESTA_INO', 'MT_DEP_CONSEILS_CII', 'REMBST_SUBV_INO',
        'MT_NET_DEP_INO', 'MT_NET_DEP_INO_DOM', 'MT_NET_DEP_INO_MPE_CORSE', 'MT_NET_DEP_INO_ME_CORSE',
        'MT_QP_CIR_RECU', 'MT_QP_COLL_RECU', 'MT_QP_CII_RECU', 'MT_AIDE_MINIMI'
    ]

    # Convertir en numérique et créer une copie défragmentée
    df_tmp = df.copy()
    for col in colonnes:
        if col in df_tmp.columns:
            if col not in ['siren_declarant', 'siren_deposant', 'DESIGN', 'COMPLT_DESIGN']:
                df_tmp[col] = df_tmp[col].apply(convertir_en_nombre)
        else:
            if col not in ['siren_declarant', 'siren_deposant', 'DESIGN', 'COMPLT_DESIGN']:
                df_tmp[col] = 0

    # Créer un dictionnaire pour stocker toutes les colonnes calculées
    calc_columns = {}

    ## I - DÉPENSES DE RECHERCHE OUVRANT DROIT AU CRÉDIT D'IMPÔT 
    ## ANNÉE CIVILE 2021

    # 1. Dépenses internes

    # Ligne 6: Autres dépenses de fonctionnement
    calc_columns['LIGNE_6_CALC'] = (df_tmp['DOT_AMORT_IMMO'] * 0.75) + \
                               ((df_tmp['DEP_CHERCH_TECH'] + df_tmp['REM_SAL_INV']) * 0.43) + \
                               df_tmp['DEP_JD']

    # Ligne 7: Total dépenses de fonctionnement
    calc_columns['LIGNE_7_CALC'] = df_tmp['DOT_AMORT_IMMO'] + df_tmp['DOT_AMORT_IMMO_SINISTR'] + \
                               df_tmp['DEP_CHERCH_TECH'] + df_tmp['REM_SAL_INV'] + \
                               df_tmp['DEP_JD'] + calc_columns['LIGNE_6_CALC']

    # Appliquer les plafonds pour les lignes concernées
    # Ligne 11: Dépenses liées à la normalisation (déjà à 50% dans la déclaration)
    calc_columns['DEP_NORMALI_RECALC'] = df_tmp['DEP_NORMALI']

    # Ligne 12: Primes et cotisations (plafond 60 000 €)
    calc_columns['PRIM_COTIZ_PLAFONNEES'] = np.minimum(df_tmp['PRIM_COTIZ'], 60000)

    # Ligne 13: Dépenses de veille technologique (plafond 60 000 €)
    calc_columns['DEP_VEIL_TECHNO_PLAFONNEES'] = np.minimum(df_tmp['DEP_VEIL_TECHNO'], 60000)

    # Ligne 14: Total dépenses internes avec plafonds appliqués
    calc_columns['LIGNE_14_CALC'] = calc_columns['LIGNE_7_CALC'] + df_tmp['FRAIS_BREV_COV'] + \
                                df_tmp['DEP_MAINT_BREV_COV'] + df_tmp['DOT_AMORT_BREV'] + \
                                calc_columns['DEP_NORMALI_RECALC'] + calc_columns['PRIM_COTIZ_PLAFONNEES'] + \
                                calc_columns['DEP_VEIL_TECHNO_PLAFONNEES']

    # 2. Dépenses de sous-traitance

    # Ligne 17: Total opérations confiées aux organismes publics (avec doublement pour non liés)
    calc_columns['LIGNE_17_CALC'] = df_tmp['DEP_ST_ORG_PUB_LIE_FR'] + df_tmp['DEP_ST_ORG_PUB_LIE_ETR'] + \
                                df_tmp['DEP_ST_ORG_PUB_NON_LIE_FR'] + df_tmp['DEP_ST_ORG_PUB_NON_LIE_ETR']

    # Ligne 20: Total opérations confiées aux organismes privés
    calc_columns['LIGNE_20_CALC'] = df_tmp['DEP_ST_ORG_PRIV_LIE_FR'] + df_tmp['DEP_ST_ORG_PRIV_LIE_ETR'] + \
                                df_tmp['DEP_ST_ORG_PRIV_NON_LIE_FR'] + df_tmp['DEP_ST_ORG_PRIV_NON_LIE_ETR']

    # Préparation pour les calculs complexes
    df_calc = pd.DataFrame(calc_columns)
    df_calc_temp = pd.concat([df_tmp, df_calc], axis=1)

    # Ligne 21: Plafonnement des opérations privées (3x dépenses internes + publiques)
    calc_columns['LIGNE_21_CALC'] = np.minimum(df_calc_temp['LIGNE_20_CALC'], 
                                            (df_calc_temp['LIGNE_14_CALC'] + df_calc_temp['LIGNE_17_CALC']) * 3)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_21_CALC'] = calc_columns['LIGNE_21_CALC']

    # Ligne 22: Total des opérations de sous-traitance
    calc_columns['LIGNE_22_CALC'] = df_calc_temp['LIGNE_17_CALC'] + df_calc_temp['LIGNE_21_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_22_CALC'] = calc_columns['LIGNE_22_CALC']

    # Ligne 23: Plafonnement organismes liés (2M€)
    ligne_23_values = []
    for _, row in df_calc_temp.iterrows():
        dep_liees_pub = row['DEP_ST_ORG_PUB_LIE_FR'] + row['DEP_ST_ORG_PUB_LIE_ETR']
        dep_liees_priv = row['DEP_ST_ORG_PRIV_LIE_FR'] + row['DEP_ST_ORG_PRIV_LIE_ETR']
        # Pour les privés, prendre seulement la part dans la limite de ligne 21
        dep_priv_plafonnees = min(row['DEP_ST_ORG_PRIV_LIE_FR'] + row['DEP_ST_ORG_PRIV_LIE_ETR'], 
                                 row['LIGNE_21_CALC'])
        total_liees = dep_liees_pub + dep_priv_plafonnees
        ligne_23_values.append(min(total_liees, 2000000))
    calc_columns['LIGNE_23_CALC'] = np.array(ligne_23_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_23_CALC'] = calc_columns['LIGNE_23_CALC']

    # Ligne 24: Montant plafonné organismes non liés
    ligne_24_values = []
    for _, row in df_calc_temp.iterrows():
        dep_non_liees_pub = row['DEP_ST_ORG_PUB_NON_LIE_FR'] + row['DEP_ST_ORG_PUB_NON_LIE_ETR']
        dep_non_liees_priv = row['DEP_ST_ORG_PRIV_NON_LIE_FR'] + row['DEP_ST_ORG_PRIV_NON_LIE_ETR']
        total_non_liees = dep_non_liees_pub + dep_non_liees_priv
        plafond_restant = row['LIGNE_22_CALC'] - row['LIGNE_23_CALC']
        ligne_24_values.append(min(total_non_liees, plafond_restant))
    calc_columns['LIGNE_24_CALC'] = np.array(ligne_24_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_24_CALC'] = calc_columns['LIGNE_24_CALC']

    # Ligne 25: Plafonnement général
    ligne_25_values = []
    for _, row in df_calc_temp.iterrows():
        # Vérifier quelles lignes sont remplies pour déterminer le plafond
        has_liees = (row['DEP_ST_ORG_PUB_LIE_FR'] + row['DEP_ST_ORG_PUB_LIE_ETR'] +
                    row['DEP_ST_ORG_PRIV_LIE_FR'] + row['DEP_ST_ORG_PRIV_LIE_ETR']) > 0
        has_non_liees_priv = (row['DEP_ST_ORG_PRIV_NON_LIE_FR'] + row['DEP_ST_ORG_PRIV_NON_LIE_ETR']) > 0
        has_non_liees_pub = (row['DEP_ST_ORG_PUB_NON_LIE_FR'] + row['DEP_ST_ORG_PUB_NON_LIE_ETR']) > 0
        
        if has_liees and not has_non_liees_priv and not has_non_liees_pub:
            # Seules les opérations liées
            ligne_25_values.append(2000000)
        elif has_liees and has_non_liees_priv and not has_non_liees_pub:
            # Liées + non liées privées
            ligne_25_values.append(10000000)
        elif has_liees and has_non_liees_priv and has_non_liees_pub:
            # Toutes les catégories
            pub_non_liees = row['DEP_ST_ORG_PUB_NON_LIE_FR'] + row['DEP_ST_ORG_PUB_NON_LIE_ETR']
            ligne_25_values.append(10000000 + min(pub_non_liees, 2000000))
        else:
            ligne_25_values.append(10000000)  # Par défaut
    calc_columns['LIGNE_25_CALC'] = np.array(ligne_25_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_25_CALC'] = calc_columns['LIGNE_25_CALC']

    # Ligne 26: Total des dépenses de sous-traitance après plafonnements
    ligne_26_values = []
    for _, row in df_calc_temp.iterrows():
        total_avant_plaf = row['LIGNE_23_CALC'] + row['LIGNE_24_CALC']
        ligne_26_values.append(min(total_avant_plaf, row['LIGNE_25_CALC']))
    calc_columns['LIGNE_26_CALC'] = np.array(ligne_26_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_26_CALC'] = calc_columns['LIGNE_26_CALC']

    # 3. Montant total des dépenses de recherche

    # Ligne 27: Total dépenses recherche
    calc_columns['LIGNE_27_CALC'] = df_calc_temp['LIGNE_14_CALC'] + df_calc_temp['LIGNE_26_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_27_CALC'] = calc_columns['LIGNE_27_CALC']

    # Ligne 31a: Montant net total des dépenses de recherche
    calc_columns['LIGNE_31A_CALC'] = df_calc_temp['LIGNE_27_CALC'] - df_tmp['MT_AID_SUBV'] - \
                                  df_tmp['MT_ENC_PRESTA'] - df_tmp['MT_DEP_CONSEILS_CIR'] + \
                                  df_tmp['REMBST_SUBV']

    # Ligne 31b: Utiliser la valeur de MT_NET_DEP_RD_DOM directement
    calc_columns['LIGNE_31B_CALC'] = df_tmp['MT_NET_DEP_RD_DOM']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_31A_CALC'] = calc_columns['LIGNE_31A_CALC']
    df_calc_temp['LIGNE_31B_CALC'] = calc_columns['LIGNE_31B_CALC']

    ## II - DÉPENSES DE COLLECTION OUVRANT DROIT AU CRÉDIT D'IMPÔT
    ## ANNÉE CIVILE 2021

    # Ligne 33: Frais de défense des dessins et modèles (plafond 60 000 €)
    calc_columns['FRAIS_DEF_DESSIN_PLAFONNES'] = np.minimum(df_tmp['FRAIS_DEF_DESSIN'], 60000)

    # Ligne 34: Total des dépenses de collection
    calc_columns['LIGNE_34_CALC'] = df_tmp['FRAIS_COLL'] + calc_columns['FRAIS_DEF_DESSIN_PLAFONNES']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_34_CALC'] = calc_columns['LIGNE_34_CALC']

    # Ligne 38a: Montant net des dépenses de collection
    calc_columns['LIGNE_38A_CALC'] = df_calc_temp['LIGNE_34_CALC'] - df_tmp['MT_AID_SUBV_COLL'] - \
                                  df_tmp['MT_DEP_CONSEILS_CIR_COLL'] + df_tmp['REMBST_SUBV_COLL']

    # Ligne 38b: Utiliser la valeur de MT_NET_DEP_COLL_DOM directement
    calc_columns['LIGNE_38B_CALC'] = df_tmp['MT_NET_DEP_COLL_DOM']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_38A_CALC'] = calc_columns['LIGNE_38A_CALC']
    df_calc_temp['LIGNE_38B_CALC'] = calc_columns['LIGNE_38B_CALC']

    # Ligne 39a: Montant net total des dépenses de recherche et de collection
    calc_columns['LIGNE_39A_CALC'] = df_calc_temp['LIGNE_31A_CALC'] + df_calc_temp['LIGNE_38A_CALC']

    # Ligne 39b: Montant net total des dépenses de recherche et de collection DOM
    calc_columns['LIGNE_39B_CALC'] = df_calc_temp['LIGNE_31B_CALC'] + df_calc_temp['LIGNE_38B_CALC']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_39A_CALC'] = calc_columns['LIGNE_39A_CALC']
    df_calc_temp['LIGNE_39B_CALC'] = calc_columns['LIGNE_39B_CALC']

    ## III - CALCUL DU CRÉDIT D'IMPÔT AU TITRE DES DÉPENSES DE RECHERCHE ET DE COLLECTION

    # Identifier les entreprises <= 100M€
    calc_columns['DEPENSES_MOINS_100M'] = df_calc_temp['LIGNE_39A_CALC'] <= 100000000

    # Mise à jour du DataFrame temporaire
    df_calc_temp['DEPENSES_MOINS_100M'] = calc_columns['DEPENSES_MOINS_100M']

    # A. Dépenses <= 100 000 000 €

    # CIR Recherche pour entreprises <= 100M€ (ligne 41)
    ligne_41_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_41_values.append(max(0, (row['LIGNE_31A_CALC'] - row['LIGNE_31B_CALC']) * 0.3 +
                      row['LIGNE_31B_CALC'] * 0.5))
        else:
            ligne_41_values.append(0)
    calc_columns['LIGNE_41_CALC'] = np.array(ligne_41_values)

    # Ligne 42: Quote-part de crédit d'impôt recherche (mise à 0)
    calc_columns['LIGNE_42'] = np.zeros(len(df_tmp))

    # Ligne 43a: Montant du crédit d'impôt pour dépenses de recherche
    calc_columns['LIGNE_43A_CALC'] = np.maximum(calc_columns['LIGNE_41_CALC'] + calc_columns['LIGNE_42'], 0)

    # Ligne 43b: Montant du CIR pour dépenses de recherche DOM
    calc_columns['LIGNE_43B_CALC'] = np.maximum(df_calc_temp['LIGNE_31B_CALC'] * 0.5, 0)

    # Mise à jour du DataFrame temporaire
    for col in ['LIGNE_41_CALC', 'LIGNE_42', 'LIGNE_43A_CALC', 'LIGNE_43B_CALC']:
        df_calc_temp[col] = calc_columns[col]

    # CIR Collection pour entreprises <= 100M€ (ligne 45)
    ligne_45_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_45_values.append(max(0, (row['LIGNE_38A_CALC'] - row['LIGNE_38B_CALC']) * 0.3 +
                      row['LIGNE_38B_CALC'] * 0.5))
        else:
            ligne_45_values.append(0)
    calc_columns['LIGNE_45_CALC'] = np.array(ligne_45_values)

    # Ligne 46: Quote-part de crédit d'impôt collection (mise à 0)
    calc_columns['LIGNE_46'] = np.zeros(len(df_tmp))

    # Ligne 47a: Montant total du crédit d'impôt collection avant plafonnement
    calc_columns['LIGNE_47A_CALC'] = np.maximum(calc_columns['LIGNE_45_CALC'] + calc_columns['LIGNE_46'], 0)

    # Ligne 47b: Montant du crédit collection DOM avant plafonnement
    calc_columns['LIGNE_47B_CALC'] = np.maximum(df_calc_temp['LIGNE_38B_CALC'] * 0.5, 0)

    # Ligne 48: Montant des aides de minimis
    calc_columns['LIGNE_48'] = df_tmp['MT_AIDE_MINIMI']

    # Ligne 49: Montant cumulé du crédit d'impôt et des aides de minimis
    calc_columns['LIGNE_49'] = calc_columns['LIGNE_47A_CALC'] + calc_columns['LIGNE_48']

    # Mise à jour du DataFrame temporaire
    for col in ['LIGNE_45_CALC', 'LIGNE_46', 'LIGNE_47A_CALC', 'LIGNE_47B_CALC', 'LIGNE_48', 'LIGNE_49']:
        df_calc_temp[col] = calc_columns[col]

    # Ligne 50a: Montant du crédit d'impôt collection après plafonnement
    ligne_50a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['LIGNE_48'] >= 200000:  # Si aides de minimis déjà à 200k€
            ligne_50a_values.append(0)
        elif row['LIGNE_49'] < 200000:  # Si cumul < 200k€
            ligne_50a_values.append(max(0, row['LIGNE_47A_CALC']))
        else:  # Si dépassement
            ligne_50a_values.append(max(0, 200000 - row['LIGNE_48']))
    calc_columns['LIGNE_50A_CALC'] = np.array(ligne_50a_values)

    # Ligne 50b: Montant du crédit collection DOM après plafonnement
    calc_columns['LIGNE_50B_CALC'] = np.maximum(df_tmp['MT_CI_COLL_APRS_MINIMI_DOM'] if 'MT_CI_COLL_APRS_MINIMI_DOM' in df_tmp.columns else calc_columns['LIGNE_47B_CALC'], 0)

    # Ligne 51a: Montant total du crédit d'impôt recherche et collection
    calc_columns['LIGNE_51A_CALC'] = np.maximum(calc_columns['LIGNE_43A_CALC'] + calc_columns['LIGNE_50A_CALC'], 0)

    # Ligne 51b: Montant du crédit d'impôt recherche et collection DOM
    calc_columns['LIGNE_51B_CALC'] = np.maximum(calc_columns['LIGNE_43B_CALC'] + calc_columns['LIGNE_50B_CALC'], 0)

    # Mise à jour du DataFrame temporaire
    for col in ['LIGNE_50A_CALC', 'LIGNE_50B_CALC', 'LIGNE_51A_CALC', 'LIGNE_51B_CALC']:
        df_calc_temp[col] = calc_columns[col]

    # B. Dépenses > 100 000 000 €

    # Identifier les entreprises > 100M€
    calc_columns['DEPENSES_PLUS_100M'] = ~df_calc_temp['DEPENSES_MOINS_100M']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['DEPENSES_PLUS_100M'] = calc_columns['DEPENSES_PLUS_100M']

    # Ligne 52a: Limiter les dépenses recherche à 100M€
    ligne_52a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_52a_values.append(min(row['LIGNE_31A_CALC'], 100000000))
        else:
            ligne_52a_values.append(0)
    calc_columns['LIGNE_52A_CALC'] = np.array(ligne_52a_values)

    # Ligne 52b: Proportion DOM dans la limite des 100M€
    ligne_52b_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_52b_values.append(min(row['LIGNE_31B_CALC'], 100000000))
        else:
            ligne_52b_values.append(0)
    calc_columns['LIGNE_52B_CALC'] = np.array(ligne_52b_values)

    # Ligne 53: CIR recherche première tranche
    ligne_53_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_53_values.append(max(0, (calc_columns['LIGNE_52A_CALC'][_] - calc_columns['LIGNE_52B_CALC'][_]) * 0.3 +
                           calc_columns['LIGNE_52B_CALC'][_] * 0.5))
        else:
            ligne_53_values.append(0)
    calc_columns['LIGNE_53_CALC'] = np.array(ligne_53_values)

    # Ligne 54: Dépenses > 100M€
    ligne_54_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_54_values.append(max(0, row['LIGNE_31A_CALC'] - 100000000))
        else:
            ligne_54_values.append(0)
    calc_columns['LIGNE_54_CALC'] = np.array(ligne_54_values)

    # Ligne 55: CIR recherche deuxième tranche
    calc_columns['LIGNE_55_CALC'] = np.maximum(calc_columns['LIGNE_54_CALC'] * 0.05, 0)

    # Ligne 56: CIR recherche total pour entreprises > 100M€
    calc_columns['LIGNE_56_CALC'] = np.maximum(calc_columns['LIGNE_53_CALC'] + calc_columns['LIGNE_55_CALC'], 0)

    # Ligne 57: Quote-part recherche > 100M€ (mise à 0)
    calc_columns['LIGNE_57'] = np.zeros(len(df_tmp))

    # Ligne 58a: Total CIR recherche > 100M€
    calc_columns['LIGNE_58A_CALC'] = np.maximum(calc_columns['LIGNE_56_CALC'] + calc_columns['LIGNE_57'], 0)

    # Ligne 58b: CIR recherche DOM > 100M€
    calc_columns['LIGNE_58B_CALC'] = np.maximum(calc_columns['LIGNE_52B_CALC'] * 0.5, 0)

    # Mise à jour du DataFrame temporaire
    for col in ['LIGNE_52A_CALC', 'LIGNE_52B_CALC', 'LIGNE_53_CALC', 'LIGNE_54_CALC', 'LIGNE_55_CALC', 
               'LIGNE_56_CALC', 'LIGNE_57', 'LIGNE_58A_CALC', 'LIGNE_58B_CALC']:
        df_calc_temp[col] = calc_columns[col]

    # Collection pour > 100M€
    # Ligne 59a,59b: Dépenses collection
    calc_columns['LIGNE_59A_CALC'] = calc_columns['LIGNE_38A_CALC']
    calc_columns['LIGNE_59B_CALC'] = calc_columns['LIGNE_38B_CALC']

    # Ligne 60: Plafond disponible
    ligne_60_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_60_values.append(max(0, 100000000 - row['LIGNE_52A_CALC']))
        else:
            ligne_60_values.append(0)
    calc_columns['LIGNE_60_CALC'] = np.array(ligne_60_values)

    # Ligne 61: CIR collection première tranche
    ligne_61_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            dep_coll_plaf = min(calc_columns['LIGNE_59A_CALC'][_], calc_columns['LIGNE_60_CALC'][_])
            dep_coll_dom_plaf = min(calc_columns['LIGNE_59B_CALC'][_], calc_columns['LIGNE_60_CALC'][_])
            ligne_61_values.append(max(0, (dep_coll_plaf - dep_coll_dom_plaf) * 0.3 + dep_coll_dom_plaf * 0.5))
        else:
            ligne_61_values.append(0)
    calc_columns['LIGNE_61_CALC'] = np.array(ligne_61_values)

    # Ligne 62: CIR collection deuxième tranche
    ligne_62_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_PLUS_100M']:
            ligne_62_values.append(max(0, max(0, calc_columns['LIGNE_59A_CALC'][_] - calc_columns['LIGNE_60_CALC'][_]) * 0.05))
        else:
            ligne_62_values.append(0)
    calc_columns['LIGNE_62_CALC'] = np.array(ligne_62_values)

    # Ligne 63: CIR collection avant plafonnement
    calc_columns['LIGNE_63_CALC'] = np.maximum(calc_columns['LIGNE_61_CALC'] + calc_columns['LIGNE_62_CALC'], 0)

    # Ligne 64: Quote-part collection > 100M€ (mise à 0)
    calc_columns['LIGNE_64'] = np.zeros(len(df_tmp))

    # Ligne 65: Total collection avant plafonnement
    calc_columns['LIGNE_65'] = np.maximum(calc_columns['LIGNE_63_CALC'] + calc_columns['LIGNE_64'], 0)

    # Ligne 66: Aides de minimis > 100M€
    calc_columns['LIGNE_66'] = df_tmp['MT_AIDE_MINIMI']

    # Ligne 67: Cumul collection + aides
    calc_columns['LIGNE_67'] = calc_columns['LIGNE_65'] + calc_columns['LIGNE_66']

    # Mise à jour du DataFrame temporaire
    for col in ['LIGNE_59A_CALC', 'LIGNE_59B_CALC', 'LIGNE_60_CALC', 'LIGNE_61_CALC', 'LIGNE_62_CALC',
               'LIGNE_63_CALC', 'LIGNE_64', 'LIGNE_65', 'LIGNE_66', 'LIGNE_67']:
        df_calc_temp[col] = calc_columns[col]

    # Ligne 68a: Collection après plafonnement > 100M€
    ligne_68a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['LIGNE_66'] >= 200000:
            ligne_68a_values.append(0)
        elif row['LIGNE_67'] < 200000:
            ligne_68a_values.append(max(0, row['LIGNE_65']))
        else:
            ligne_68a_values.append(max(0, 200000 - row['LIGNE_66']))
    calc_columns['LIGNE_68A_CALC'] = np.array(ligne_68a_values)

    # Ligne 68b: Collection DOM après plafonnement > 100M€
    calc_columns['LIGNE_68B_CALC'] = np.maximum(df_tmp['MT_CI_COLL_APRS_MINIMI_DOM'] if 'MT_CI_COLL_APRS_MINIMI_DOM' in df_tmp.columns else calc_columns['LIGNE_59B_CALC'] * 0.5, 0)

    # Ligne 69a: Total CIR recherche + collection > 100M€
    calc_columns['LIGNE_69A_CALC'] = np.maximum(calc_columns['LIGNE_58A_CALC'] + calc_columns['LIGNE_68A_CALC'], 0)

    # Ligne 69b: Total CIR DOM > 100M€
    calc_columns['LIGNE_69B_CALC'] = np.maximum(calc_columns['LIGNE_58B_CALC'] + calc_columns['LIGNE_68B_CALC'], 0)

    # Mise à jour du DataFrame temporaire
    for col in ['LIGNE_68A_CALC', 'LIGNE_68B_CALC', 'LIGNE_69A_CALC', 'LIGNE_69B_CALC']:
        df_calc_temp[col] = calc_columns[col]

    # Somme du CIR recherche selon le cas
    cir_recherche_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            cir_recherche_values.append(row['LIGNE_43A_CALC'])
        else:
            cir_recherche_values.append(row['LIGNE_58A_CALC'])
    calc_columns['CIR_RECHERCHE_RECALCULE'] = np.maximum(np.array(cir_recherche_values), 0)

    # Somme du CIR collection après plafonnement selon le cas
    cir_collection_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            cir_collection_values.append(row['LIGNE_50A_CALC'])
        else:
            cir_collection_values.append(row['LIGNE_68A_CALC'])
    calc_columns['CIR_COLLECTION_RECALCULE'] = np.maximum(np.array(cir_collection_values), 0)

    # Mise à jour du DataFrame temporaire
    for col in ['CIR_RECHERCHE_RECALCULE', 'CIR_COLLECTION_RECALCULE']:
        df_calc_temp[col] = calc_columns[col]

    ## IV - DÉPENSES D'INNOVATION (CIR-INNOVATION)
    ## ANNÉE CIVILE 2021
    
    # Vérifier si les données d'innovation sont disponibles
    if all(col in df_tmp.columns for col in ['DOT_AMORT_IMMO_INO', 'DEP_PERSONEL_INO']):
        # Ligne 72: Autres dépenses de fonctionnement innovation (calcul spécifique 2021)
        calc_columns['LIGNE_72_CALC'] = (df_tmp['DOT_AMORT_IMMO_INO'] * 0.75) + (df_tmp['DEP_PERSONEL_INO'] * 0.43)

        # Total dépenses d'innovation (ligne 76)
        calc_columns['LIGNE_76_CALC'] = df_tmp['DOT_AMORT_IMMO_INO'] + df_tmp['DEP_PERSONEL_INO'] + \
                                     calc_columns['LIGNE_72_CALC'] + df_tmp['FRAIS_BREV_COV_INO'] + \
                                     df_tmp['FRAIS_DEF_BREV_INO'] + df_tmp['OP_INOV_EXT']

        # Plafonnement à 400 000 € (ligne 77)
        calc_columns['LIGNE_77_CALC'] = np.minimum(calc_columns['LIGNE_76_CALC'], 400000)

        # Montant net des dépenses d'innovation (ligne 82a)
        calc_columns['LIGNE_82A_CALC'] = calc_columns['LIGNE_77_CALC'] - df_tmp['MT_AID_SUBV_INO'] - \
                                       df_tmp['MT_ENC_PRESTA_INO'] - df_tmp['MT_DEP_CONSEILS_CII'] + \
                                       df_tmp['REMBST_SUBV_INO']

        # Calcul des parts DOM et Corse
        calc_columns['LIGNE_82B_CALC'] = df_tmp['MT_NET_DEP_INO_DOM']
        calc_columns['LIGNE_82C_CALC'] = df_tmp['MT_NET_DEP_INO_MPE_CORSE']
        calc_columns['LIGNE_82D_CALC'] = df_tmp['MT_NET_DEP_INO_ME_CORSE']

        # Mise à jour du DataFrame temporaire
        for col in ['LIGNE_72_CALC', 'LIGNE_76_CALC', 'LIGNE_77_CALC', 'LIGNE_82A_CALC',
                    'LIGNE_82B_CALC', 'LIGNE_82C_CALC', 'LIGNE_82D_CALC']:
            df_calc_temp[col] = calc_columns[col]

        # CIR Innovation selon les taux 2021 - taux de 20% (ligne 83)
        calc_columns['LIGNE_83_CALC'] = np.maximum(((df_calc_temp['LIGNE_82A_CALC'] - df_calc_temp['LIGNE_82B_CALC'] -
                                      df_calc_temp['LIGNE_82C_CALC'] - df_calc_temp['LIGNE_82D_CALC']) * 0.2) + \
                                     (df_calc_temp['LIGNE_82B_CALC'] * 0.4) + \
                                     (df_calc_temp['LIGNE_82C_CALC'] * 0.4) + \
                                     (df_calc_temp['LIGNE_82D_CALC'] * 0.35), 0)

        # Ligne 84: Quote-part innovation (mise à 0)
        calc_columns['LIGNE_84'] = np.zeros(len(df_tmp))

        # Ligne 85a: Total CIR innovation
        calc_columns['LIGNE_85A_CALC'] = np.maximum(calc_columns['LIGNE_83_CALC'] + calc_columns['LIGNE_84'], 0)

        # Ligne 85b: CIR innovation DOM
        calc_columns['LIGNE_85B_CALC'] = np.maximum(df_tmp['MT_CII_YC_QP_DOM'] if 'MT_CII_YC_QP_DOM' in df_tmp.columns else df_calc_temp['LIGNE_82B_CALC'] * 0.4, 0)

        # Ligne 85c: CIR innovation Corse
        calc_columns['LIGNE_85C_CALC'] = np.maximum(df_tmp['MT_CII_CORSE'] if 'MT_CII_CORSE' in df_tmp.columns else (df_calc_temp['LIGNE_82C_CALC'] * 0.4 + df_calc_temp['LIGNE_82D_CALC'] * 0.35), 0)

        calc_columns['CIR_INNOVATION_RECALCULE'] = np.maximum(calc_columns['LIGNE_85A_CALC'], 0)
    else:
        # Si données non disponibles, mettre à 0
        calc_columns['CIR_INNOVATION_RECALCULE'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_72_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_76_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_77_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_82A_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_82B_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_82C_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_82D_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_83_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_84'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_85A_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_85B_CALC'] = np.zeros(len(df_tmp))
        calc_columns['LIGNE_85C_CALC'] = np.zeros(len(df_tmp))

    # Mise à jour du DataFrame temporaire
    for col in ['CIR_INNOVATION_RECALCULE', 'LIGNE_72_CALC', 'LIGNE_76_CALC', 'LIGNE_77_CALC',
               'LIGNE_82A_CALC', 'LIGNE_82B_CALC', 'LIGNE_82C_CALC', 'LIGNE_82D_CALC',
               'LIGNE_83_CALC', 'LIGNE_84', 'LIGNE_85A_CALC', 'LIGNE_85B_CALC', 'LIGNE_85C_CALC']:
        if col in calc_columns:
            df_calc_temp[col] = calc_columns[col]

    # Ligne 86a: Total CIR recherche + collection + innovation
    ligne_86a_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_86a_values.append(max(0, row['LIGNE_51A_CALC'] + row['LIGNE_85A_CALC']))
        else:
            ligne_86a_values.append(max(0, row['LIGNE_69A_CALC'] + row['LIGNE_85A_CALC']))
    calc_columns['LIGNE_86A_CALC'] = np.array(ligne_86a_values)

    # Ligne 86b: Total CIR DOM
    ligne_86b_values = []
    for _, row in df_calc_temp.iterrows():
        if row['DEPENSES_MOINS_100M']:
            ligne_86b_values.append(max(0, row['LIGNE_51B_CALC'] + row['LIGNE_85B_CALC']))
        else:
            ligne_86b_values.append(max(0, row['LIGNE_69B_CALC'] + row['LIGNE_85B_CALC']))
    calc_columns['LIGNE_86B_CALC'] = np.array(ligne_86b_values)

    # Mise à jour du DataFrame temporaire
    df_calc_temp['LIGNE_86A_CALC'] = calc_columns['LIGNE_86A_CALC']
    df_calc_temp['LIGNE_86B_CALC'] = calc_columns['LIGNE_86B_CALC']

    ## TOTAL DU CRÉDIT D'IMPÔT FINAL (CIR + CII) - Pas de CRC en 2021

    # Total CIR recalculé
    calc_columns['CIR_TOTAL_RECALCULE'] = np.maximum(calc_columns['CIR_RECHERCHE_RECALCULE'] + \
                                          calc_columns['CIR_COLLECTION_RECALCULE'], 0)

    # Ajouter la composante Innovation (CII) si elle a été calculée
    if 'CIR_INNOVATION_RECALCULE' in calc_columns:
        calc_columns['CIR_TOTAL_RECALCULE'] = np.maximum(calc_columns['CIR_TOTAL_RECALCULE'] + np.nan_to_num(calc_columns['CIR_INNOVATION_RECALCULE']), 0)

    # Mettre à jour le DataFrame temporaire
    df_calc_temp['CIR_TOTAL_RECALCULE'] = calc_columns['CIR_TOTAL_RECALCULE']

    # ANALYSE DES ÉCARTS

    # Écarts sur toutes les lignes recalculées
    # 1. Dépenses internes
    calc_columns['ECART_LIGNE_6'] = calc_columns['LIGNE_6_CALC'] - df_tmp['OTR_DEP_FONCT']
    calc_columns['ECART_LIGNE_7'] = calc_columns['LIGNE_7_CALC'] - df_tmp['MT_DEP_FONCT_TOT']
    calc_columns['ECART_LIGNE_14'] = calc_columns['LIGNE_14_CALC'] - df_tmp['MT_TOT_RD_1']

    # 2. Dépenses de sous-traitance
    calc_columns['ECART_LIGNE_17'] = calc_columns['LIGNE_17_CALC'] - df_tmp['MT_TOT_ST_ORG_PUB']
    calc_columns['ECART_LIGNE_20'] = calc_columns['LIGNE_20_CALC'] - df_tmp['MT_TOT_ST_ORG_PRIV']
    calc_columns['ECART_LIGNE_21'] = calc_columns['LIGNE_21_CALC'] - df_tmp['PLAF_ST_ORG_PRIV']
    calc_columns['ECART_LIGNE_22'] = calc_columns['LIGNE_22_CALC'] - df_tmp['MT_TOT_ST']
    calc_columns['ECART_LIGNE_23'] = calc_columns['LIGNE_23_CALC'] - df_tmp['PLAF_ST_ORG_LIE']
    calc_columns['ECART_LIGNE_24'] = calc_columns['LIGNE_24_CALC'] - df_tmp['MT_ST_ORG_NON_LIE_PLAF']
    calc_columns['ECART_LIGNE_25'] = calc_columns['LIGNE_25_CALC'] - df_tmp['PLAF_GNRL_ST']
    calc_columns['ECART_LIGNE_26'] = calc_columns['LIGNE_26_CALC'] - df_tmp['MT_TOT_ST_PLAF']

    # 3. Montant total et net
    calc_columns['ECART_LIGNE_27'] = calc_columns['LIGNE_27_CALC'] - df_tmp['MT_TOT_RD_2']
    calc_columns['ECART_LIGNE_31A'] = calc_columns['LIGNE_31A_CALC'] - df_tmp['MT_NET_DEP_RD']
    calc_columns['ECART_LIGNE_31B'] = calc_columns['LIGNE_31B_CALC'] - df_tmp['MT_NET_DEP_RD_DOM']

    # 4. Dépenses de collection
    calc_columns['ECART_LIGNE_34'] = calc_columns['LIGNE_34_CALC'] - df_tmp['MT_TOT_DEP_COLL']
    calc_columns['ECART_LIGNE_38A'] = calc_columns['LIGNE_38A_CALC'] - df_tmp['MT_NET_DEP_COLL']
    calc_columns['ECART_LIGNE_38B'] = calc_columns['LIGNE_38B_CALC'] - df_tmp['MT_NET_DEP_COLL_DOM']

    # 5. Innovation
    if all(col in df_tmp.columns for col in ['MT_TOT_DEP_INO', 'MT_TOT_DEP_INO_PLAF', 'MT_NET_DEP_INO']):
        calc_columns['ECART_LIGNE_76'] = calc_columns['LIGNE_76_CALC'] - df_tmp['MT_TOT_DEP_INO']
        calc_columns['ECART_LIGNE_77'] = calc_columns['LIGNE_77_CALC'] - df_tmp['MT_TOT_DEP_INO_PLAF']
        calc_columns['ECART_LIGNE_82A'] = calc_columns['LIGNE_82A_CALC'] - df_tmp['MT_NET_DEP_INO']

    # 6. Crédits d'impôt
    calc_columns['ECART_CIR_RECHERCHE'] = calc_columns['CIR_RECHERCHE_RECALCULE'] - df_tmp['MT_CIR_RECH_YC_QP']
    calc_columns['ECART_CIR_COLLECTION'] = calc_columns['CIR_COLLECTION_RECALCULE'] - df_tmp['MT_CI_COLL_APRS_MINIMI']

    if 'CIR_INNOVATION_RECALCULE' in df_calc_temp.columns:
        calc_columns['ECART_CIR_INNOVATION'] = df_calc_temp['CIR_INNOVATION_RECALCULE'] - df_tmp['MT_CII_YC_QP']
    else:
        calc_columns['ECART_CIR_INNOVATION'] = -df_tmp['MT_CII_YC_QP']

    # Écart total CIR
    calc_columns['ECART_CIR'] = df_calc_temp['CIR_TOTAL_RECALCULE'] - df_tmp['MT_TOT_CIR_CI_COLL_CII']

    # Mise à jour du DataFrame temporaire
    df_calc_temp['ECART_CIR'] = calc_columns['ECART_CIR']

    # Indicateur de correspondance (avec seuil de tolérance de 1)
    correspondance_values = []
    for ecart in calc_columns['ECART_CIR']:
        if abs(ecart) <= 1:
            correspondance_values.append("Oui")
        else:
            correspondance_values.append("Non")
    calc_columns['CORRESPONDANCE'] = np.array(correspondance_values)

    # Écart relatif
    ecart_relatif_values = []
    for i, row in df_calc_temp.iterrows():
        if row['MT_TOT_CIR_CI_COLL_CII'] > 0:
            ecart_relatif_values.append((row['ECART_CIR'] / row['MT_TOT_CIR_CI_COLL_CII'] * 100))
        else:
            ecart_relatif_values.append(100 if row['ECART_CIR'] > 0 else 0)
    calc_columns['ECART_RELATIF'] = np.array(ecart_relatif_values)

    # Création du DataFrame final avec toutes les colonnes calculées
    df_calc_final = pd.DataFrame(calc_columns)

    # Création du DataFrame final pour l'analyse
    df_num = pd.concat([df_tmp, df_calc_final], axis=1)

    ## CRÉATION DU FICHIER DE COMPARAISON DÉTAILLÉ

    # Préparation du dictionnaire de mapping pour la sortie
    mapping_colonnes = {
        # Colonnes d'identification
        'siren_declarant': 'SIREN_DECLARANT',
        'siren_deposant': 'SIREN_DEPOSANT',
        'DESIGN': 'DESIGNATION',
        'COMPLT_DESIGN': 'COMPLEMENT_DESIGNATION',

        # I. Dépenses internes
        'DOT_AMORT_IMMO': 'L1_DOTATION_AMORT_IMMO',
        'DOT_AMORT_IMMO_SINISTR': 'L2_DOTATION_AMORT_SINISTR',
        'DEP_CHERCH_TECH': 'L3_DEPENSES_PERSONNEL_CHERCHEURS',
        'REM_SAL_INV': 'L4_REMUNERATION_INVENTEURS',
        'DEP_JD': 'L5_DEPENSES_JEUNES_DOCTEURS',
        'OTR_DEP_FONCT': 'L6_AUTRES_DEP_FONCT_DECLARE',
        'LIGNE_6_CALC': 'L6_AUTRES_DEP_FONCT_CALCULE',
        'ECART_LIGNE_6': 'L6_ECART',
        'MT_DEP_FONCT_TOT': 'L7_TOTAL_DEP_FONCT_DECLARE',
        'LIGNE_7_CALC': 'L7_TOTAL_DEP_FONCT_CALCULE',
        'ECART_LIGNE_7': 'L7_ECART',
        'FRAIS_BREV_COV': 'L8_FRAIS_BREVETS_COV',
        'DEP_MAINT_BREV_COV': 'L9_DEPENSES_DEFENSE_BREVETS',
        'DOT_AMORT_BREV': 'L10_DOTATION_AMORT_BREVETS',
        'DEP_NORMALI': 'L11_DEPENSES_NORMALISATION',
        'PRIM_COTIZ': 'L12_PRIMES_COTISATIONS_BRUT',
        'PRIM_COTIZ_PLAFONNEES': 'L12_PRIMES_COTISATIONS_PLAFONNEES',
        'DEP_VEIL_TECHNO': 'L13_VEILLE_TECHNO_BRUT',
        'DEP_VEIL_TECHNO_PLAFONNEES': 'L13_VEILLE_TECHNO_PLAFONNEE',
        'MT_TOT_RD_1': 'L14_TOTAL_DEPENSES_INTERNES_DECLARE',
        'LIGNE_14_CALC': 'L14_TOTAL_DEPENSES_INTERNES_CALCULE',
        'ECART_LIGNE_14': 'L14_ECART',

        # Sous-traitance organismes publics
        'DEP_ST_ORG_PUB_LIE_FR': 'L15A_ST_PUB_LIES_FR',
        'DEP_ST_ORG_PUB_LIE_ETR': 'L15B_ST_PUB_LIES_ETR',
        'DEP_ST_ORG_PUB_NON_LIE_FR': 'L16A_ST_PUB_NON_LIES_FR',
        'DEP_ST_ORG_PUB_NON_LIE_ETR': 'L16B_ST_PUB_NON_LIES_ETR',
        'MT_TOT_ST_ORG_PUB': 'L17_TOTAL_ST_PUBLIQUE_DECLARE',
        'LIGNE_17_CALC': 'L17_TOTAL_ST_PUBLIQUE_CALCULE',
        'ECART_LIGNE_17': 'L17_ECART',

        # Sous-traitance organismes privés
        'DEP_ST_ORG_PRIV_LIE_FR': 'L18A_ST_PRIV_LIES_FR',
        'DEP_ST_ORG_PRIV_LIE_ETR': 'L18B_ST_PRIV_LIES_ETR',
        'DEP_ST_ORG_PRIV_NON_LIE_FR': 'L19A_ST_PRIV_NON_LIES_FR',
        'DEP_ST_ORG_PRIV_NON_LIE_ETR': 'L19B_ST_PRIV_NON_LIES_ETR',
        'MT_TOT_ST_ORG_PRIV': 'L20_TOTAL_ST_PRIVEE_DECLARE',
        'LIGNE_20_CALC': 'L20_TOTAL_ST_PRIVEE_CALCULE',
        'ECART_LIGNE_20': 'L20_ECART',
        'PLAF_ST_ORG_PRIV': 'L21_PLAFOND_ST_PRIVEE_DECLARE',
        'LIGNE_21_CALC': 'L21_PLAFOND_ST_PRIVEE_CALCULE',
        'ECART_LIGNE_21': 'L21_ECART',
        'MT_TOT_ST': 'L22_TOTAL_ST_DECLARE',
        'LIGNE_22_CALC': 'L22_TOTAL_ST_CALCULE',
        'ECART_LIGNE_22': 'L22_ECART',
        'PLAF_ST_ORG_LIE': 'L23_PLAFOND_ORGANISMES_LIES_DECLARE',
        'LIGNE_23_CALC': 'L23_PLAFOND_ORGANISMES_LIES_CALCULE',
        'ECART_LIGNE_23': 'L23_ECART',
        'MT_ST_ORG_NON_LIE_PLAF': 'L24_PLAFOND_ORG_NON_LIES_DECLARE',
        'LIGNE_24_CALC': 'L24_PLAFOND_ORG_NON_LIES_CALCULE',
        'ECART_LIGNE_24': 'L24_ECART',
        'PLAF_GNRL_ST': 'L25_PLAFOND_GENERAL_DECLARE',
        'LIGNE_25_CALC': 'L25_PLAFOND_GENERAL_CALCULE',
        'ECART_LIGNE_25': 'L25_ECART',
        'MT_TOT_ST_PLAF': 'L26_TOTAL_ST_PLAFONNE_DECLARE',
        'LIGNE_26_CALC': 'L26_TOTAL_ST_PLAFONNE_CALCULE',
        'ECART_LIGNE_26': 'L26_ECART',

        # Montant total et net
        'MT_TOT_RD_2': 'L27_TOTAL_DEPENSES_RECHERCHE_DECLARE',
        'LIGNE_27_CALC': 'L27_TOTAL_DEPENSES_RECHERCHE_CALCULE',
        'ECART_LIGNE_27': 'L27_ECART',
        'MT_AID_SUBV': 'L28A_SUBVENTIONS',
        'MT_ENC_PRESTA': 'L28B_SOMMES_ENCAISSEES_TIERS',
        'MT_DEP_CONSEILS_CIR': 'L29_DEPENSES_CONSEIL_CIR',
        'REMBST_SUBV': 'L30_REMBOURSEMENTS_SUBVENTIONS',
        'MT_NET_DEP_RD': 'L31A_MONTANT_NET_DEPENSES_DECLARE',
        'LIGNE_31A_CALC': 'L31A_MONTANT_NET_DEPENSES_CALCULE',
        'ECART_LIGNE_31A': 'L31A_ECART',
        'MT_NET_DEP_RD_DOM': 'L31B_MONTANT_NET_DEPENSES_DOM_DECLARE',
        'LIGNE_31B_CALC': 'L31B_MONTANT_NET_DEPENSES_DOM_CALCULE',
        'ECART_LIGNE_31B': 'L31B_ECART',

        # II. Dépenses de collection
        'FRAIS_COLL': 'L32_FRAIS_COLLECTION',
        'FRAIS_DEF_DESSIN': 'L33_FRAIS_DEFENSE_DESSINS_BRUT',
        'FRAIS_DEF_DESSIN_PLAFONNES': 'L33_FRAIS_DEFENSE_DESSINS_PLAFONNES',
        'MT_TOT_DEP_COLL': 'L34_TOTAL_DEPENSES_COLLECTION_DECLARE',
        'LIGNE_34_CALC': 'L34_TOTAL_DEPENSES_COLLECTION_CALCULE',
        'ECART_LIGNE_34': 'L34_ECART',
        'MT_AID_SUBV_COLL': 'L35_SUBVENTIONS_COLLECTION',
        'MT_DEP_CONSEILS_CIR_COLL': 'L36_DEPENSES_CONSEIL_COLLECTION',
        'REMBST_SUBV_COLL': 'L37_REMBOURSEMENTS_SUBVENTIONS_COLL',
        'MT_NET_DEP_COLL': 'L38A_MONTANT_NET_COLLECTION_DECLARE',
        'LIGNE_38A_CALC': 'L38A_MONTANT_NET_COLLECTION_CALCULE',
        'ECART_LIGNE_38A': 'L38A_ECART',
        'MT_NET_DEP_COLL_DOM': 'L38B_MONTANT_NET_COLLECTION_DOM_DECLARE',
        'LIGNE_38B_CALC': 'L38B_MONTANT_NET_COLLECTION_DOM_CALCULE',
        'ECART_LIGNE_38B': 'L38B_ECART',
        'LIGNE_39A_CALC': 'L39A_MONTANT_NET_TOTAL_RD_COLL',
        'LIGNE_39B_CALC': 'L39B_MONTANT_NET_TOTAL_RD_COLL_DOM',

        # III. Calcul CIR <= 100M€
        'LIGNE_41_CALC': 'L41_CREDIT_IMPOT_RECHERCHE_MOINS_100M',
        'LIGNE_42': 'L42_QUOTE_PART_RECHERCHE_SOC_PERSONNES',
        'LIGNE_43A_CALC': 'L43A_CREDIT_IMPOT_RECHERCHE_TOTAL',
        'LIGNE_43B_CALC': 'L43B_CREDIT_IMPOT_RECHERCHE_DOM',
        'LIGNE_45_CALC': 'L45_CREDIT_IMPOT_COLLECTION_MOINS_100M',
        'LIGNE_46': 'L46_QUOTE_PART_COLLECTION_SOC_PERSONNES',
        'LIGNE_47A_CALC': 'L47A_CREDIT_IMPOT_COLL_AVANT_PLAF',
        'LIGNE_47B_CALC': 'L47B_CREDIT_IMPOT_COLL_DOM_AVANT_PLAF',
        'LIGNE_48': 'L48_AIDES_MINIMIS',
        'LIGNE_49': 'L49_CUMUL_CREDIT_IMPOT_ET_AIDES',
        'LIGNE_50A_CALC': 'L50A_CREDIT_IMPOT_COLL_APRES_PLAF',
        'LIGNE_50B_CALC': 'L50B_CREDIT_IMPOT_COLL_DOM_APRES_PLAF',
        'LIGNE_51A_CALC': 'L51A_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION',
        'LIGNE_51B_CALC': 'L51B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM',

        # III. Calcul CIR > 100M€
        'LIGNE_52A_CALC': 'L52A_DEPENSES_RECHERCHE_LIMITE_100M',
        'LIGNE_52B_CALC': 'L52B_DEPENSES_RECHERCHE_DOM_LIMITE',
        'LIGNE_53_CALC': 'L53_CIR_RECHERCHE_PREMIERE_TRANCHE',
        'LIGNE_54_CALC': 'L54_DEPENSES_RECHERCHE_SUP_100M',
        'LIGNE_55_CALC': 'L55_CIR_RECHERCHE_DEUXIEME_TRANCHE',
        'LIGNE_56_CALC': 'L56_CIR_RECHERCHE_PLUS_100M',
        'LIGNE_57': 'L57_QUOTE_PART_RECHERCHE_SOC_PERSONNES_PLUS_100M',
        'LIGNE_58A_CALC': 'L58A_CREDIT_IMPOT_RECHERCHE_TOTAL_PLUS_100M',
        'LIGNE_58B_CALC': 'L58B_CREDIT_IMPOT_RECHERCHE_DOM_PLUS_100M',
        'LIGNE_59A_CALC': 'L59A_MONTANT_NET_COLLECTION_PLUS_100M',
        'LIGNE_59B_CALC': 'L59B_MONTANT_NET_COLLECTION_DOM_PLUS_100M',
        'LIGNE_60_CALC': 'L60_PLAFOND_DISPO_COLLECTION',
        'LIGNE_61_CALC': 'L61_CIR_COLLECTION_PREMIERE_TRANCHE',
        'LIGNE_62_CALC': 'L62_CIR_COLLECTION_DEUXIEME_TRANCHE',
        'LIGNE_63_CALC': 'L63_CIR_COLLECTION_PLUS_100M',
        'LIGNE_64': 'L64_QUOTE_PART_COLLECTION_SOC_PERSONNES_PLUS_100M',
        'LIGNE_65': 'L65_CREDIT_IMPOT_COLL_AVANT_PLAF_PLUS_100M',
        'LIGNE_66': 'L66_AIDES_MINIMIS_PLUS_100M',
        'LIGNE_67': 'L67_CUMUL_CREDIT_IMPOT_ET_AIDES_PLUS_100M',
        'LIGNE_68A_CALC': 'L68A_CREDIT_IMPOT_COLL_APRES_PLAF_PLUS_100M',
        'LIGNE_68B_CALC': 'L68B_CREDIT_IMPOT_COLL_DOM_APRES_PLAF_PLUS_100M',
        'LIGNE_69A_CALC': 'L69A_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_PLUS_100M',
        'LIGNE_69B_CALC': 'L69B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM_PLUS_100M',

        # IV. Dépenses d'innovation
        'DOT_AMORT_IMMO_INO': 'L70_DOTATION_AMORT_IMMO_INNOVATION',
        'DEP_PERSONEL_INO': 'L71_DEPENSES_PERSONNEL_INNOVATION',
        'LIGNE_72_CALC': 'L72_AUTRES_DEPENSES_FONCT_INNOVATION',
        'FRAIS_BREV_COV_INO': 'L73_FRAIS_BREVETS_INNOVATION',
        'FRAIS_DEF_BREV_INO': 'L74_FRAIS_DEFENSE_BREVETS_INNOVATION',
        'OP_INOV_EXT': 'L75_OPERATIONS_CONFIEES_INNOVATION',
        'MT_TOT_DEP_INO': 'L76_TOTAL_DEPENSES_INNOVATION_DECLARE',
        'LIGNE_76_CALC': 'L76_TOTAL_DEPENSES_INNOVATION_CALCULE',
        'ECART_LIGNE_76': 'L76_ECART',
        'MT_TOT_DEP_INO_PLAF': 'L77_DEPENSES_INNOVATION_PLAFONNEES_DECLARE',
        'LIGNE_77_CALC': 'L77_DEPENSES_INNOVATION_PLAFONNEES_CALCULE',
        'ECART_LIGNE_77': 'L77_ECART',
        'MT_AID_SUBV_INO': 'L78_SUBVENTIONS_INNOVATION',
        'MT_ENC_PRESTA_INO': 'L79_PRESTATIONS_INNOVATION',
        'MT_DEP_CONSEILS_CII': 'L80_DEPENSES_CONSEIL_INNOVATION',
        'REMBST_SUBV_INO': 'L81_REMBOURSEMENTS_SUBVENTIONS_INNO',
        'MT_NET_DEP_INO': 'L82A_MONTANT_NET_INNOVATION_DECLARE',
        'LIGNE_82A_CALC': 'L82A_MONTANT_NET_INNOVATION_CALCULE',
        'ECART_LIGNE_82A': 'L82A_ECART',
        'MT_NET_DEP_INO_DOM': 'L82B_MONTANT_NET_INNOVATION_DOM',
        'MT_NET_DEP_INO_MPE_CORSE': 'L82C_MONTANT_NET_INNOVATION_CORSE_MPE',
        'MT_NET_DEP_INO_ME_CORSE': 'L82D_MONTANT_NET_INNOVATION_CORSE_ME',
        'LIGNE_83_CALC': 'L83_CREDIT_IMPOT_INNOVATION',
        'LIGNE_84': 'L84_QUOTE_PART_INNOVATION_SOC_PERSONNES',
        'LIGNE_85A_CALC': 'L85A_TOTAL_CREDIT_IMPOT_INNOVATION',
        'LIGNE_85B_CALC': 'L85B_CREDIT_IMPOT_INNOVATION_DOM',
        'LIGNE_85C_CALC': 'L85C_CREDIT_IMPOT_INNOVATION_CORSE',
        'LIGNE_86A_CALC': 'L86A_TOTAL_CIR_RECH_COLL_INNO',
        'LIGNE_86B_CALC': 'L86B_TOTAL_CIR_RECH_COLL_INNO_DOM',

        # Crédits d'impôt par composante
        'MT_CIR_RECH_YC_QP': 'CIR_RECHERCHE_DECLARE',
        'CIR_RECHERCHE_RECALCULE': 'CIR_RECHERCHE_CALCULE',
        'ECART_CIR_RECHERCHE': 'CIR_RECHERCHE_ECART',

        'MT_CI_COLL_APRS_MINIMI': 'CIR_COLLECTION_DECLARE',
        'CIR_COLLECTION_RECALCULE': 'CIR_COLLECTION_CALCULE',
        'ECART_CIR_COLLECTION': 'CIR_COLLECTION_ECART',

        'MT_CII_YC_QP': 'CIR_INNOVATION_DECLARE',
        'CIR_INNOVATION_RECALCULE': 'CIR_INNOVATION_CALCULE',
        'ECART_CIR_INNOVATION': 'CIR_INNOVATION_ECART',

        # CIR Total
        'MT_TOT_CIR_CI_COLL_CII': 'CIR_TOTAL_DECLARE',
        'CIR_TOTAL_RECALCULE': 'CIR_TOTAL_CALCULE',
        'ECART_CIR': 'CIR_TOTAL_ECART',

        # Indicateurs
        'CORRESPONDANCE': 'CORRESPONDANCE_CIR',
        'ECART_RELATIF': 'ECART_RELATIF_POURCENT'
    }

    # Création du DataFrame de comparaison
    df_comparaison = pd.DataFrame()

    # Copier chaque colonne si elle existe
    for col_orig, col_dest in mapping_colonnes.items():
        if col_orig in df_num.columns:
            df_comparaison[col_dest] = df_num[col_orig]

    # Concaténation de la désignation et du complément si disponible
    if 'COMPLEMENT_DESIGNATION' in df_comparaison.columns:
        df_comparaison['DESIGNATION'] = df_comparaison.apply(
            lambda x: f"{x['DESIGNATION']} {x['COMPLEMENT_DESIGNATION']}" if pd.notna(x['COMPLEMENT_DESIGNATION']) else x['DESIGNATION'],
            axis=1
        )
        df_comparaison.drop('COMPLEMENT_DESIGNATION', axis=1, inplace=True)

    # Formatage des montants
    montant_cols = [col for col in df_comparaison.columns if col not in ['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'DESIGNATION', 'CORRESPONDANCE_CIR']]
    for col in montant_cols:
        df_comparaison[col] = df_comparaison[col].round(2)

    # Sauvegarde du fichier de sortie
    output_filename = f"Calcul_Creance_CIR_2021.csv"
    output_path = os.path.join("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB", output_filename)

    df_comparaison.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')

    print(f"\nFichier de comparaison détaillée créé: {output_path}")

    # RÉSULTATS

    # Somme des CIR déclarés
    cir_declare_total = df_tmp['MT_TOT_CIR_CI_COLL_CII'].sum()

    # Somme des CIR recalculés
    cir_recalcule_total = df_calc_final['CIR_TOTAL_RECALCULE'].sum()

    # Nombre d'entreprises
    nb_entreprises_avec_cir = len(df_tmp[df_tmp['MT_TOT_CIR_CI_COLL_CII'] > 0])
    nb_entreprises_plus_100m = df_calc_final['DEPENSES_PLUS_100M'].sum()

    # Classement des écarts
    ecarts_positifs = df_num[df_num['ECART_CIR'] > 1]
    ecarts_negatifs = df_num[df_num['ECART_CIR'] < -1]
    ecarts_conformes = df_num[abs(df_num['ECART_CIR']) <= 1]

    # AFFICHAGE DES RÉSULTATS
    print("\n===== COMPARAISON CIR DÉCLARÉ VS RECALCULÉ 2021 =====")

    print(f"\nNombre total d'entreprises analysées: {len(df_num):,}")
    print(f"Nombre d'entreprises avec CIR déclaré: {nb_entreprises_avec_cir:,}")
    print(f"Nombre d'entreprises avec dépenses > 100M€: {nb_entreprises_plus_100m:,}")

    print(f"\nCIR TOTAL:")
    print(f"CIR déclaré total: {cir_declare_total:,.2f} €")
    print(f"CIR recalculé total: {cir_recalcule_total:,.2f} €")

    difference = cir_recalcule_total - cir_declare_total
    print(f"Différence: {difference:,.2f} €")
    if cir_declare_total > 0:
        print(f"Écart relatif: {difference/cir_declare_total*100:.2f}%")

    print(f"\nANALYSE DES ÉCARTS:")
    print(f"Entreprises avec CIR conforme (écart ≤ 1€): {len(ecarts_conformes):,} ({len(ecarts_conformes)/len(df_num)*100:.2f}%)")

    print(f"Entreprises avec CIR recalculé > CIR déclaré (écart > 1€): {len(ecarts_positifs):,}")
    print(f"Montant total des écarts positifs: {ecarts_positifs['ECART_CIR'].sum():,.2f} €")

    print(f"Entreprises avec CIR recalculé < CIR déclaré (écart < -1€): {len(ecarts_negatifs):,}")
    print(f"Montant total des écarts négatifs: {ecarts_negatifs['ECART_CIR'].sum():,.2f} €")

    # Détail par composante du CIR
    print(f"\nDÉTAIL PAR COMPOSANTE DU CIR RECALCULÉ:")
    print(f"CIR Recherche: {df_num['CIR_RECHERCHE_RECALCULE'].sum():,.2f} €")
    print(f"CIR Collection: {df_num['CIR_COLLECTION_RECALCULE'].sum():,.2f} €")

    if 'CIR_INNOVATION_RECALCULE' in df_num.columns:
        print(f"CIR Innovation: {df_num['CIR_INNOVATION_RECALCULE'].sum():,.2f} €")
    else:
        print(f"CIR Innovation: 0.00 €")

    return df_comparaison

# Exécuter le calcul
try:
    df_resultat = comparer_cir_declare_recalcule_2021()
    print("\nTraitement terminé avec succès!")
except Exception as e:
    print(f"\nErreur: {type(e).__name__}: {str(e)}")

Nombre total d'entreprises: 4,493


C:\Users\msamb\AppData\Local\Temp\ipykernel_928\1750787560.py:879: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_comparaison[col_dest] = df_num[col_orig]
C:\Users\msamb\AppData\Local\Temp\ipykernel_928\1750787560.py:879: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_comparaison[col_dest] = df_num[col_orig]
C:\Users\msamb\AppData\Local\Temp\ipykernel_928\1750787560.py:879: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consid


Fichier de comparaison détaillée créé: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB\Calcul_Creance_CIR_2021.csv

===== COMPARAISON CIR DÉCLARÉ VS RECALCULÉ 2021 =====

Nombre total d'entreprises analysées: 4,493
Nombre d'entreprises avec CIR déclaré: 1,435
Nombre d'entreprises avec dépenses > 100M€: 3

CIR TOTAL:
CIR déclaré total: 470,488,283.00 €
CIR recalculé total: 453,725,388.33 €
Différence: -16,762,894.67 €
Écart relatif: -3.56%

ANALYSE DES ÉCARTS:
Entreprises avec CIR conforme (écart ≤ 1€): 3,073 (68.40%)
Entreprises avec CIR recalculé > CIR déclaré (écart > 1€): 839
Montant total des écarts positifs: 43,788,357.41 €
Entreprises avec CIR recalculé < CIR déclaré (écart < -1€): 581
Montant total des écarts négatifs: -60,551,253.01 €

DÉTAIL PAR COMPOSANTE DU CIR RECALCULÉ:
CIR Recherche: 414,591,903.05 €
CIR Collection: 1,544,332.70 €
CIR Innovation: 37,589,152.58 €

Traitement terminé avec succès!


## Erreur depense personnel

In [17]:
import pandas as pd
import numpy as np

# Charger le fichier original pour 2021
df = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv", sep=';', encoding='utf-8-sig')
print(f"Fichier CIR 2021 chargé: {len(df)} lignes")

# Identifier les lignes concernées (ajout du critère correspondance)
mask = (
    (df['L3_DEPENSES_PERSONNEL_CHERCHEURS'] == 0.0) &
    (df['L2_DOTATION_AMORT_SINISTR'] != 0.0) &
    (df['CORRESPONDANCE_CIR'] == "Non")
)
indices_to_update = df[mask].index
print(f"Nombre de lignes à corriger: {len(indices_to_update)}")

# Sauvegarder les valeurs originales pour analyse
cir_calcule_avant = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].copy()

# Pour chaque ligne à corriger
for idx in indices_to_update:
    # 1. Transférer la valeur de dotation aux amortissements sinistrés vers dépenses de personnel
    val_amort_sinistr = df.at[idx, 'L2_DOTATION_AMORT_SINISTR']
    df.at[idx, 'L3_DEPENSES_PERSONNEL_CHERCHEURS'] = val_amort_sinistr
    df.at[idx, 'L2_DOTATION_AMORT_SINISTR'] = 0.0

    # 2. Recalculer la ligne 72 (autres dépenses de fonctionnement innovation - spécifique 2021)
    if 'L72_AUTRES_DEPENSES_FONCT_INNOVATION' in df.columns:
        dot_amort_immo_ino = df.at[idx, 'L70_DOTATION_AMORT_IMMO_INNOVATION']
        dep_personnel_ino = df.at[idx, 'L71_DEPENSES_PERSONNEL_INNOVATION']
        autres_dep_fonct_ino = (dot_amort_immo_ino * 0.75) + (dep_personnel_ino * 0.43)
        df.at[idx, 'L72_AUTRES_DEPENSES_FONCT_INNOVATION'] = autres_dep_fonct_ino

    # 3. Recalculer la ligne 6 (autres dépenses de fonctionnement recherche)
    dot_amort_immo = df.at[idx, 'L1_DOTATION_AMORT_IMMO']
    dep_cherch_tech = df.at[idx, 'L3_DEPENSES_PERSONNEL_CHERCHEURS']  # Nouvelle valeur
    rem_sal_inv = df.at[idx, 'L4_REMUNERATION_INVENTEURS']
    dep_jd = df.at[idx, 'L5_DEPENSES_JEUNES_DOCTEURS']

    autres_dep_fonct = (dot_amort_immo * 0.75) + ((dep_cherch_tech + rem_sal_inv) * 0.43) + dep_jd
    df.at[idx, 'L6_AUTRES_DEP_FONCT_CALCULE'] = autres_dep_fonct

    # 4. Recalculer la ligne 7 (total dépenses de fonctionnement)
    total_dep_fonct = dot_amort_immo + 0.0 + dep_cherch_tech + rem_sal_inv + dep_jd + autres_dep_fonct
    df.at[idx, 'L7_TOTAL_DEP_FONCT_CALCULE'] = total_dep_fonct

    # 5. Recalculer la ligne 14 (total dépenses internes)
    frais_brev_cov = df.at[idx, 'L8_FRAIS_BREVETS_COV']
    dep_maint_brev_cov = df.at[idx, 'L9_DEPENSES_DEFENSE_BREVETS']
    dot_amort_brev = df.at[idx, 'L10_DOTATION_AMORT_BREVETS']
    dep_normali = df.at[idx, 'L11_DEPENSES_NORMALISATION']
    
    # Plafonnements spécifiques 2021
    prim_cotiz_brut = df.at[idx, 'L12_PRIMES_COTISATIONS_BRUT'] if 'L12_PRIMES_COTISATIONS_BRUT' in df.columns else 0
    prim_cotiz = min(prim_cotiz_brut, 60000)
    
    dep_veil_techno_brut = df.at[idx, 'L13_VEILLE_TECHNO_BRUT'] if 'L13_VEILLE_TECHNO_BRUT' in df.columns else 0
    dep_veil_techno = min(dep_veil_techno_brut, 60000)

    total_dep_internes = total_dep_fonct + frais_brev_cov + dep_maint_brev_cov + dot_amort_brev + dep_normali + prim_cotiz + dep_veil_techno
    df.at[idx, 'L14_TOTAL_DEPENSES_INTERNES_CALCULE'] = total_dep_internes

    # 6. Recalculer la ligne 27 (total dépenses de recherche) - 2021 utilise L26 pour ST plafonnée
    total_dep_st_plafonnee = df.at[idx, 'L26_TOTAL_ST_PLAFONNE_CALCULE'] if 'L26_TOTAL_ST_PLAFONNE_CALCULE' in df.columns else 0
    total_dep_recherche = total_dep_internes + total_dep_st_plafonnee
    df.at[idx, 'L27_TOTAL_DEPENSES_RECHERCHE_CALCULE'] = total_dep_recherche

    # 7. Recalculer la ligne 31A (montant net des dépenses)
    mt_aid_subv = df.at[idx, 'L28A_SUBVENTIONS'] if 'L28A_SUBVENTIONS' in df.columns else 0
    mt_enc_presta = df.at[idx, 'L28B_SOMMES_ENCAISSEES_TIERS'] if 'L28B_SOMMES_ENCAISSEES_TIERS' in df.columns else 0
    mt_dep_conseils_cir = df.at[idx, 'L29_DEPENSES_CONSEIL_CIR'] if 'L29_DEPENSES_CONSEIL_CIR' in df.columns else 0
    rembst_subv = df.at[idx, 'L30_REMBOURSEMENTS_SUBVENTIONS'] if 'L30_REMBOURSEMENTS_SUBVENTIONS' in df.columns else 0

    montant_net = total_dep_recherche - mt_aid_subv - mt_enc_presta - mt_dep_conseils_cir + rembst_subv
    df.at[idx, 'L31A_MONTANT_NET_DEPENSES_CALCULE'] = montant_net

    # 8. Recalculer le CIR recherche (avec gestion DOM/non-DOM pour 2021)
    montant_net_dom = df.at[idx, 'L31B_MONTANT_NET_DEPENSES_DOM_CALCULE'] if 'L31B_MONTANT_NET_DEPENSES_DOM_CALCULE' in df.columns else 0
    
    # Taux 2021 : 30% métropole, 50% DOM
    taux_dom = 0.50
    taux_non_dom = 0.30
    
    if montant_net_dom > 0:
        montant_net_non_dom = max(0, montant_net - montant_net_dom)
        cir_recherche = (montant_net_non_dom * taux_non_dom) + (montant_net_dom * taux_dom)
    else:
        cir_recherche = montant_net * taux_non_dom
    
    # S'assurer que le CIR recherche n'est pas négatif
    cir_recherche = max(0, cir_recherche)
    df.at[idx, 'CIR_RECHERCHE_CALCULE'] = cir_recherche

    # 9. Recalculer le CIR total (sans CRC pour 2021)
    cir_collection = df.at[idx, 'CIR_COLLECTION_CALCULE'] if 'CIR_COLLECTION_CALCULE' in df.columns else 0
    cir_innovation = df.at[idx, 'CIR_INNOVATION_CALCULE'] if 'CIR_INNOVATION_CALCULE' in df.columns else 0
    
    # Pas de CRC en 2021
    cir_total = cir_recherche + cir_collection + cir_innovation
    
    # S'assurer que le CIR total n'est pas négatif
    cir_total = max(0, cir_total)
    df.at[idx, 'CIR_TOTAL_CALCULE'] = cir_total

    # 10. Recalculer l'écart et mettre à jour la correspondance
    cir_declare = df.at[idx, 'CIR_TOTAL_DECLARE']
    ecart_cir = cir_total - cir_declare
    df.at[idx, 'CIR_TOTAL_ECART'] = ecart_cir
    
    # Écart relatif
    if abs(cir_declare) > 0.01:
        ecart_relatif = (ecart_cir / cir_declare) * 100
    else:
        ecart_relatif = 100 if ecart_cir > 0 else 0
    df.at[idx, 'ECART_RELATIF_POURCENT'] = ecart_relatif
    
    # Correspondance (seuil de tolérance de 1€)
    df.at[idx, 'CORRESPONDANCE_CIR'] = "Oui" if abs(ecart_cir) <= 1 else "Non"

# Afficher les résultats
somme_declare = df.loc[indices_to_update, 'CIR_TOTAL_DECLARE'].sum()
somme_calcule_avant = cir_calcule_avant.sum()
somme_calcule_apres = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].sum()

print(f"\nRésultats de la correction CIR 2021:")
print(f"Somme CIR_TOTAL_DECLARE: {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE avant correction: {somme_calcule_avant:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE après correction: {somme_calcule_apres:,.2f} €")
print(f"Écart avant correction: {somme_calcule_avant - somme_declare:,.2f} €")
print(f"Écart après correction: {somme_calcule_apres - somme_declare:,.2f} €")

# Vérifier le nombre de correspondances après correction
nouvelles_correspondances = df.loc[indices_to_update, 'CORRESPONDANCE_CIR'].value_counts()
print(f"\nNombre de correspondances après correction:")
for correspondance, count in nouvelles_correspondances.items():
    print(f"  - {correspondance}: {count}")

# Détail par composante CIR après correction
if len(indices_to_update) > 0:
    print(f"\nDétail par composante CIR recalculé (lignes corrigées):")
    total_recherche = df.loc[indices_to_update, 'CIR_RECHERCHE_CALCULE'].sum()
    total_collection = df.loc[indices_to_update, 'CIR_COLLECTION_CALCULE'].sum() if 'CIR_COLLECTION_CALCULE' in df.columns else 0
    total_innovation = df.loc[indices_to_update, 'CIR_INNOVATION_CALCULE'].sum() if 'CIR_INNOVATION_CALCULE' in df.columns else 0
    
    print(f"  - CIR Recherche: {total_recherche:,.2f} €")
    print(f"  - CIR Collection: {total_collection:,.2f} €")
    print(f"  - CIR Innovation: {total_innovation:,.2f} €")
    print(f"  - TOTAL: {total_recherche + total_collection + total_innovation:,.2f} €")

# Sauvegarder le fichier corrigé pour 2021
output_filename = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//lignes_restantes_a_analyser_corrigees.csv"
df.to_csv(output_filename, sep=';', encoding='utf-8-sig', index=False)
print(f"\nFichier CIR 2021 corrigé sauvegardé: {output_filename}")

# Statistiques finales
total_correspondances_finales = df['CORRESPONDANCE_CIR'].value_counts()
print(f"\nStatistiques finales du fichier complet:")
for correspondance, count in total_correspondances_finales.items():
    print(f"  - {correspondance}: {count:,} lignes")

print(f"\nAméliorations apportées:")
ecart_avant = abs(somme_calcule_avant - somme_declare)
ecart_apres = abs(somme_calcule_apres - somme_declare)
if ecart_avant > 0:
    reduction_ecart = ((ecart_avant - ecart_apres) / ecart_avant) * 100
    print(f"  - Réduction de l'écart absolu: {ecart_avant - ecart_apres:,.2f} € ({reduction_ecart:.1f}%)")
else:
    print(f"  - Écart initial déjà nul")

print(f"\nCorrection terminée pour le millésime 2021.")

Fichier CIR 2021 chargé: 4493 lignes
Nombre de lignes à corriger: 2

Résultats de la correction CIR 2021:
Somme CIR_TOTAL_DECLARE: 43,074.00 €
Somme CIR_TOTAL_CALCULE avant correction: 117,773.06 €
Somme CIR_TOTAL_CALCULE après correction: 131,007.81 €
Écart avant correction: 74,699.06 €
Écart après correction: 87,933.81 €

Nombre de correspondances après correction:
  - Non: 2

Détail par composante CIR recalculé (lignes corrigées):
  - CIR Recherche: 48,305.73 €
  - CIR Collection: 0.00 €
  - CIR Innovation: 82,702.08 €
  - TOTAL: 131,007.81 €

Fichier CIR 2021 corrigé sauvegardé: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//lignes_restantes_a_analyser_corrigees.csv

Statistiques finales du fichier complet:
  - Oui: 3,073 lignes
  - Non: 1,420 lignes

Améliorations apportées:
  - Réduction de l'écart absolu: -13,234.75 € (-17.7%)

Correction terminée pour le millésime 2021.


### mise à jour ligne restante

In [18]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv", sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Appliquer les filtres :
# - L5_DEPENSES_JEUNES_DOCTEURS = 0.0
# - L2_DOTATION_AMORT_SINISTR différent de 0.0
df_filtered = df[
    (df['L3_DEPENSES_PERSONNEL_CHERCHEURS'] == 0.0) &
    (df['L2_DOTATION_AMORT_SINISTR'] != 0.0) &
    (df['CORRESPONDANCE_CIR'] == "Non") 
]

print(f"Nombre de lignes après filtrage: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_jeunes_docteurs_0_amort_sinistr_non0.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

# Afficher quelques statistiques sur les lignes trouvées
if len(df_filtered) > 0:
    print("\nAperçu des données trouvées:")
    print(f"Nombre d'entreprises: {len(df_filtered)}")

Fichier chargé: 4493 lignes
Nombre de lignes après filtrage: 2
Somme CIR_TOTAL_DECLARE : 43,074.00 €
Somme CIR_TOTAL_CALCULE : 117,773.06 €
Résultats exportés dans: lignes_jeunes_docteurs_0_amort_sinistr_non0.csv

Aperçu des données trouvées:
Nombre d'entreprises: 2


In [19]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')  # Nouveau fichier

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])  # Nouveau set

# Combiner tous les SIREN à exclure
siren_a_exclure =  siren_jeunes_docteurs  # Ajout du nouveau set

# Filtrer le DataFrame principal
# - Exclure les SIREN déjà analysés
# - Garder uniquement les lignes où CORRESPONDANCE_CIR est "Non"
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) & 
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 1418
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur DEP Fonctionnement

In [20]:
import pandas as pd
import numpy as np

# Charger le fichier original
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Identifier les lignes à corriger
mask = (df['L7_TOTAL_DEP_FONCT_DECLARE'] == df['L8_FRAIS_BREVETS_COV']) & (df['L7_TOTAL_DEP_FONCT_DECLARE'] != 0.0)
indices_to_update = df[mask].index
print(f"Nombre de lignes à corriger: {len(indices_to_update)}")

# Sauvegarder les valeurs originales pour analyse
cir_calcule_avant = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].copy()
l7_original = df.loc[indices_to_update, 'L7_TOTAL_DEP_FONCT_DECLARE'].copy()

# Pour chaque ligne à corriger
for idx in indices_to_update:
    # 1. Calculer correctement le montant de la ligne 7 (total dépenses de fonctionnement)
    dot_amort_immo = df.at[idx, 'L1_DOTATION_AMORT_IMMO']
    dot_amort_sinistr = df.at[idx, 'L2_DOTATION_AMORT_SINISTR']
    dep_cherch_tech = df.at[idx, 'L3_DEPENSES_PERSONNEL_CHERCHEURS']
    rem_sal_inv = df.at[idx, 'L4_REMUNERATION_INVENTEURS']
    dep_jd = df.at[idx, 'L5_DEPENSES_JEUNES_DOCTEURS']

    # Calculer L6 si nécessaire
    autres_dep_fonct = (dot_amort_immo * 0.75) + ((dep_cherch_tech + rem_sal_inv) * 0.43) + dep_jd

    # Mettre à jour L6 calculé
    df.at[idx, 'L6_AUTRES_DEP_FONCT_CALCULE'] = autres_dep_fonct

    # Calculer le vrai montant de L7
    total_dep_fonct = dot_amort_immo + dot_amort_sinistr + dep_cherch_tech + rem_sal_inv + dep_jd + autres_dep_fonct

    # 2. Mettre à jour L7
    df.at[idx, 'L7_TOTAL_DEP_FONCT_CALCULE'] = total_dep_fonct # laisser L7 tel quel (inchangé)

    # Garder L8_FRAIS_BREVETS_COV tel quel (inchangé)
    frais_brev_cov = df.at[idx, 'L8_FRAIS_BREVETS_COV'] # mettre à 0

    # 3. Recalculer la ligne 14 (total dépenses internes)
    dep_maint_brev_cov = df.at[idx, 'L9_DEPENSES_DEFENSE_BREVETS']
    dot_amort_brev = df.at[idx, 'L10_DOTATION_AMORT_BREVETS']
    dep_normali = df.at[idx, 'L11_DEPENSES_NORMALISATION_DECLARE']
    prim_cotiz = df.at[idx, 'L12_PRIMES_COTISATIONS_PLAFONNEES']
    dep_veil_techno = df.at[idx, 'L13_VEILLE_TECHNO_PLAFONNEE']

    total_dep_internes = total_dep_fonct + frais_brev_cov + dep_maint_brev_cov + dot_amort_brev + dep_normali + prim_cotiz + dep_veil_techno
    df.at[idx, 'L14_TOTAL_DEPENSES_INTERNES_CALCULE'] = total_dep_internes

    # 4. Recalculer la ligne 22 (total dépenses de recherche)
    total_dep_ext = df.at[idx, 'L21_TOTAL_DEP_EXT_PLAFONNEES_CALCULE']
    total_dep_recherche = total_dep_internes + total_dep_ext
    df.at[idx, 'L22_TOTAL_DEPENSES_RECHERCHE_CALCULE'] = total_dep_recherche

    # 5. Recalculer la ligne 26A (montant net des dépenses)
    mt_aid_subv = df.at[idx, 'L23A_SUBVENTIONS']
    mt_enc_presta = df.at[idx, 'L23B_SOMMES_ENCAISSEES_TIERS']
    mt_dep_conseils_cir = df.at[idx, 'L24_DEPENSES_CONSEIL_CIR']
    rembst_subv = df.at[idx, 'L25_REMBOURSEMENTS_SUBVENTIONS']

    montant_net = total_dep_recherche - mt_aid_subv - mt_enc_presta - mt_dep_conseils_cir + rembst_subv
    df.at[idx, 'L26A_MONTANT_NET_DEPENSES_CALCULE'] = montant_net

    # 6. Récupérer le montant DOM pour le calcul du CIR
    montant_net_dom = df.at[idx, 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE']

    # 7. Recalculer le CIR recherche (30% ou 50% selon localisation)
    cir_recherche = ((montant_net - montant_net_dom) * 0.30) + (montant_net_dom * 0.50)
    df.at[idx, 'CIR_RECHERCHE_CALCULE'] = cir_recherche

    # 8. Recalculer le CIR total
    cir_collection = df.at[idx, 'CIR_COLLECTION_CALCULE']
    cir_innovation = df.at[idx, 'CIR_INNOVATION_CALCULE']
    cir_collaboratif = df.at[idx, 'CIR_COLLABORATIF_CALCULE']

    cir_total = cir_recherche + cir_collection + cir_innovation + cir_collaboratif
    df.at[idx, 'CIR_TOTAL_CALCULE'] = cir_total

    # 9. Recalculer l'écart et mettre à jour la correspondance
    ecart_cir = cir_total - df.at[idx, 'CIR_TOTAL_DECLARE']
    df.at[idx, 'CIR_TOTAL_ECART'] = ecart_cir
    df.at[idx, 'CORRESPONDANCE_CIR'] = "Oui" if abs(ecart_cir) <= 1 else "Non"

# Afficher le résumé des modifications
l7_corrige = df.loc[indices_to_update, 'L7_TOTAL_DEP_FONCT_CALCULE']
somme_declare = df.loc[indices_to_update, 'CIR_TOTAL_DECLARE'].sum()
somme_calcule_avant = cir_calcule_avant.sum()
somme_calcule_apres = df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'].sum()

print(f"\nRésumé des corrections :")
print(f"Somme L7_TOTAL_DEP_FONCT_DECLARE avant correction: {l7_original.sum():,.2f} €")
print(f"Somme L7_TOTAL_DEP_FONCT_CALCULE après correction: {l7_corrige.sum():,.2f} €")
print(f"Différence sur L7: {l7_corrige.sum() - l7_original.sum():,.2f} €")

print(f"\nImpact sur le CIR :")
print(f"Somme CIR_TOTAL_DECLARE: {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE avant correction: {somme_calcule_avant:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE après correction: {somme_calcule_apres:,.2f} €")
print(f"Écart avant correction: {somme_calcule_avant - somme_declare:,.2f} €")
print(f"Écart après correction: {somme_calcule_apres - somme_declare:,.2f} €")
print(f"Amélioration de l'écart: {abs(somme_calcule_avant - somme_declare) - abs(somme_calcule_apres - somme_declare):,.2f} €")

# Compter les lignes où la correspondance s'est améliorée
count_improved = 0
for idx in indices_to_update:
    if df.at[idx, 'CORRESPONDANCE_CIR'] == "Oui" and abs(cir_calcule_avant[idx] - df.at[idx, 'CIR_TOTAL_DECLARE']) > 1:
        count_improved += 1

print(f"\nNombre de lignes où la correspondance est passée de 'Non' à 'Oui': {count_improved}")

# Sauvegarder le fichier corrigé
df.to_csv("lignes_restantes_a_analyser_corrigees_L7L8.csv", sep=';', encoding='utf-8-sig', index=False)
print(f"\nFichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees_L7L8.csv")

# Afficher des exemples de modifications
print("\nExemples de modifications significatives:")
# Calculer l'impact en pourcentage sur le CIR
df_impact = pd.DataFrame({
    'SIREN': df.loc[indices_to_update, 'SIREN_DECLARANT'],
    'Désignation': df.loc[indices_to_update, 'DESIGNATION'],
    'L7 avant': l7_original,
    'L7 après': l7_corrige,
    'CIR avant': cir_calcule_avant,
    'CIR après': df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'],
    'Impact': abs(df.loc[indices_to_update, 'CIR_TOTAL_CALCULE'] - cir_calcule_avant)
})

# Afficher les 5 cas avec le plus grand impact en valeur absolue
for _, row in df_impact.sort_values('Impact', ascending=False).head(5).iterrows():
    print(f"SIREN: {row['SIREN']}")
    print(f"  Désignation: {row['Désignation']}")
    print(f"  L7 avant: {row['L7 avant']:,.2f} €")
    print(f"  L7 après: {row['L7 après']:,.2f} €")
    print(f"  CIR avant: {row['CIR avant']:,.2f} €")
    print(f"  CIR après: {row['CIR après']:,.2f} €")
    print(f"  Impact: {row['Impact']:,.2f} € ({row['Impact']/row['CIR avant']*100 if row['CIR avant'] != 0 else 0:.2f}%)")
    print()

Fichier chargé: 1418 lignes
Nombre de lignes à corriger: 0

Résumé des corrections :
Somme L7_TOTAL_DEP_FONCT_DECLARE avant correction: 0.00 €
Somme L7_TOTAL_DEP_FONCT_CALCULE après correction: 0.00 €
Différence sur L7: 0.00 €

Impact sur le CIR :
Somme CIR_TOTAL_DECLARE: 0.00 €
Somme CIR_TOTAL_CALCULE avant correction: 0.00 €
Somme CIR_TOTAL_CALCULE après correction: 0.00 €
Écart avant correction: 0.00 €
Écart après correction: 0.00 €
Amélioration de l'écart: 0.00 €

Nombre de lignes où la correspondance est passée de 'Non' à 'Oui': 0

Fichier corrigé sauvegardé: lignes_restantes_a_analyser_corrigees_L7L8.csv

Exemples de modifications significatives:


### ligne liée au erreur dep fonctionnement

In [21]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où L7_TOTAL_DEP_FONCT_DECLARE = L8_FRAIS_BREVETS_COV
df_filtered = df[(df['L7_TOTAL_DEP_FONCT_DECLARE'] == df['L8_FRAIS_BREVETS_COV']) & (df['L7_TOTAL_DEP_FONCT_DECLARE'] != 0.0)]

print(f"Nombre de lignes où L7_TOTAL_DEP_FONCT_DECLARE = L8_FRAIS_BREVETS_COV: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_L7_egal_L8.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 1418 lignes
Nombre de lignes où L7_TOTAL_DEP_FONCT_DECLARE = L8_FRAIS_BREVETS_COV: 0
Somme CIR_TOTAL_DECLARE : 0.00 €
Somme CIR_TOTAL_CALCULE : 0.00 €
Résultats exportés dans: lignes_L7_egal_L8.csv


In [22]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')  # Nouveau fichier
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT']) 
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure
siren_a_exclure =  siren_jeunes_docteurs | siren_l7_l8 

# Filtrer le DataFrame principal
# - Exclure les SIREN déjà analysés
# - Garder uniquement les lignes où CORRESPONDANCE_CIR est "Non"
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) & 
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 1418
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir Recherche

In [25]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("lignes_restantes_a_analyser.csv", sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Appliquer les filtres :
# - L27_TOTAL_DEPENSES_RECHERCHE_DECLARE différent de 0.0
# - CIR_RECHERCHE_DECLARE = 0.0
df_filtered = df[
    (df['L27_TOTAL_DEPENSES_RECHERCHE_DECLARE'] != 0.0) &
    (df['CIR_RECHERCHE_DECLARE'] == 0.0)
]

print(f"Nombre de lignes après filtrage: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_depenses_non0_cir_recherche_0_2021.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

# Afficher quelques statistiques sur les lignes trouvées
if len(df_filtered) > 0:
    print("\nAperçu des données trouvées:")
    print(f"Nombre d'entreprises: {len(df_filtered)}")

Fichier chargé: 1418 lignes
Nombre de lignes après filtrage: 934
Somme CIR_TOTAL_DECLARE : 434,828,912.00 €
Somme CIR_TOTAL_CALCULE : 429,530,660.55 €
Résultats exportés dans: lignes_depenses_non0_cir_recherche_0_2021.csv

Aperçu des données trouvées:
Nombre d'entreprises: 934


In [26]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du nouveau cas)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8  | siren_depenses_non0_cir0

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 1418
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur depense Externalisée

In [31]:
import pandas as pd

# Chemin du fichier
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//lignes_restantes_a_analyser.csv"

# Charger le fichier
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Colonnes d'écart pour les dépenses de sous-traitance (2021)
colonnes_ext = ['L17_ECART', 'L20_ECART', 'L21_ECART', 'L22_ECART', 'L26_ECART']

# Extraire les lignes qui répondent à tous les critères
mask_correspondance = df['CORRESPONDANCE_CIR'] == "Non"
mask_cir = df['CIR_RECHERCHE_ECART'] != 0
mask_ext = df[colonnes_ext].abs().sum(axis=1) > 0

# Combiner tous les filtres
df_resultat = df[mask_correspondance & mask_cir & mask_ext]

print(f"Nombre de lignes extraites: {len(df_resultat)}")

# Exporter vers un fichier CSV
output_file = "lignes_ecart_cir_recherche_externalise.csv"
df_resultat.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Lignes exportées dans: {output_file}")

Fichier chargé: 1418 lignes
Nombre de lignes extraites: 0
Lignes exportées dans: lignes_ecart_cir_recherche_externalise.csv


In [32]:
import pandas as pd
import numpy as np

# Charger le fichier CSV généré précédemment
fichier = "lignes_ecart_cir_recherche_externalise.csv"
df = pd.read_csv(fichier, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Convertir les colonnes en numérique pour s'assurer que les calculs seront corrects
df['L31A_ECART'] = pd.to_numeric(df['L31A_ECART'], errors='coerce')
df['CIR_TOTAL_ECART'] = pd.to_numeric(df['CIR_TOTAL_ECART'], errors='coerce')

# Créer une colonne pour vérifier si L31A_ECART = CIR_TOTAL_ECART / 0.3
# Permettre une petite marge d'erreur pour les arrondis (0.1%)
df['RATIO'] = df['L31A_ECART'] / (df['CIR_TOTAL_ECART'] / 0.3)
df['EST_EGAL'] = np.isclose(df['RATIO'], 1, rtol=1)  # Tolérance relative de 0.1%

# Compter combien de lignes respectent cette relation
nb_egal = df['EST_EGAL'].sum()
pourcentage = (nb_egal / len(df)) * 100

print(f"\nRésultats de l'analyse:")
print(f"- Nombre de lignes où L31A_ECART = CIR_TOTAL_ECART / 3: {nb_egal} sur {len(df)}")
print(f"- Pourcentage: {pourcentage:.2f}%")

# Afficher quelques exemples où la relation est respectée
print("\nExemples où L31A_ECART = CIR_TOTAL_ECART / 3:")
exemples_egal = df[df['EST_EGAL']].head(5)
for _, row in exemples_egal.iterrows():
    print(f"SIREN: {row['SIREN_DECLARANT']}, L31A_ECART: {row['L31A_ECART']}, CIR_TOTAL_ECART/3: {row['CIR_TOTAL_ECART']/3}")
    # Filtrer uniquement les lignes où L31A_ECART = CIR_TOTAL_ECART / 3 est vrai
    df_resultat = df[df['EST_EGAL']]

    # Afficher le nombre de lignes correspondantes
    print(f"Nombre de lignes où L31A_ECART = CIR_TOTAL_ECART / 3: {len(df_resultat)}")

    # Exporter les résultats filtrés dans un fichier CSV
    df_resultat.to_csv("lignes_L31A_egal_CIR_TOTAL_div_3.csv", sep=';', encoding='utf-8-sig', index=False)
    # Calculer les sommes
    somme_declare = df_resultat['CIR_TOTAL_DECLARE'].sum()
    somme_calcule = df_resultat['CIR_TOTAL_CALCULE'].sum()

    print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
    print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
    print(f"Résultats exportés dans: lignes_L31A_egal_CIR_TOTAL_div_3_2021.csv")
# Afficher quelques exemples où la relation n'est pas respectée
print("\nExemples où L31A_ECART ≠ CIR_TOTAL_ECART / 3:")
exemples_different = df[~df['EST_EGAL']].head(5)
for _, row in exemples_different.iterrows():
    print(f"SIREN: {row['SIREN_DECLARANT']}, L31A_ECART: {row['L31A_ECART']}, CIR_TOTAL_ECART/3: {row['CIR_TOTAL_ECART']/3}, Ratio: {row['RATIO']:.4f}")

# Exporter les résultats avec la colonne d'analyse supplémentaire
df.to_csv("analyse_L31A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig', index=False)
print(f"\nRésultats détaillés exportés dans: analyse_L31A_vs_CIR_TOTAL.csv")

Fichier chargé: 0 lignes

Résultats de l'analyse:
- Nombre de lignes où L31A_ECART = CIR_TOTAL_ECART / 3: 0 sur 0
- Pourcentage: nan%

Exemples où L31A_ECART = CIR_TOTAL_ECART / 3:

Exemples où L31A_ECART ≠ CIR_TOTAL_ECART / 3:

Résultats détaillés exportés dans: analyse_L31A_vs_CIR_TOTAL.csv


C:\Users\msamb\AppData\Local\Temp\ipykernel_928\700265953.py:20: RuntimeWarning: invalid value encountered in scalar divide
  pourcentage = (nb_egal / len(df)) * 100


In [34]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas analyse_L26A_vs_CIR_TOTAL.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | siren_analyse_l26a

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 1417
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur doublement sans motif de la créance total

In [35]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer selon les critères :
# - ECART_RELATIF_POURCENT == -50 ou 50
# - CIR_INNOVATION_ECART == 0
# - CIR_RECHERCHE_ECART entre -1 et 1 inclus
mask = (
    df['ECART_RELATIF_POURCENT'].isin([-50.0, 50.0]) &
    (df['CIR_INNOVATION_ECART'] == 0.0) &
    (df['CIR_RECHERCHE_ECART'].between(-1, 1))
)

df_filtered = df[mask]
print(f"Lignes extraites: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_ecart_relatif_50_innovation0_recherche1.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 1417 lignes
Lignes extraites: 0
Somme CIR_TOTAL_DECLARE : 0.00 €
Somme CIR_TOTAL_CALCULE : 0.00 €
Résultats exportés dans: lignes_ecart_relatif_50_innovation0_recherche1.csv


In [37]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas analyse_L26A_vs_CIR_TOTAL.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 1417
Résultats exportés dans: lignes_restantes_a_analyser.csv


## erreur diff Cir total compris entre -500 et 500

In [38]:
import pandas as pd

# Charger le fichier des lignes restantes à analyser
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où CIR_TOTAL_ECART est entre -500 et 500 et CORRESPONDANCE_CIR == "Non"
mask_ecart = df['CIR_TOTAL_ECART'].between(-500, 500) & df['CORRESPONDANCE_CIR'].eq("Non")
df_resultat = df[mask_ecart]

print(f"Nombre de lignes extraites: {len(df_resultat)}")

# Exporter les résultats
output_file = "lignes_ecart_total_entre_-500_et_500.csv"
df_resultat.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_resultat['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_resultat['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 1417 lignes
Nombre de lignes extraites: 3
Somme CIR_TOTAL_DECLARE : 196,752.00 €
Somme CIR_TOTAL_CALCULE : 196,690.58 €
Résultats exportés dans: lignes_ecart_total_entre_-500_et_500.csv


In [39]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas lignes_ecart_total_entre_-500_et_500.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | siren_ecart_total500

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 1414
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Plafond Partout

In [40]:
import pandas as pd

# Charger le fichier avec les 703 lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où ECART_RELATIF_POURCENT est égal à -100
df_ecart_negatif = df[df['ECART_RELATIF_POURCENT'] == -100]
print(f"Nombre de lignes avec ECART_RELATIF_POURCENT = -100: {len(df_ecart_negatif)}")

# Exporter les résultats dans un fichier CSV
output_file = "lignes_ecart_relatif_moins_100.csv"
df_ecart_negatif.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_ecart_negatif['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_ecart_negatif['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 1414 lignes
Nombre de lignes avec ECART_RELATIF_POURCENT = -100: 446
Somme CIR_TOTAL_DECLARE : 12,611,200.00 €
Somme CIR_TOTAL_CALCULE : 0.43 €
Résultats exportés dans: lignes_ecart_relatif_moins_100.csv


In [41]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas lignes_ecart_relatif_moins_100.csv)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 968
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir innovation

In [42]:
import pandas as pd
import numpy as np

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1)
mask = np.isclose(df['CIR_TOTAL_ECART'], df['CIR_INNOVATION_ECART'], atol=1.0)
df_filtered = df[mask]

print(f"\nNombre de lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1): {len(df_filtered)}")

# Afficher quelques exemples
if len(df_filtered) > 0:
    print("\nExemples :")
    for _, row in df_filtered.head().iterrows():
        print(f"SIREN: {row['SIREN_DECLARANT']}")
        print(f"CIR_TOTAL_ECART: {row['CIR_TOTAL_ECART']:.2f}")
        print(f"CIR_INNOVATION_ECART: {row['CIR_INNOVATION_ECART']:.2f}")

# Exporter les résultats
output_file = "lignes_ecart_total_egal_innovation.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"\nRésultats exportés dans: {output_file}")

Fichier chargé: 968 lignes

Nombre de lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1): 643

Exemples :
SIREN: 421100645
CIR_TOTAL_ECART: 54071.43
CIR_INNOVATION_ECART: 54071.20
SIREN: 314704057
CIR_TOTAL_ECART: 3944.88
CIR_INNOVATION_ECART: 3944.80
SIREN: 531583888
CIR_TOTAL_ECART: 67314.91
CIR_INNOVATION_ECART: 67314.40
SIREN: 435405923
CIR_TOTAL_ECART: 70034.23
CIR_INNOVATION_ECART: 70034.60
SIREN: 537523615
CIR_TOTAL_ECART: 33959.99
CIR_INNOVATION_ECART: 33960.32
Somme CIR_TOTAL_DECLARE : 34,392,892.00 €
Somme CIR_TOTAL_CALCULE : 60,719,903.55 €

Résultats exportés dans: lignes_ecart_total_egal_innovation.csv


In [43]:
import pandas as pd
import numpy as np

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1)
mask = np.isclose(df['CIR_TOTAL_ECART'], df['CIR_INNOVATION_ECART'], atol=1.0)
df_filtered = df[mask]

print(f"\nNombre de lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1): {len(df_filtered)}")

# Afficher quelques exemples
if len(df_filtered) > 0:
    print("\nExemples :")
    for _, row in df_filtered.head().iterrows():
        print(f"SIREN: {row['SIREN_DECLARANT']}")
        print(f"CIR_TOTAL_ECART: {row['CIR_TOTAL_ECART']:.2f}")
        print(f"CIR_INNOVATION_ECART: {row['CIR_INNOVATION_ECART']:.2f}")

# Exporter les résultats
output_file = "lignes_ecart_total_egal_innovation.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"\nRésultats exportés dans: {output_file}")

Fichier chargé: 968 lignes

Nombre de lignes où CIR_TOTAL_ECART = CIR_INNOVATION_ECART (±1): 643

Exemples :
SIREN: 421100645
CIR_TOTAL_ECART: 54071.43
CIR_INNOVATION_ECART: 54071.20
SIREN: 314704057
CIR_TOTAL_ECART: 3944.88
CIR_INNOVATION_ECART: 3944.80
SIREN: 531583888
CIR_TOTAL_ECART: 67314.91
CIR_INNOVATION_ECART: 67314.40
SIREN: 435405923
CIR_TOTAL_ECART: 70034.23
CIR_INNOVATION_ECART: 70034.60
SIREN: 537523615
CIR_TOTAL_ECART: 33959.99
CIR_INNOVATION_ECART: 33960.32
Somme CIR_TOTAL_DECLARE : 34,392,892.00 €
Somme CIR_TOTAL_CALCULE : 60,719,903.55 €

Résultats exportés dans: lignes_ecart_total_egal_innovation.csv


## Erreur Cir Collection

In [44]:
import pandas as pd
import numpy as np

# Charger le fichier
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Créer le masque pour identifier où CIR_TOTAL_ECART = CIR_COLLECTION_ECART (±1)
mask = np.isclose(df['CIR_TOTAL_ECART'], df['CIR_COLLECTION_ECART'], atol=1.0)

# Filtrer les lignes
df_filtered = df[mask]

print(f"\nNombre de lignes où CIR_TOTAL_ECART = CIR_COLLECTION_ECART (±1): {len(df_filtered)}")

# Afficher quelques statistiques sur ces lignes
if len(df_filtered) > 0:
    print("\nQuelques exemples :")
    for _, row in df_filtered.head().iterrows():
        print(f"\nSIREN: {row['SIREN_DECLARANT']}")
        print(f"CIR_TOTAL_ECART: {row['CIR_TOTAL_ECART']:.2f}")
        print(f"CIR_COLLECTION_ECART: {row['CIR_COLLECTION_ECART']:.2f}")

# Exporter les résultats
output_file = "lignes_ecart_total_egal_collection.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"\nRésultats exportés dans: {output_file}")

Fichier chargé: 968 lignes

Nombre de lignes où CIR_TOTAL_ECART = CIR_COLLECTION_ECART (±1): 0
Somme CIR_TOTAL_DECLARE : 0.00 €
Somme CIR_TOTAL_CALCULE : 0.00 €

Résultats exportés dans: lignes_ecart_total_egal_collection.csv


In [46]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
#analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas collection_egal_total)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 324
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur Cir Déclarer vaut 0 alors que sa ne l'est pas

In [47]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où ECART_RELATIF_POURCENT = 100 et CIR_TOTAL_DECLARE = 0
df_filtered = df[(df['ECART_RELATIF_POURCENT'] == 100) & (df['CIR_TOTAL_DECLARE'] == 0)]

print(f"Nombre de lignes où ECART_RELATIF_POURCENT = 100 et CIR_TOTAL_DECLARE = 0: {len(df_filtered)}")

# Exporter les résultats
output_file = "lignes_ecart_100_cir_declare_0.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 324 lignes
Nombre de lignes où ECART_RELATIF_POURCENT = 100 et CIR_TOTAL_DECLARE = 0: 3
Somme CIR_TOTAL_DECLARE : 0.00 €
Somme CIR_TOTAL_CALCULE : 234,295.90 €
Résultats exportés dans: lignes_ecart_100_cir_declare_0.csv


In [48]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas ecart_100_cir_declare_0)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 321
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur liée à L6

In [49]:
import pandas as pd
import numpy as np

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Filtrer les lignes où CIR_TOTAL_ECART = CIR_RECHERCHE_ECART (±1)
mask_ecart = np.isclose(df['CIR_TOTAL_ECART'], df['CIR_RECHERCHE_ECART'], atol=1.0)

# Filtrer les lignes où L6_ECART est strictement en dehors de [-1, 1]
mask_l6 = (df['L6_ECART'] < -1) | (df['L6_ECART'] > 1)

# Appliquer les deux filtres
df_filtered = df[mask_ecart & mask_l6]

print(f"\nNombre de lignes où CIR_TOTAL_ECART = CIR_RECHERCHE_ECART (±1) et L6_ECART hors [-1, 1]: {len(df_filtered)}")
# Exporter les résultats
output_file = "lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"\nRésultats exportés dans: {output_file}")

Fichier chargé: 321 lignes

Nombre de lignes où CIR_TOTAL_ECART = CIR_RECHERCHE_ECART (±1) et L6_ECART hors [-1, 1]: 0
Somme CIR_TOTAL_DECLARE : 0.00 €
Somme CIR_TOTAL_CALCULE : 0.00 €

Résultats exportés dans: lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv


In [50]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')
l6_ecart_hors_1 = pd.read_csv("lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])
siren_l6_ecart_hors_1 = set(l6_ecart_hors_1['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas l6_ecart_hors_1)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0 | siren_l6_ecart_hors_1

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 321
Résultats exportés dans: lignes_restantes_a_analyser.csv


## Erreur liée à L26

In [53]:
import pandas as pd

# Charger le fichier des lignes restantes
file_path = "lignes_restantes_a_analyser.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier chargé: {len(df)} lignes")

# Appliquer les filtres demandés (adaptés pour 2021)
mask = (
    ((df['L31A_ECART'] > 1) | (df['L31A_ECART'] < -1))
    & ((df['L14_ECART'] > -1) & (df['L14_ECART'] < 1))
    & ((df['L22_ECART'] > -1) & (df['L22_ECART'] < 1))
    & ((df['L26_ECART'] > -1) & (df['L26_ECART'] < 1))
    & ((df['L27_ECART'] > -1) & (df['L27_ECART'] < 1))
)

df_filtered = df[mask]
print(f"Nombre de lignes correspondant au cas demandé : {len(df_filtered)}")

# Afficher quelques exemples
if len(df_filtered) > 0:
    print(df_filtered[['SIREN_DECLARANT', 'L31A_ECART', 'L14_ECART', 'L22_ECART', 'L26_ECART']].head())

# Exporter les résultats
output_file = "lignes_cas_L31A_ecart_hors_1_L14_L22_L26_dans_1_2021.csv"
df_filtered.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
# Calculer les sommes
somme_declare = df_filtered['CIR_TOTAL_DECLARE'].sum()
somme_calcule = df_filtered['CIR_TOTAL_CALCULE'].sum()

print(f"Somme CIR_TOTAL_DECLARE : {somme_declare:,.2f} €")
print(f"Somme CIR_TOTAL_CALCULE : {somme_calcule:,.2f} €")
print(f"Résultats exportés dans: {output_file}")

Fichier chargé: 321 lignes
Nombre de lignes correspondant au cas demandé : 0
Somme CIR_TOTAL_DECLARE : 0.00 €
Somme CIR_TOTAL_CALCULE : 0.00 €
Résultats exportés dans: lignes_cas_L31A_ecart_hors_1_L14_L22_L26_dans_1_2021.csv


In [54]:
import pandas as pd

# Charger le fichier principal
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')
print(f"Fichier principal chargé: {len(df)} lignes")

# Charger tous les fichiers d'analyse précédents
jeunes_docteurs = pd.read_csv("lignes_jeunes_docteurs_0_amort_sinistr_non0.csv", sep=';', encoding='utf-8-sig')
l7_egal_l8 = pd.read_csv("lignes_L7_egal_L8.csv", sep=';', encoding='utf-8-sig')
depenses_non0_cir0 = pd.read_csv("lignes_depenses_non0_cir_recherche_0.csv", sep=';', encoding='utf-8-sig')
analyse_l26a = pd.read_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//analyse_L26A_vs_CIR_TOTAL.csv", sep=';', encoding='utf-8-sig')
ecart_relatif_50_innov0_rech1 = pd.read_csv("lignes_ecart_relatif_50_innovation0_recherche1.csv", sep=';', encoding='utf-8-sig')
ecart_total500 = pd.read_csv("lignes_ecart_total_entre_-500_et_500.csv", sep=';', encoding='utf-8-sig')
ecart_moins_100 = pd.read_csv("lignes_ecart_relatif_moins_100.csv", sep=';', encoding='utf-8-sig')
innovation_egal_total = pd.read_csv("lignes_ecart_total_egal_innovation.csv", sep=';', encoding='utf-8-sig')
collection_egal_total = pd.read_csv("lignes_ecart_total_egal_collection.csv", sep=';', encoding='utf-8-sig')
ecart_100_cir_declare_0 = pd.read_csv("lignes_ecart_100_cir_declare_0.csv", sep=';', encoding='utf-8-sig')
l6_ecart_hors_1 = pd.read_csv("lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv", sep=';', encoding='utf-8-sig')
cas_l26a = pd.read_csv("lignes_cas_L26A_ecart_hors_1_L14_L20_L21_dans_1.csv", sep=';', encoding='utf-8-sig')

# Identifier les SIREN à exclure
siren_jeunes_docteurs = set(jeunes_docteurs['SIREN_DECLARANT'])
siren_l7_l8 = set(l7_egal_l8['SIREN_DECLARANT'])
siren_depenses_non0_cir0 = set(depenses_non0_cir0['SIREN_DECLARANT'])
siren_analyse_l26a = set(analyse_l26a['SIREN_DECLARANT'])
siren_ecart_relatif_50_innov0_rech1 = set(ecart_relatif_50_innov0_rech1['SIREN_DECLARANT'])
siren_ecart_total500 = set(ecart_total500['SIREN_DECLARANT'])
siren_ecart_moins_100 = set(ecart_moins_100['SIREN_DECLARANT'])
siren_innovation_egal_total = set(innovation_egal_total['SIREN_DECLARANT'])
siren_collection_egal_total = set(collection_egal_total['SIREN_DECLARANT'])
siren_ecart_100_cir_declare_0 = set(ecart_100_cir_declare_0['SIREN_DECLARANT'])
siren_l6_ecart_hors_1 = set(l6_ecart_hors_1['SIREN_DECLARANT'])
siren_cas_l26a = set(cas_l26a['SIREN_DECLARANT'])

# Combiner tous les SIREN à exclure (ajout du cas l6_ecart_hors_1)
siren_a_exclure = siren_jeunes_docteurs | siren_l7_l8 | siren_depenses_non0_cir0 | \
                  siren_analyse_l26a | siren_ecart_relatif_50_innov0_rech1 | \
                  siren_ecart_total500 | siren_ecart_moins_100 | \
                  siren_innovation_egal_total | siren_collection_egal_total | \
                  siren_ecart_100_cir_declare_0 | siren_l6_ecart_hors_1 | siren_cas_l26a

# Filtrer le DataFrame principal
df_restant = df[
    (~df['SIREN_DECLARANT'].isin(siren_a_exclure)) &
    (df['CORRESPONDANCE_CIR'] == "Non")
]

print(f"\nNombre de lignes restantes à analyser: {len(df_restant)}")

# Exporter les résultats
output_file = "lignes_restantes_a_analyser.csv"
df_restant.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"Résultats exportés dans: {output_file}")

Fichier principal chargé: 4493 lignes

Nombre de lignes restantes à analyser: 321
Résultats exportés dans: lignes_restantes_a_analyser.csv


# Correction Creance

In [56]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# ===================================================================
# FONCTIONS UTILITAIRES
# ===================================================================

def format_siren(siren):
    """Formate les SIREN pour avoir exactement 9 caractères"""
    if pd.isna(siren):
        return siren
    siren_str = str(siren).strip()
    # Gère les formats type '123456789.0'
    if '.' in siren_str:
        siren_str = siren_str.split('.')[0]
    return siren_str.zfill(9)

def convertir_en_nombre(valeur, defaut=0.0):
    """Convertit une valeur en nombre flottant de façon sécurisée"""
    if pd.isna(valeur):
        return defaut
    valeur_str = str(valeur).strip()
    if valeur_str == '':
        return defaut
    try:
        # Remplace la virgule par un point pour la conversion décimale
        return float(valeur_str.replace(',', '.'))
    except ValueError:
        # Retourne la valeur par défaut en cas d'erreur de conversion
        return defaut
    except Exception:
         # Capture d'autres erreurs potentielles
        return defaut

# ===================================================================
# CHARGEMENT ET PRÉPARATION DES DONNÉES
# ===================================================================

def charger_donnees(file_path="Calcul_Creance_CIR_2021.csv"):
    """Charge et prépare les données du fichier CIR 2021"""
    print(f"Chargement et préparation des données depuis: {file_path}...")
    try:
        df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig',
                         converters={'SIREN_DECLARANT': str, 'SIREN_DEPOSANT': str},
                         low_memory=False)
        print(f"Fichier principal chargé: {len(df):,} lignes")
    except FileNotFoundError:
        print(f"ERREUR: Fichier non trouvé: {file_path}.")
        return None
    except Exception as e:
        print(f"ERREUR lors du chargement du fichier principal: {str(e)}")
        return None

    print("Formatage des SIRENs...")
    if 'SIREN_DECLARANT' in df.columns:
        df['SIREN_DECLARANT'] = df['SIREN_DECLARANT'].apply(format_siren)
    else:
        print("ERREUR CRITIQUE: Colonne 'SIREN_DECLARANT' non trouvée.")
        return None
    if 'SIREN_DEPOSANT' in df.columns:
        df['SIREN_DEPOSANT'] = df['SIREN_DEPOSANT'].apply(format_siren)

    if 'TRAITEMENT_APPLIQUE' not in df.columns:
        df['TRAITEMENT_APPLIQUE'] = "Non traité"
    else:
         # Assure que la colonne est de type string
         df['TRAITEMENT_APPLIQUE'] = df['TRAITEMENT_APPLIQUE'].astype(str)

    # Liste exhaustive des colonnes numériques à convertir (pour 2021)
    numeric_cols = [
        'L1_DOTATION_AMORT_IMMO', 'L2_DOTATION_AMORT_SINISTR', 'L3_DEPENSES_PERSONNEL_CHERCHEURS',
        'L4_REMUNERATION_INVENTEURS', 'L5_DEPENSES_JEUNES_DOCTEURS', 'L6_AUTRES_DEP_FONCT_CALCULE',
        'L7_TOTAL_DEP_FONCT_DECLARE', 'L7_TOTAL_DEP_FONCT_CALCULE', 'L8_FRAIS_BREVETS_COV',
        'L9_DEPENSES_DEFENSE_BREVETS', 'L10_DOTATION_AMORT_BREVETS', 'L11_DEPENSES_NORMALISATION',
        'L12_PRIMES_COTISATIONS_BRUT', 'L12_PRIMES_COTISATIONS_PLAFONNEES', 'L13_VEILLE_TECHNO_BRUT',
        'L13_VEILLE_TECHNO_PLAFONNEE', 'L14_TOTAL_DEPENSES_INTERNES_CALCULE', 'L26_TOTAL_ST_PLAFONNE_CALCULE',
        'L27_TOTAL_DEPENSES_RECHERCHE_CALCULE', 'L28A_SUBVENTIONS', 'L28B_SOMMES_ENCAISSEES_TIERS',
        'L29_DEPENSES_CONSEIL_CIR', 'L30_REMBOURSEMENTS_SUBVENTIONS', 'L31A_MONTANT_NET_DEPENSES_CALCULE',
        'L31B_MONTANT_NET_DEPENSES_DOM_CALCULE', 'CIR_RECHERCHE_CALCULE', 'CIR_COLLECTION_CALCULE',
        'CIR_INNOVATION_CALCULE', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE',
        'CIR_TOTAL_ECART', 'ECART_RELATIF_POURCENT', 'CIR_RECHERCHE_ECART', 'CIR_INNOVATION_ECART',
        'CIR_COLLECTION_ECART', 'L6_ECART', 'L14_ECART', 'L17_ECART', 'L20_ECART',
        'L21_ECART', 'L22_ECART', 'L26_ECART', 'L27_ECART', 'L31A_ECART', 'L31B_ECART',
        'CIR_TOTAL_CORRIGE' # Inclure si existe déjà
    ]

    print("Conversion des colonnes numériques...")
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertir_en_nombre)

    # Initialisation de CIR_TOTAL_CORRIGE après les conversions
    if 'CIR_TOTAL_CORRIGE' not in df.columns:
        print("Initialisation de la colonne 'CIR_TOTAL_CORRIGE'...")
        if 'CIR_TOTAL_CALCULE' in df.columns:
            df['CIR_TOTAL_CORRIGE'] = df['CIR_TOTAL_CALCULE'].copy()
        else:
            print("  Attention: 'CIR_TOTAL_CALCULE' non trouvé. 'CIR_TOTAL_CORRIGE' initialisé à 0.0.")
            df['CIR_TOTAL_CORRIGE'] = 0.0
    else:
        # Assurer qu'elle est bien numérique si elle existait
        df['CIR_TOTAL_CORRIGE'] = df['CIR_TOTAL_CORRIGE'].apply(convertir_en_nombre)
        print("La colonne 'CIR_TOTAL_CORRIGE' existait déjà et a été (re)convertie en numérique.")

    # Vérification finale des colonnes essentielles
    essential_cols = ['SIREN_DECLARANT', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_CORRIGE']
    for col in essential_cols:
        if col not in df.columns:
            print(f"ERREUR CRITIQUE: Colonne essentielle '{col}' est manquante après chargement/préparation.")
            return None

    print("Préparation des données terminée.")
    return df

def charger_fichier_intermediaire(file_path, main_df_columns):
    """Charge un fichier intermédiaire, formate SIREN et convertit num."""
    try:
        df_inter = pd.read_csv(file_path, sep=';', encoding='utf-8-sig',
                               converters={'SIREN_DECLARANT': str, 'SIREN_DEPOSANT': str},
                               low_memory=False)
        if 'SIREN_DECLARANT' in df_inter.columns:
            df_inter['SIREN_DECLARANT'] = df_inter['SIREN_DECLARANT'].apply(format_siren)
        else:
             print(f"Attention: 'SIREN_DECLARANT' manquant dans '{file_path}'.")
             return pd.DataFrame(columns=main_df_columns)

        numeric_cols_inter = [
            'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_ECART',
            'CIR_RECHERCHE_ECART', 'CIR_COLLECTION_ECART', 'CIR_INNOVATION_ECART'
        ]
        for col in numeric_cols_inter:
            if col in df_inter.columns:
                df_inter[col] = df_inter[col].apply(convertir_en_nombre)
        if 'EST_EGAL' in df_inter.columns:
             df_inter['EST_EGAL'] = df_inter['EST_EGAL'].apply(lambda x: str(x).strip().lower() == 'true')
        return df_inter
    except FileNotFoundError:
        print(f"Info: Fichier intermédiaire '{file_path}' non trouvé.")
        return pd.DataFrame(columns=main_df_columns)
    except Exception as e:
        print(f"Erreur chargement fichier intermédiaire '{file_path}': {str(e)}")
        return pd.DataFrame(columns=main_df_columns)

# ===================================================================
# FONCTIONS DE CORRECTION (adaptées pour 2021)
# ===================================================================

def correction_erreurs_personnel(df, processed_sirens, fichier_intermediaire="lignes_jeunes_docteurs_0_amort_sinistr_non0.csv"):
    """Corrige les erreurs de dépenses de personnel (jeunes docteurs) pour 2021."""
    print("\n1. CORRECTION ERREURS DÉPENSES PERSONNEL (JEUNES DOCTEURS) 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        l1_val=convertir_en_nombre(df.loc[idx,'L1_DOTATION_AMORT_IMMO']); l2_val=convertir_en_nombre(df.loc[idx,'L2_DOTATION_AMORT_SINISTR'])
        l4_val=convertir_en_nombre(df.loc[idx,'L4_REMUNERATION_INVENTEURS']); l5_val=convertir_en_nombre(df.loc[idx,'L5_DEPENSES_JEUNES_DOCTEURS'])
        l8_val=convertir_en_nombre(df.loc[idx,'L8_FRAIS_BREVETS_COV']); l9_val=convertir_en_nombre(df.loc[idx,'L9_DEPENSES_DEFENSE_BREVETS'])
        l10_val=convertir_en_nombre(df.loc[idx,'L10_DOTATION_AMORT_BREVETS']); l11_val=convertir_en_nombre(df.loc[idx,'L11_DEPENSES_NORMALISATION'])
        l12_brut_val=convertir_en_nombre(df.loc[idx,'L12_PRIMES_COTISATIONS_BRUT']); l13_brut_val=convertir_en_nombre(df.loc[idx,'L13_VEILLE_TECHNO_BRUT'])
        l26_calcule_val=convertir_en_nombre(df.loc[idx,'L26_TOTAL_ST_PLAFONNE_CALCULE']); l28a_val=convertir_en_nombre(df.loc[idx,'L28A_SUBVENTIONS'])
        l28b_val=convertir_en_nombre(df.loc[idx,'L28B_SOMMES_ENCAISSEES_TIERS']); l29_val=convertir_en_nombre(df.loc[idx,'L29_DEPENSES_CONSEIL_CIR'])
        l30_val=convertir_en_nombre(df.loc[idx,'L30_REMBOURSEMENTS_SUBVENTIONS']); cir_coll_val=convertir_en_nombre(df.loc[idx,'CIR_COLLECTION_CALCULE'])
        cir_inno_val=convertir_en_nombre(df.loc[idx,'CIR_INNOVATION_CALCULE'])
        l31b_val=convertir_en_nombre(df.loc[idx,'L31B_MONTANT_NET_DEPENSES_DOM_CALCULE'])
        df.loc[idx,'L3_DEPENSES_PERSONNEL_CHERCHEURS']=l2_val; df.loc[idx,'L2_DOTATION_AMORT_SINISTR']=0.0
        dep_cherch_tech_updated=df.loc[idx,'L3_DEPENSES_PERSONNEL_CHERCHEURS']; l2_updated=df.loc[idx,'L2_DOTATION_AMORT_SINISTR']
        autres_dep_fonct=(l1_val*0.75)+((dep_cherch_tech_updated+l4_val)*0.43)+l5_val; df.loc[idx,'L6_AUTRES_DEP_FONCT_CALCULE']=autres_dep_fonct
        total_dep_fonct=l1_val+l2_updated+dep_cherch_tech_updated+l4_val+l5_val+autres_dep_fonct; df.loc[idx,'L7_TOTAL_DEP_FONCT_CALCULE']=total_dep_fonct
        prim_cotiz_plaf=min(l12_brut_val,60000); dep_veil_techno_plaf=min(l13_brut_val,60000)
        total_dep_internes=total_dep_fonct+l8_val+l9_val+l10_val+l11_val+prim_cotiz_plaf+dep_veil_techno_plaf; df.loc[idx,'L14_TOTAL_DEPENSES_INTERNES_CALCULE']=total_dep_internes
        total_dep_recherche=total_dep_internes+l26_calcule_val; df.loc[idx,'L27_TOTAL_DEPENSES_RECHERCHE_CALCULE']=total_dep_recherche
        montant_net=total_dep_recherche-l28a_val-l28b_val-l29_val+l30_val; df.loc[idx,'L31A_MONTANT_NET_DEPENSES_CALCULE']=montant_net
        taux_dom,taux_non_dom=0.50,0.30
        if 'L31B_MONTANT_NET_DEPENSES_DOM_CALCULE' in df.columns and l31b_val > 0 :
             montant_net_non_dom = max(0, montant_net - l31b_val); cir_recherche = (montant_net_non_dom * taux_non_dom) + (l31b_val * taux_dom)
        else: cir_recherche = montant_net * taux_non_dom
        cir_recherche=max(0, cir_recherche); df.loc[idx,'CIR_RECHERCHE_CALCULE']=cir_recherche
        cir_total=cir_recherche+cir_coll_val+cir_inno_val; df.loc[idx,'CIR_TOTAL_CALCULE']=cir_total
        df.loc[idx,'CIR_TOTAL_CORRIGE']=cir_total; df.loc[idx,'TRAITEMENT_APPLIQUE']="Err_dep_personnel"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count+=1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_erreurs_frais_brevets(df, processed_sirens, fichier_intermediaire="lignes_L7_egal_L8.csv"):
    """Corrige les erreurs où L7 (déclaré) est égal à L8 pour 2021."""
    print("\n2. CORRECTION ERREUR L7 = L8 (FRAIS BREVETS) 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        l1_val=convertir_en_nombre(df.loc[idx,'L1_DOTATION_AMORT_IMMO']); l2_val=convertir_en_nombre(df.loc[idx,'L2_DOTATION_AMORT_SINISTR'])
        l3_val=convertir_en_nombre(df.loc[idx,'L3_DEPENSES_PERSONNEL_CHERCHEURS']); l4_val=convertir_en_nombre(df.loc[idx,'L4_REMUNERATION_INVENTEURS'])
        l5_val=convertir_en_nombre(df.loc[idx,'L5_DEPENSES_JEUNES_DOCTEURS']); l9_val=convertir_en_nombre(df.loc[idx,'L9_DEPENSES_DEFENSE_BREVETS'])
        l10_val=convertir_en_nombre(df.loc[idx,'L10_DOTATION_AMORT_BREVETS']); l11_val=convertir_en_nombre(df.loc[idx,'L11_DEPENSES_NORMALISATION'])
        l12_plaf_val=convertir_en_nombre(df.loc[idx,'L12_PRIMES_COTISATIONS_PLAFONNEES']); l13_plaf_val=convertir_en_nombre(df.loc[idx,'L13_VEILLE_TECHNO_PLAFONNEE'])
        l26_calcule_val=convertir_en_nombre(df.loc[idx,'L26_TOTAL_ST_PLAFONNE_CALCULE']); l28a_val=convertir_en_nombre(df.loc[idx,'L28A_SUBVENTIONS'])
        l28b_val=convertir_en_nombre(df.loc[idx,'L28B_SOMMES_ENCAISSEES_TIERS']); l29_val=convertir_en_nombre(df.loc[idx,'L29_DEPENSES_CONSEIL_CIR'])
        l30_val=convertir_en_nombre(df.loc[idx,'L30_REMBOURSEMENTS_SUBVENTIONS']); cir_coll_val=convertir_en_nombre(df.loc[idx,'CIR_COLLECTION_CALCULE'])
        cir_inno_val=convertir_en_nombre(df.loc[idx,'CIR_INNOVATION_CALCULE'])
        l31b_val=convertir_en_nombre(df.loc[idx,'L31B_MONTANT_NET_DEPENSES_DOM_CALCULE'])
        df.loc[idx, 'L8_FRAIS_BREVETS_COV'] = 0.0; l8_corrected_val = 0.0
        autres_dep_fonct = (l1_val*0.75) + ((l3_val + l4_val)*0.43) + l5_val; df.loc[idx, 'L6_AUTRES_DEP_FONCT_CALCULE'] = autres_dep_fonct
        total_dep_fonct = l1_val+l2_val+l3_val+l4_val+l5_val+autres_dep_fonct; df.loc[idx, 'L7_TOTAL_DEP_FONCT_CALCULE'] = total_dep_fonct
        total_dep_internes = total_dep_fonct + l8_corrected_val + l9_val + l10_val + l11_val + l12_plaf_val + l13_plaf_val
        df.loc[idx, 'L14_TOTAL_DEPENSES_INTERNES_CALCULE'] = total_dep_internes
        total_dep_recherche = total_dep_internes + l26_calcule_val; df.loc[idx, 'L27_TOTAL_DEPENSES_RECHERCHE_CALCULE'] = total_dep_recherche
        montant_net = total_dep_recherche - l28a_val - l28b_val - l29_val + l30_val; df.loc[idx, 'L31A_MONTANT_NET_DEPENSES_CALCULE'] = montant_net
        taux_dom, taux_non_dom = 0.50, 0.30
        if 'L31B_MONTANT_NET_DEPENSES_DOM_CALCULE' in df.columns and l31b_val > 0 :
             montant_net_non_dom = max(0, montant_net - l31b_val); cir_recherche = (montant_net_non_dom * taux_non_dom) + (l31b_val * taux_dom)
        else: cir_recherche = montant_net * taux_non_dom
        cir_recherche = max(0, cir_recherche); df.loc[idx, 'CIR_RECHERCHE_CALCULE'] = cir_recherche
        cir_total = cir_recherche + cir_coll_val + cir_inno_val; df.loc[idx, 'CIR_TOTAL_CALCULE'] = cir_total
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = cir_total; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_Frais_BRV"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_recherche_manquant(df, processed_sirens, fichier_intermediaire="lignes_depenses_non0_cir_recherche_0.csv"):
    """Corrige les cas où le CIR recherche déclaré est manquant ou incohérent pour 2021."""
    print("\n3. CORRECTION CIR RECHERCHE MANQUANT/INCOHÉRENT 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_CIR_RECH"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count



def correction_doublement_sans_motif(df, processed_sirens, fichier_intermediaire="lignes_ecart_relatif_50_innovation0_recherche1.csv"):
    """Corrige les erreurs de doublement apparent (écart relatif ~50%) pour 2021. Prend la moyenne."""
    print("\n5. CORRECTION DOUBLEMENT SANS MOTIF (ÉCART ~50%) 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        declared = df.loc[idx, 'CIR_TOTAL_DECLARE']; calculated = df.loc[idx, 'CIR_TOTAL_CALCULE']
        corrected = calculated if abs(declared) < 0.01 else (declared + calculated) / 2
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = corrected; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_dbl_creance_totale"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_ecart_petit(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_entre_-500_et_500.csv"):
    """Corrige les petits écarts (-500 à 500 EUR) en prenant la moyenne pour 2021."""
    print("\n6. CORRECTION PETIT ÉCART (-500 à 500 EUR) 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        declared = df.loc[idx, 'CIR_TOTAL_DECLARE']; calculated = df.loc[idx, 'CIR_TOTAL_CALCULE']
        corrected = (declared + calculated) / 2
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = corrected; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_petite"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_plafond_partout(df, processed_sirens, fichier_intermediaire="lignes_ecart_relatif_moins_100.csv"):
    """Corrige les cas où l'écart est -100% (calculé=0), utilise la valeur calculée (0) pour 2021."""
    print("\n7. CORRECTION PLAFOND PARTOUT (ÉCART -100%) 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE'] # Should be ~0
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Err_PLAF_EVW"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_innovation(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_egal_innovation.csv"):
    """Corrige les cas où l'écart total correspond au CIR Innovation calculé pour 2021."""
    print("\n8. CORRECTION CIR INNOVATION MANQUANT (ÉCART = INNOVATION) 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "innov_manquant"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_collection(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_egal_collection.csv"):
    """Corrige les cas où l'écart total correspond au CIR Collection calculé pour 2021."""
    print("\n9. CORRECTION CIR COLLECTION MANQUANT (ÉCART = COLLECTION) 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "collect_manquant"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_cir_declare_zero(df, processed_sirens, fichier_intermediaire="lignes_ecart_100_cir_declare_0.csv"):
    """Corrige les cas où le CIR déclaré est 0 mais le calculé est positif pour 2021."""
    print("\n10. CORRECTION CIR DÉCLARÉ = 0 (ÉCART +100%) 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = 0.0 # Déclaré est 0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "err_cir_total_declarer"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

def correction_erreurs_l6(df, processed_sirens, fichier_intermediaire="lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv"):
    """Corrige les erreurs liées à la ligne L6 (Autres dépenses de fonctionnement) pour 2021."""
    print("\n11. CORRECTION ERREURS LIÉES À L6 - 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value; df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "err_l6"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count



def correction_valeur_calculee_reste(df, processed_sirens, fichier_intermediaire="lignes_restantes_a_analyser.csv"):
    """Applique la valeur calculée aux cas restants identifiés pour 2021."""
    print("\n13. VALEUR CALCULÉE PRIVILÉGIÉE POUR LE RESTE IDENTIFIÉ - 2021")
    df_error = charger_fichier_intermediaire(fichier_intermediaire, df.columns)
    if df_error.empty: return 0
    count = 0
    before_sum = df_error['CIR_TOTAL_DECLARE'].sum() if 'CIR_TOTAL_DECLARE' in df_error.columns else 0.0
    indices_traites_localement = []
    for _, row in df_error.iterrows():
        siren = row['SIREN_DECLARANT']
        if siren in processed_sirens: continue
        mask = df['SIREN_DECLARANT'] == siren
        if not mask.any(): continue
        idx = df.index[mask].tolist()[0]
        calculated_value = df.loc[idx, 'CIR_TOTAL_CALCULE']
        df.loc[idx, 'CIR_TOTAL_CORRIGE'] = calculated_value
        df.loc[idx, 'TRAITEMENT_APPLIQUE'] = "Valeur calculée privilégiée(Reste)"
        processed_sirens.add(siren); indices_traites_localement.append(idx); count += 1
    after_sum = df.loc[indices_traites_localement, 'CIR_TOTAL_CORRIGE'].sum() if indices_traites_localement else 0.0
    print(f"-> {len(df_error)} lignes lues, {count} corrigées. Impact local: {after_sum - before_sum:,.2f} €")
    return count

# ===================================================================
# FONCTION PRINCIPALE (Orchestration - Pour 2021)
# ===================================================================

def corriger_cir(file_path="Calcul_Creance_CIR_2021.csv"):
    """Fonction principale pour charger, corriger et sauvegarder le CIR 2021."""
    df = charger_donnees(file_path)
    if df is None:
        print("\nArrêt du traitement.")
        return None

    original_declared = df['CIR_TOTAL_DECLARE'].sum()
    original_calculated = df['CIR_TOTAL_CALCULE'].sum()
    initial_corrected_sum = df['CIR_TOTAL_CORRIGE'].sum()
    print(f"\nSommes avant corrections spécifiques:")
    print(f"  - CIR Déclaré Total: {original_declared:,.2f} €")
    print(f"  - CIR Calculé Total: {original_calculated:,.2f} €")
    print(f"  - CIR Corrigé Initial Total: {initial_corrected_sum:,.2f} €")

    processed_sirens = set()
    total_corrected_count = 0

    print("\n--- Début de l'application des corrections spécifiques 2021 ---")
    # Liste des fonctions de correction et de leurs fichiers associés (pour 2021)
    correction_definitions = [
        (correction_erreurs_personnel, "lignes_jeunes_docteurs_0_amort_sinistr_non0.csv"), #1
        (correction_erreurs_frais_brevets, "lignes_L7_egal_L8.csv"), #2
        (correction_cir_recherche_manquant, "lignes_depenses_non0_cir_recherche_0.csv"), #3
        (correction_doublement_sans_motif, "lignes_ecart_relatif_50_innovation0_recherche1.csv"), #5
        (correction_ecart_petit, "lignes_ecart_total_entre_-500_et_500.csv"), #6
        (correction_plafond_partout, "lignes_ecart_relatif_moins_100.csv"), #7
        (correction_cir_innovation, "lignes_ecart_total_egal_innovation.csv"), #8
        (correction_cir_collection, "lignes_ecart_total_egal_collection.csv"), #9
        (correction_cir_declare_zero, "lignes_ecart_100_cir_declare_0.csv"), #10
        (correction_erreurs_l6, "lignes_ecart_total_egal_recherche_l6_ecart_hors_1.csv"), #11
        (correction_valeur_calculee_reste, "lignes_restantes_a_analyser_2021.csv") #13
    ]

    # Exécution séquentielle des corrections
    for func, file in correction_definitions:
        count = func(df, processed_sirens, file)
        total_corrected_count += count

    print(f"\n--- Fin de l'application des corrections spécifiques 2021 ---")
    print(f"Nombre total de modifications appliquées par les fonctions actives: {total_corrected_count:,}")
    print(f"Nombre de SIRENs uniques traités: {len(processed_sirens):,}")

    # --- Ajustement final : Mise à zéro des CIR corrigés négatifs ---
    print("\n--- Ajustement final : Mise à zéro des CIR corrigés négatifs ---")
    if 'CIR_TOTAL_CORRIGE' in df.columns:
        # Assurer le type numérique avant la comparaison
        if not pd.api.types.is_numeric_dtype(df['CIR_TOTAL_CORRIGE']):
             df['CIR_TOTAL_CORRIGE'] = df['CIR_TOTAL_CORRIGE'].apply(convertir_en_nombre)

        negative_mask = df['CIR_TOTAL_CORRIGE'] < 0
        count_negative = negative_mask.sum()
        if count_negative > 0:
            sum_before_zeroing = df.loc[negative_mask, 'CIR_TOTAL_CORRIGE'].sum()
            df.loc[negative_mask, 'CIR_TOTAL_CORRIGE'] = 0.0
            print(f"{count_negative:,} lignes avec CIR_TOTAL_CORRIGE négatif ont été mises à 0 (Impact: {-sum_before_zeroing:,.2f} €).")
        else: print("Aucune valeur négative trouvée dans CIR_TOTAL_CORRIGE après corrections.")
    else: print("Erreur: Colonne 'CIR_TOTAL_CORRIGE' non trouvée pour l'ajustement final.")

    # --- Finalisation des lignes 'Non traité' ---
    mask_non_traite_final = df['TRAITEMENT_APPLIQUE'] == "Non traité"
    count_non_traite = mask_non_traite_final.sum()
    print(f"\nConfirmation pour les lignes restées 'Non traité' ({count_non_traite:,}):")
    if count_non_traite > 0:
        # Assurer numérique et non-négatif
        df.loc[mask_non_traite_final, 'CIR_TOTAL_CORRIGE'] = df.loc[mask_non_traite_final, 'CIR_TOTAL_CORRIGE'].apply(convertir_en_nombre)
        neg_in_non_traite_mask = mask_non_traite_final & (df['CIR_TOTAL_CORRIGE'] < 0)
        count_neg_in_non_traite = neg_in_non_traite_mask.sum()
        if count_neg_in_non_traite > 0:
             sum_neg_non_traite = df.loc[neg_in_non_traite_mask, 'CIR_TOTAL_CORRIGE'].sum()
             df.loc[neg_in_non_traite_mask, 'CIR_TOTAL_CORRIGE'] = 0.0
             print(f"  - {count_neg_in_non_traite} de ces lignes étaient négatives et mises à 0 (impact: {-sum_neg_non_traite:,.2f} €).")
        else: print("  - Aucune valeur négative parmi les lignes 'Non traité'.")
    else: print("  - Aucune ligne marquée comme 'Non traité'.")

    # --- Calcul des totaux finaux ---
    final_total_corrected = df['CIR_TOTAL_CORRIGE'].sum()

    # --- Sauvegarde avec nouveau chemin local ---
    output_file = "C:/Users/msamb/Documents/Calcul_Creance_CIR_Corrige_2021.csv"
    print(f"\nSauvegarde du fichier corrigé sous: {output_file}")
    try:
        df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False, decimal='.')
        print(f"Fichier sauvegardé avec succès.")
    except Exception as e: 
        print(f"ERREUR lors de la sauvegarde: {str(e)}")

    # --- Résumé Final ---
    print("\n=====================================================")
    print("                RÉSUMÉ FINAL 2021")
    print("=====================================================")
    print(f"Montant total CIR déclaré (Original):       {original_declared:,.2f} €")
    print(f"Montant total CIR calculé (Original):       {original_calculated:,.2f} €")
    print(f"Montant total CIR après corrections:        {final_total_corrected:,.2f} €")
    initial_ecart = original_calculated - original_declared
    final_ecart = final_total_corrected - original_declared
    improvement = abs(initial_ecart) - abs(final_ecart)
    print(f"\nÉcart initial (calculé - déclaré):        {initial_ecart:,.2f} €")
    print(f"Écart final (corrigé - déclaré):          {final_ecart:,.2f} €")
    print(f"Réduction de l'écart absolu:              {improvement:,.2f} €")
    if abs(original_declared) > 0.01:
        print(f"  Écart relatif initial:                  {initial_ecart / original_declared * 100:.2f}%")
        print(f"  Écart relatif final:                    {final_ecart / original_declared * 100:.2f}%")
        if abs(initial_ecart) > 0.01: print(f"  Réduction relative de l'écart:          {improvement / abs(initial_ecart) * 100:.2f}%")
    else: print("\nCalculs relatifs non pertinents (CIR déclaré total proche de zéro).")

    print("\n--- Détails par Type de Traitement Appliqué ---")
    if 'TRAITEMENT_APPLIQUE' in df.columns:
        treatment_counts = df['TRAITEMENT_APPLIQUE'].value_counts().sort_index()
        print("\nNombre d'entreprises par type de traitement final:")
        for treatment, count in treatment_counts.items(): print(f"  - {treatment:<40}: {count:10,}")
        print("\nDétails financiers par type de traitement final:")
        treatment_order = treatment_counts.index.tolist()
        for treatment in treatment_order:
            subset = df[df['TRAITEMENT_APPLIQUE'] == treatment]
            if len(subset) == 0: continue
            declared_subset = subset['CIR_TOTAL_DECLARE'].sum()
            corrected_subset = subset['CIR_TOTAL_CORRIGE'].sum()
            ecart_final_subset = corrected_subset - declared_subset
            print(f"\nTraitement: {treatment}")
            print(f"  Nombre d'entreprises:                 {len(subset):<10,}")
            print(f"  CIR déclaré (groupe):                 {declared_subset:>18,.2f} €")
            print(f"  CIR corrigé final (groupe):           {corrected_subset:>18,.2f} €")
            print(f"  Écart final (corrigé-déclaré groupe): {ecart_final_subset:>18,.2f} €")
            if abs(declared_subset) > 0.01:
                impact_relatif = ecart_final_subset / declared_subset * 100
                print(f"  Impact relatif final (groupe):        {impact_relatif:>17.2f}%")
    else: print("\nColonne 'TRAITEMENT_APPLIQUE' non trouvée.")

    print("\n=====================================================")
    print("Traitement terminé pour 2021.")
    print("=====================================================")
    return df

# ===================================================================
# EXÉCUTION
# ===================================================================
if __name__ == "__main__":
    # Chemin adapté pour 2021
    input_filename = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv"
    print(f"*** DÉBUT DU SCRIPT DE CORRECTION CIR 2021 ***")
    # Affichage de la date et heure actuelles
    current_time_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"Date et heure: {current_time_str}")
    print(f"Fichier d'entrée: {input_filename}")
    df_corrige = corriger_cir(input_filename)
    if df_corrige is not None: print("\nScript terminé avec succès.")
    else: print("\nLe script n'a pas pu se terminer correctement.")

*** DÉBUT DU SCRIPT DE CORRECTION CIR 2021 ***
Date et heure: 2025-06-11 14:15:55
Fichier d'entrée: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv
Chargement et préparation des données depuis: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//Calcul_Creance_CIR_2021.csv...
Fichier principal chargé: 4,493 lignes
Formatage des SIRENs...
Conversion des colonnes numériques...
Initialisation de la colonne 'CIR_TOTAL_CORRIGE'...
Préparation des données terminée.

Sommes avant corrections spécifiques:
  - CIR Déclaré Total: 470,488,283.00 €
  - CIR Calculé Total: 453,725,388.47 €
  - CIR Corrigé Initial Total: 453,725,388.47 €

--- Début de l'application des corrections spécifiques 2021 ---

1. CORRECTION ERREURS DÉPENSES PERSONNEL (JEUNES DOCTEURS) 2021
-> 2 lignes lues, 2 corrigées. Impact local: 87,933.81 €

2. CORRECTION ERREUR L7 = L8 (FRAIS BREVETS) 2021

3. CORRECTION CIR RECHERCHE MANQUANT/INCOHÉRENT 2021
-> 18 lignes lues, 0 corrigées. Impact loca

# Fusion des 2 fichiers pour avoir toute les variables dont nous avons besoin pour l'etude 

In [57]:
import pandas as pd

# 1. Charger les fichiers originaux
df_init = pd.read_excel("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//output//output_parquet//new_millesime_CIR_2021-suite_corrected_sans_doublon.xlsx")
df_corrige = pd.read_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_2021.csv", 
                         sep=';', encoding='utf-8-sig', decimal='.', low_memory=False)

# 2. Standardiser les SIREN déposants puisque c'est la meilleure correspondance
def standardize_siren(siren):
    """Standardise le SIREN en supprimant tous les caractères non numériques et en complétant avec des zéros."""
    if pd.isna(siren):
        return None
    siren_str = str(siren).replace(' ', '').replace('-', '').replace('.', '')
    # Supprimer tous les caractères non numériques
    siren_str = ''.join(c for c in siren_str if c.isdigit())
    # Garder seulement les 9 derniers chiffres pour les SIREN longs
    if len(siren_str) > 9:
        siren_str = siren_str[-9:]
    # Compléter avec des zéros au début si nécessaire
    return siren_str.zfill(9)

# 3. Appliquer la standardisation
df_init['siren_deposant_std'] = df_init['siren_deposant'].apply(standardize_siren)
df_corrige['SIREN_DEPOSANT_STD'] = df_corrige['SIREN_DEPOSANT'].apply(standardize_siren)

# 4. Vérifier la correspondance
common_sirens = set(df_corrige['SIREN_DEPOSANT_STD']) & set(df_init['siren_deposant_std'])
print(f"Nombre de SIREN communs: {len(common_sirens)}")
print(f"Nombre de SIREN uniques dans df_corrige: {df_corrige['SIREN_DEPOSANT_STD'].nunique()}")
print(f"Nombre de SIREN uniques dans df_init: {df_init['siren_deposant_std'].nunique()}")

# 5. Créer les dictionnaires de correspondance pour les deux colonnes
siren_to_type = dict(zip(df_init['siren_deposant_std'], df_init['type_final']))
siren_to_mere = dict(zip(df_init['siren_deposant_std'], df_init['mere_final']))

# 6. Appliquer les deux correspondances en même temps
df_corrige['type_final'] = df_corrige['SIREN_DEPOSANT_STD'].map(siren_to_type)
df_corrige['mere_final'] = df_corrige['SIREN_DEPOSANT_STD'].map(siren_to_mere)

# 7. Vérifier les résultats
type_non_null = df_corrige['type_final'].notna().sum()
mere_non_null = df_corrige['mere_final'].notna().sum()
print(f"Nombre de lignes avec type_final non-null: {type_non_null} sur {len(df_corrige)}")
print(f"Nombre de lignes avec mere_final non-null: {mere_non_null} sur {len(df_corrige)}")

# 8. Supprimer la colonne temporaire
df_corrige = df_corrige.drop(columns=['SIREN_DEPOSANT_STD'])

# 9. Sauvegarder le résultat
df_corrige.to_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_Final_2021.csv", 
                  sep=';', encoding='utf-8-sig', index=False)

print("Fichier sauvegardé avec succès avec les deux colonnes ajoutées.")

# 10. Afficher des exemples des résultats
print("\nExemples de résultats avec les deux colonnes ajoutées:")
sample = df_corrige[['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'TRAITEMENT_APPLIQUE', 'type_final', 'mere_final']].sample(5)
print(sample)

# 11. Afficher les distributions des valeurs
print("\nDistribution des valeurs dans type_final:")
print(df_corrige['type_final'].value_counts().head(10))

print("\nDistribution des valeurs dans mere_final:")
print(df_corrige['mere_final'].value_counts().head(10))

Nombre de SIREN communs: 2997
Nombre de SIREN uniques dans df_corrige: 2997
Nombre de SIREN uniques dans df_init: 2997
Nombre de lignes avec type_final non-null: 4493 sur 4493
Nombre de lignes avec mere_final non-null: 1694 sur 4493
Fichier sauvegardé avec succès avec les deux colonnes ajoutées.

Exemples de résultats avec les deux colonnes ajoutées:
      SIREN_DECLARANT  SIREN_DEPOSANT TRAITEMENT_APPLIQUE type_final  \
4372        900155003       900155003          Non traité        IND   
2957        798561684       798561684        Err_PLAF_EVW        IND   
635         399047752       399047752          Non traité        IND   
876         435324280       435324280          Non traité        IND   
3732        844238410       844238410          Non traité        IND   

      mere_final  
4372         NaN  
2957         NaN  
635          NaN  
876          NaN  
3732         NaN  

Distribution des valeurs dans type_final:
type_final
IND      2799
FILLE    1660
MERE       34
Name

In [58]:
# 1. Charger le fichier corrigé existant
df_corrige = pd.read_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_Final_2021.csv", 
                         sep=';', encoding='utf-8-sig', low_memory=False)

# 2. Standardiser et nettoyer les SIREN
df_corrige['SIREN_DECLARANT_STD'] = df_corrige['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)
df_corrige['SIREN_DEPOSANT_STD'] = df_corrige['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)

# 3. Créer les ensembles de SIREN déclarants et déposants
set_declarants = set(df_corrige['SIREN_DECLARANT_STD'])
set_deposants = set(df_corrige['SIREN_DEPOSANT_STD'])

# 4. Identifier les SIREN qui sont déposants mais pas déclarants (en filtrant les SIREN non valides)
#if siren and not (siren.strip('0') == '')
deposants_non_declarants = {siren for siren in (set_deposants - set_declarants) }
print(f"Nombre de SIREN qui sont déposants mais pas déclarants: {len(deposants_non_declarants)}")

# 5. Créer les nouvelles lignes pour les déposants manquants
new_rows = []

for siren in deposants_non_declarants:
    # Créer un dictionnaire pour la nouvelle ligne
    new_row = {}
    
    # Copier la structure d'une ligne existante pour obtenir toutes les colonnes
    for col in df_corrige.columns:
        if col in ['SIREN_DECLARANT_STD', 'SIREN_DEPOSANT_STD']:
            continue  # On n'inclut pas ces colonnes temporaires
        
        # Déterminer le type et la valeur appropriée
        if col == 'SIREN_DECLARANT' or col == 'SIREN_DEPOSANT':
            # Utiliser toujours le format chaîne pour éviter les problèmes
            new_row[col] = siren
        elif col == 'TRAITEMENT_APPLIQUE':
            new_row[col] = 'Déposant ajouté'
        elif col == 'type_final':  # Ajouter "MERE" dans la colonne type_retenu
            new_row[col] = 'MERE'
        elif col == 'DESIGNATION' or col == 'COMPLEMENT_DESIGNATION':
            new_row[col] = '' # Champs texte vide
        elif pd.api.types.is_numeric_dtype(df_corrige[col]):
            # Pour les colonnes numériques, mettre 0
            new_row[col] = 0
        else:
            # Pour les autres colonnes non numériques, mettre une valeur vide
            new_row[col] = ''
    
    # Ajouter la ligne au tableau de nouvelles lignes
    new_rows.append(new_row)

# 6. Créer un DataFrame avec les nouvelles lignes
if new_rows:
    df_new_rows = pd.DataFrame(new_rows)
    
    # 7. Concaténer avec le DataFrame original
    df_corrige_complet = pd.concat([df_corrige.drop(['SIREN_DECLARANT_STD', 'SIREN_DEPOSANT_STD'], axis=1),
                                df_new_rows],
                               ignore_index=True)
    
    # 8. Afficher des informations sur le résultat
    print(f"DataFrame original: {len(df_corrige)} lignes")
    print(f"Nouvelles lignes ajoutées: {len(new_rows)} lignes")
    print(f"DataFrame final: {len(df_corrige_complet)} lignes")
    
    # 9. Sauvegarder le résultat
    output_file = "C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_avec_deposants.csv"
    df_corrige_complet.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
    print(f"\nFichier sauvegardé: {output_file}")
    
    # 10. Afficher un exemple des nouvelles lignes
    print("\nExemple d'une nouvelle ligne ajoutée:")
    if not df_new_rows.empty:
        cols_to_show = ['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'TRAITEMENT_APPLIQUE', 'type_final', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_CORRIGE']
        print(df_new_rows[cols_to_show].iloc[0])
else:
    print("Aucun déposant non déclarant trouvé, aucune ligne à ajouter.")

Nombre de SIREN qui sont déposants mais pas déclarants: 50
DataFrame original: 4493 lignes
Nouvelles lignes ajoutées: 50 lignes
DataFrame final: 4543 lignes

Fichier sauvegardé: C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_avec_deposants.csv

Exemple d'une nouvelle ligne ajoutée:
SIREN_DECLARANT              537685885
SIREN_DEPOSANT               537685885
TRAITEMENT_APPLIQUE    Déposant ajouté
type_final                        MERE
CIR_TOTAL_DECLARE                    0
CIR_TOTAL_CALCULE                    0
CIR_TOTAL_CORRIGE                    0
Name: 0, dtype: object


## Difference pour verifier si tous les deposants sont declarants.

In [59]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_avec_deposants.csv", 
                 sep=';', encoding='utf-8-sig', low_memory=False)

# Standardiser les SIREN (en string, 9 caractères, sans espaces)
set_declarants = set(df['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9))
set_deposants = set(df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9))

# Différence symétrique (éléments dans l'un ou l'autre ensemble, mais pas les deux)
difference_symetrique = set_declarants ^ set_deposants

print(f"Nombre de SIREN présents soit comme déclarant, soit comme déposant (mais pas les deux) : {len(difference_symetrique)}")

if len(difference_symetrique) == 0:
    print("La différence symétrique est vide.")
else:
    print("Exemples de SIREN présents dans un seul ensemble :", list(difference_symetrique)[:5])
    
    # Si vous voulez aussi voir spécifiquement les SIREN qui sont uniquement déclarants
    seulement_declarants = set_declarants - set_deposants
    print(f"\nNombre de SIREN qui sont uniquement déclarants : {len(seulement_declarants)}")
    if len(seulement_declarants) > 0:
        print("Exemples:", list(seulement_declarants)[:5])
    
    # Et les SIREN qui sont uniquement déposants
    seulement_deposants = set_deposants - set_declarants
    print(f"Nombre de SIREN qui sont uniquement déposants : {len(seulement_deposants)}")
    if len(seulement_deposants) > 0:
        print("Exemples:", list(seulement_deposants)[:5])

Nombre de SIREN présents soit comme déclarant, soit comme déposant (mais pas les deux) : 1526
Exemples de SIREN présents dans un seul ensemble : ['808233399', '537915357', '487474306', '437181217', '418736526']

Nombre de SIREN qui sont uniquement déclarants : 1526
Exemples: ['808233399', '537915357', '487474306', '437181217', '418736526']
Nombre de SIREN qui sont uniquement déposants : 0


In [60]:
import pandas as pd
import numpy as np

# 1. Charger le fichier
print("Chargement du fichier...")
df = pd.read_csv("C://Users//msamb//Documents//Calcul_Creance_CIR_Corrige_avec_deposants.csv", 
                 sep=';', encoding='utf-8-sig', low_memory=False)

# 2. Standardiser les SIREN des déposants pour éviter les problèmes de format
print("Standardisation des SIREN...")
df['SIREN_DEPOSANT_STD'] = df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)

# 3. Calculer la somme du CIR par déposant
print("Calcul de la somme du CIR par déposant...")
# Utiliser CIR_TOTAL_CORRIGE qui est le montant final retenu après corrections
cir_par_deposant = df.groupby('SIREN_DEPOSANT_STD')['CIR_TOTAL_CORRIGE'].sum().reset_index()
cir_par_deposant.rename(columns={'CIR_TOTAL_CORRIGE': 'cir_benef_total_temp'}, inplace=True)

# 4. Fusionner cette somme avec le DataFrame original
print("Ajout de la nouvelle colonne au DataFrame...")
df = df.merge(cir_par_deposant, on='SIREN_DEPOSANT_STD', how='left')

# 5. Mettre à zéro la colonne cir_benef_total pour toutes les lignes qui ne sont pas de type MERE
print("Attribution des valeurs selon le type...")
# Créer la colonne finale 'cir_benef_total'
df['cir_benef_total'] = 0.0  # Initialiser à zéro pour toutes les lignes

# Mettre la somme calculée uniquement pour les lignes de type 'MERE' et 'IND'
df.loc[df['type_final'] == 'MERE', 'cir_benef_total'] = df.loc[df['type_final'] == 'MERE', 'cir_benef_total_temp']
df.loc[df['type_final'] == 'IND', 'cir_benef_total'] = df.loc[df['type_final'] == 'IND', 'cir_benef_total_temp']

# Supprimer la colonne temporaire
df.drop(['SIREN_DEPOSANT_STD', 'cir_benef_total_temp'], axis=1, inplace=True)

# 6. Afficher quelques statistiques
print(f"\nNombre total d'entreprises: {len(df)}")
print(f"Nombre de déposants uniques: {df['SIREN_DEPOSANT'].nunique()}")

# Vérifier la distribution des valeurs cir_benef_total selon le type
print("\nDistribution des valeurs cir_benef_total par type:")
print(df.groupby('type_final')['cir_benef_total'].describe())

# Trier par montant de cir_benef_total pour voir les déposants avec le plus grand total
top_deposants = df[df['cir_benef_total'] > 0][['SIREN_DEPOSANT', 'cir_benef_total']].drop_duplicates().sort_values('cir_benef_total', ascending=False)
print("\nTop 5 des déposants (MERE) avec le plus grand total de CIR:")
print(top_deposants.head(5))

# 7. Sauvegarder le résultat
output_file = "C://Users//msamb//Documents//Calcul_Creance_CIR_avec_benef_total.csv"
df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
print(f"\nFichier sauvegardé: {output_file}")

# 8. Vérification - calcul de la somme totale de CIR
total_cir_corrige = df['CIR_TOTAL_CORRIGE'].sum()
total_cir_benef = df['cir_benef_total'].sum()
print(f"\nSomme totale de CIR_TOTAL_CORRIGE: {total_cir_corrige:,.2f} €")
print(f"Somme totale de cir_benef_total (pour les MERE et IND): {total_cir_benef:,.2f} €")

Chargement du fichier...
Standardisation des SIREN...
Calcul de la somme du CIR par déposant...
Ajout de la nouvelle colonne au DataFrame...
Attribution des valeurs selon le type...

Nombre total d'entreprises: 4543
Nombre de déposants uniques: 2997

Distribution des valeurs cir_benef_total par type:
             count           mean           std  min  25%        50%  \
type_final                                                             
FILLE       1660.0       0.000000  0.000000e+00  0.0  0.0       0.00   
IND         2799.0   42687.906049  6.096652e+05  0.0  0.0       0.00   
MERE          84.0  939523.869524  4.764457e+06  0.0  0.0  125607.19   

                    75%          max  
type_final                            
FILLE            0.0000         0.00  
IND              0.0000  29833356.67  
MERE        249960.1325  42695101.36  

Top 5 des déposants (MERE) avec le plus grand total de CIR:
      SIREN_DEPOSANT  cir_benef_total
4505       552059024      42695101.36
452  

In [61]:
import pandas as pd
import numpy as np

# Charger le fichier
df = pd.read_csv(output_file, sep=';', encoding='utf-8-sig', low_memory=False)

# Standardiser les SIREN des déposants et des déclarants
df['SIREN_DEPOSANT_STD'] = df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)
df['SIREN_DECLARANT_STD'] = df['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.strip().str.zfill(9)

# Créer une colonne pour la correspondance
df['IS_MATCH'] = df['SIREN_DEPOSANT_STD'] == df['SIREN_DECLARANT_STD']

# Calculer la somme du CIR par déposant
cir_total_par_deposant = df.groupby('SIREN_DEPOSANT_STD')['CIR_TOTAL_CORRIGE'].sum().reset_index()
cir_total_par_deposant.rename(columns={'CIR_TOTAL_CORRIGE': 'TOTAL_CIR'}, inplace=True)

# Initialiser la nouvelle colonne
df['cir_benef_total'] = 0.0

# Méthode simplifiée: traiter chaque SIREN déposant séparément
for siren in cir_total_par_deposant['SIREN_DEPOSANT_STD'].unique():
    montant_total = cir_total_par_deposant.loc[cir_total_par_deposant['SIREN_DEPOSANT_STD'] == siren, 'TOTAL_CIR'].values[0]
    
    # Sélectionner toutes les lignes avec ce SIREN déposant
    lignes_siren = df[df['SIREN_DEPOSANT_STD'] == siren]
    
    # Vérifier s'il y a des lignes correspondantes (SIREN_DEPOSANT = SIREN_DECLARANT)
    lignes_match = lignes_siren[lignes_siren['IS_MATCH']]
    
    if len(lignes_match) > 0:
        # S'il y a des correspondances, distribuer le montant également entre ces lignes
        montant_par_ligne = montant_total / len(lignes_match)
        df.loc[lignes_match.index, 'cir_benef_total'] = montant_par_ligne
    else:
        # S'il n'y a pas de correspondance, mettre le montant total sur la première ligne
        df.loc[lignes_siren.index[0], 'cir_benef_total'] = montant_total

# Vérification
total_cir_corrige = df['CIR_TOTAL_CORRIGE'].sum()
total_cir_benef = df['cir_benef_total'].sum()

# Créer un fichier de vérification avec les totaux par SIREN déposant
verification = df.groupby('SIREN_DEPOSANT_STD')[['CIR_TOTAL_CORRIGE', 'cir_benef_total']].sum()
verification['difference'] = verification['CIR_TOTAL_CORRIGE'] - verification['cir_benef_total']
verification.to_csv("M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//verification_totaux_par_siren.csv", sep=';')

# Comptage des SIREN sans correspondance
siren_avec_match = df[df['IS_MATCH']]['SIREN_DEPOSANT_STD'].unique()
siren_tous = df['SIREN_DEPOSANT_STD'].unique()
siren_sans_match = set(siren_tous) - set(siren_avec_match)

# Supprimer les colonnes temporaires avant de sauvegarder
df = df.drop(['SIREN_DEPOSANT_STD', 'SIREN_DECLARANT_STD', 'IS_MATCH'], axis=1)

# Sauvegarder le résultat
output_file_final = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final.csv"
df.to_csv(output_file_final, sep=';', encoding='utf-8-sig', index=False)

print(f"Fichier sauvegardé: {output_file_final}")
print(f"\nSomme totale de CIR_TOTAL_CORRIGE: {total_cir_corrige:,.2f} €")
print(f"Somme totale de cir_benef_total: {total_cir_benef:,.2f} €")
print(f"Différence entre les deux sommes: {total_cir_corrige - total_cir_benef:,.2f} €")
print(f"\nNombre de SIREN déposants sans correspondance: {len(siren_sans_match)}")

# Montant CIR pour les SIREN sans correspondance
cir_sans_match = sum(verification.loc[[s for s in siren_sans_match if s in verification.index], 'CIR_TOTAL_CORRIGE'])
print(f"Montant CIR pour ces SIREN: {cir_sans_match:,.2f} € ({cir_sans_match/total_cir_corrige*100:.2f}% du total)")

Fichier sauvegardé: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final.csv

Somme totale de CIR_TOTAL_CORRIGE: 453,738,653.93 €
Somme totale de cir_benef_total: 453,738,653.93 €
Différence entre les deux sommes: 0.00 €

Nombre de SIREN déposants sans correspondance: 1
Montant CIR pour ces SIREN: 0.00 € (0.00% du total)


In [62]:
import pandas as pd

# Charger le fichier CSV
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')

# Dictionnaire de renommage des colonnes
renommage = {
    # Renommage des colonnes CIR recherche
    'CIR_RECHERCHE_DECLARE': 'crerd_gen_declare',
    'CIR_RECHERCHE_CALCULE': 'crerd_gen_calcule',
    'CIR_RECHERCHE_ECART': 'crerd_gen_ecart',
    
    # Renommage des colonnes CIR collection
    'CIR_COLLECTION_DECLARE': 'crecoll_gen_declare',
    'CIR_COLLECTION_CALCULE': 'crecoll_gen_calcule',
    'CIR_COLLECTION_ECART': 'crecoll_gen_ecart',
    
    # Renommage des colonnes CIR innovation
    'CIR_INNOVATION_DECLARE': 'creinno_gen_declare',
    'CIR_INNOVATION_CALCULE': 'creinno_gen_calcule',
    'CIR_INNOVATION_ECART': 'creinno_gen_ecart',
    
    # Renommage des colonnes CIR collaboratif
    'CIR_COLLABORATIF_DECLARE': 'crecrc_declare',
    'CIR_COLLABORATIF_CALCULE': 'crecrc_calcule',
    'CIR_COLLABORATIF_ECART': 'crecrc_ecart',
    
    # Renommage des colonnes CIR total
    'CIR_TOTAL_DECLARE': 'cretot_gen_declare',
    'CIR_TOTAL_CALCULE': 'cretot_gen_calcule',
    'CIR_TOTAL_ECART': 'cretot_gen_ecart',
    'CIR_TOTAL_CORRIGE': 'cretot_gen_corrige'
}

# Appliquer le renommage (uniquement pour les colonnes qui existent)
colonnes_existantes = {col: nouveau_nom for col, nouveau_nom in renommage.items() if col in df.columns}
df = df.rename(columns=colonnes_existantes)

# Sauvegarder le fichier avec les nouvelles colonnes
output_file = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final_renamed.csv"
df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)

print(f"Fichier sauvegardé avec succès: {output_file}")
print(f"Colonnes renommées: {list(colonnes_existantes.keys())}")

Fichier sauvegardé avec succès: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final_renamed.csv
Colonnes renommées: ['CIR_RECHERCHE_DECLARE', 'CIR_RECHERCHE_CALCULE', 'CIR_RECHERCHE_ECART', 'CIR_COLLECTION_DECLARE', 'CIR_COLLECTION_CALCULE', 'CIR_COLLECTION_ECART', 'CIR_INNOVATION_DECLARE', 'CIR_INNOVATION_CALCULE', 'CIR_INNOVATION_ECART', 'CIR_TOTAL_DECLARE', 'CIR_TOTAL_CALCULE', 'CIR_TOTAL_ECART', 'CIR_TOTAL_CORRIGE']


In [63]:
import pandas as pd
import numpy as np

# Charger le fichier avec les types de données spécifiés pour améliorer les performances
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_avec_benef_total_final.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')

print(f"Nombre total de lignes: {len(df)}")

# 1. RENOMMER TOUTES LES COLONNES (opération rapide)
renommage_complet = {
    'CIR_RECHERCHE_DECLARE': 'crerd_gen_declare',
    'CIR_RECHERCHE_CALCULE': 'crerd_gen',
    'CIR_RECHERCHE_ECART': 'crerd_gen_ecart',
    'CIR_COLLECTION_DECLARE': 'crecoll_gen_declare',
    'CIR_COLLECTION_CALCULE': 'crecoll_gen',
    'CIR_COLLECTION_ECART': 'crecoll_gen_ecart',
    'CIR_INNOVATION_DECLARE': 'creinno_gen_declare',
    'CIR_INNOVATION_CALCULE': 'creinno_gen',
    'CIR_INNOVATION_ECART': 'creinno_gen_ecart',
    'CIR_COLLABORATIF_DECLARE': 'crecrc_declare',
    'CIR_COLLABORATIF_CALCULE': 'crecrc',
    'CIR_COLLABORATIF_ECART': 'crecrc_ecart',
    'CIR_TOTAL_DECLARE': 'cretot_gen_declare',
    'CIR_TOTAL_CALCULE': 'cretot_gen_calcule',
    'CIR_TOTAL_ECART': 'cretot_gen_ecart',
    'CIR_TOTAL_CORRIGE': 'cretot_gen'
}

# Renommer uniquement les colonnes qui existent
colonnes_existantes = {col: nouveau_nom for col, nouveau_nom in renommage_complet.items() if col in df.columns}
df = df.rename(columns=colonnes_existantes)

print(f"Colonnes renommées: {len(colonnes_existantes)}")

# 2. CALCULER LES CRÉDITS BÉNÉFICIAIRES DE MANIÈRE OPTIMISÉE
# Standardiser les SIREN une seule fois
df['SIREN_DEPOSANT_STD'] = df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(9)
df['SIREN_DECLARANT_STD'] = df['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(9)
df['IS_MATCH'] = df['SIREN_DEPOSANT_STD'] == df['SIREN_DECLARANT_STD']

# Version vectorisée et optimisée pour calculer les valeurs bénéficiaires
def calculer_benef_optimise(df, colonne_source):
    # Créer un DataFrame avec les informations essentielles
    df_temp = df[['SIREN_DEPOSANT_STD', 'IS_MATCH', colonne_source]].copy()
    
    # Calculer la somme totale par déposant
    sommes = df_temp.groupby('SIREN_DEPOSANT_STD')[colonne_source].sum().to_dict()
    
    # Créer une série pour stocker les résultats
    resultats = pd.Series(index=df.index, dtype='float64')
    
    # Pour chaque SIREN déposant unique
    for siren, montant in sommes.items():
        # Lignes correspondant à ce SIREN
        mask_siren = df['SIREN_DEPOSANT_STD'] == siren
        
        # Lignes où il y a correspondance
        mask_match = mask_siren & df['IS_MATCH']
        count_match = mask_match.sum()
        
        if count_match > 0:
            # Distribuer le montant également entre les lignes correspondantes
            resultats[mask_match] = montant / count_match
        else:
            # Première ligne du SIREN
            if mask_siren.any():
                idx = df[mask_siren].index[0]
                resultats[idx] = montant
    
    return resultats

# Calculer tous les bénéficiaires d'un coup pour gagner du temps
print("Calcul des montants bénéficiaires...")
colonnes_source = ['crerd_gen', 'crecoll_gen', 'creinno_gen', 'crecrc', 'cretot_gen']
colonnes_benef = ['cirrech_benef', 'cic_benef', 'cii_benef', 'crecrc_benef', 'cirtot_benef']

for col_source, col_benef in zip(colonnes_source, colonnes_benef):
    if col_source in df.columns:
        print(f"  Calcul de {col_benef}...")
        df[col_benef] = calculer_benef_optimise(df, col_source)
    else:
        print(f"  Colonne {col_source} non trouvée, {col_benef} initialisé à 0")
        df[col_benef] = 0

# 3. VÉRIFIER LA COHÉRENCE DES MONTANTS
print("Vérification de la cohérence des montants...")

# Vérifier que cir_benef_total correspond à cirtot_benef (si présent)
if 'cir_benef_total' in df.columns:
    ecart_total = (df['cirtot_benef'] - df['cir_benef_total']).abs().sum()
    print(f"Écart entre cirtot_benef et cir_benef_total: {ecart_total:.2f}")

# Vérifier que la somme des composantes bénéficiaires égale le total bénéficiaire (vectorisé)
df['somme_composantes_benef'] = df[['cirrech_benef', 'cic_benef', 'cii_benef', 'crecrc_benef']].fillna(0).sum(axis=1)
ecart_composantes = (df['somme_composantes_benef'] - df['cirtot_benef']).abs().sum()
print(f"Écart entre somme des composantes et total bénéficiaire: {ecart_composantes:.2f}")

# Comparer les totaux (vectorisé)
print("\nVérification des totaux générés vs bénéficiaires:")
print(f"{'Type de crédit':<15}{'Généré':<15}{'Bénéficiaire':<15}{'Écart':<15}{'Écart %':<10}")
print("-" * 65)

for col_gen, col_benef in zip(colonnes_source, colonnes_benef):
    if col_gen in df.columns and col_benef in df.columns:
        total_gen = df[col_gen].sum()
        total_benef = df[col_benef].sum()
        ecart = total_gen - total_benef
        ecart_pct = 0 if total_gen == 0 else ecart / total_gen * 100
        print(f"{col_gen:<15}{total_gen:<15.2f}{total_benef:<15.2f}{ecart:<15.2f}{ecart_pct:<10.2f}%")

# Ajouter les variables pour les DOM-TOM en une seule opération vectorisée
for col in ['crerd_genom', 'crecoll_genom', 'creinno_genom', 'cretot_genom']:
    df[col] = 0

# Supprimer les colonnes temporaires en une seule opération
df = df.drop(['SIREN_DEPOSANT_STD', 'SIREN_DECLARANT_STD', 'IS_MATCH', 'somme_composantes_benef'], axis=1)

# Sauvegarder le fichier
output_file = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_2021.csv"
print(f"Sauvegarde du fichier: {output_file}")
df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)

print("Traitement terminé avec succès!")

Nombre total de lignes: 4543
Colonnes renommées: 13
Calcul des montants bénéficiaires...
  Calcul de cirrech_benef...
  Calcul de cic_benef...
  Calcul de cii_benef...
  Colonne crecrc non trouvée, crecrc_benef initialisé à 0
  Calcul de cirtot_benef...
Vérification de la cohérence des montants...
Écart entre cirtot_benef et cir_benef_total: 0.00
Écart entre somme des composantes et total bénéficiaire: 344.23

Vérification des totaux générés vs bénéficiaires:
Type de crédit Généré         Bénéficiaire   Écart          Écart %   
-----------------------------------------------------------------
crerd_gen      414605137.83   414605137.83   -0.00          -0.00     %
crecoll_gen    1544332.70     1544332.70     0.00           0.00      %
creinno_gen    37589152.49    37589152.49    0.00           0.00      %
cretot_gen     453738653.93   453738653.93   0.00           0.00      %
Sauvegarde du fichier: M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_

In [64]:
import pandas as pd
import numpy as np

# Charger le fichier obtenu précédemment
file_path = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_2021.csv"
df = pd.read_csv(file_path, sep=';', encoding='utf-8-sig')

print(f"Nombre total de lignes: {len(df)}")
print(f"Colonnes disponibles: {len(df.columns)}")



print("\n=== CALCUL DES CRÉDITS DOM-TOM ===")

# Rechercher les colonnes DOM spécifiques
colonnes_dom_rd = [col for col in df.columns if 'DOM' in col and ('RECHERCHE' in col or 'DEPENSES' in col)]
colonnes_dom_coll = [col for col in df.columns if 'DOM' in col and 'COLLECTION' in col]
colonnes_dom_inno = [col for col in df.columns if 'DOM' in col and 'INNOVATION' in col]

print(f"Colonnes DOM R&D trouvées: {colonnes_dom_rd}")
print(f"Colonnes DOM Collection trouvées: {colonnes_dom_coll}")
print(f"Colonnes DOM Innovation trouvées: {colonnes_dom_inno}")

# Calculer les crédits DOM-TOM avec les taux spécifiques
# CIR Recherche DOM : 50%
if 'L26B_MONTANT_NET_DEPENSES_DOM_CALCULE' in df.columns:
    df['crerd_genom'] = df['L26B_MONTANT_NET_DEPENSES_DOM_CALCULE'].fillna(0) * 0.50
    print(f"CIR Recherche DOM calculé: {df['crerd_genom'].sum():,.2f} €")
else:
    df['crerd_genom'] = 0
    print("Pas de données DOM pour la recherche trouvées")

# CIR Collection DOM : 50%
col_coll_dom = next((col for col in df.columns if 'L33B' in col and 'DOM' in col), None)
if col_coll_dom:
    df['crecoll_genom'] = df[col_coll_dom].fillna(0) * 0.50
    print(f"CIR Collection DOM calculé: {df['crecoll_genom'].sum():,.2f} €")
else:
    df['crecoll_genom'] = 0
    print("Pas de données DOM pour la collection trouvées")

# CIR Innovation DOM : 60% + Corse
col_inno_dom = next((col for col in df.columns if 'L76B' in col and 'DOM' in col), None)
col_inno_corse_mpe = next((col for col in df.columns if 'L76C' in col and 'CORSE' in col), None)
col_inno_corse_me = next((col for col in df.columns if 'L76D' in col and 'CORSE' in col), None)

if col_inno_dom:
    cii_dom = df[col_inno_dom].fillna(0) * 0.60
else:
    cii_dom = 0

if col_inno_corse_mpe:
    cii_corse_mpe = df[col_inno_corse_mpe].fillna(0) * 0.40
else:
    cii_corse_mpe = 0

if col_inno_corse_me:
    cii_corse_me = df[col_inno_corse_me].fillna(0) * 0.35
else:
    cii_corse_me = 0

df['creinno_genom'] = cii_dom + cii_corse_mpe + cii_corse_me
print(f"CIR Innovation DOM+Corse calculé: {df['creinno_genom'].sum():,.2f} €")

# CIR Total DOM
df['cretot_genom'] = df['crerd_genom'] + df['crecoll_genom'] + df['creinno_genom']
print(f"CIR Total DOM calculé: {df['cretot_genom'].sum():,.2f} €")



print("\n=== CALCUL DES MONTANTS NETS BÉNÉFICIAIRES ===")

# Standardiser les SIREN pour assurer la cohérence
df['SIREN_DEPOSANT_STD'] = df['SIREN_DEPOSANT'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(9)
df['SIREN_DECLARANT_STD'] = df['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(9)
df['IS_MATCH'] = df['SIREN_DEPOSANT_STD'] == df['SIREN_DECLARANT_STD']

# Recherche automatique des colonnes de dépenses
def trouver_colonne_depenses(patterns, description):
    """Trouve la meilleure colonne correspondant aux patterns donnés"""
    for pattern in patterns:
        colonnes_trouvees = [col for col in df.columns if pattern in col]
        if colonnes_trouvees:
            print(f"Colonne {description} trouvée: {colonnes_trouvees[0]}")
            return colonnes_trouvees[0]
    print(f"Aucune colonne {description} trouvée")
    return None

# Rechercher les colonnes de dépenses
colonne_depenses_rd = trouver_colonne_depenses([
    'L26A_MONTANT_NET_DEPENSES_CALCULE',
    'L26A_MONTANT_NET_DEPENSES_DECLARE',
    'MONTANT_NET_DEPENSES'
], "dépenses R&D")

col_dep_coll = trouver_colonne_depenses([
    'L33A_MONTANT_NET_COLLECTION_CALCULE',
    'L33A_MONTANT_NET_COLLECTION_DECLARE',
    'MONTANT_NET_COLLECTION'
], "dépenses Collection")

col_dep_inno = trouver_colonne_depenses([
    'L76A_MONTANT_NET_INNOVATION_CALCULE',
    'L76A_MONTANT_NET_INNOVATION_DECLARE',
    'MONTANT_NET_INNOVATION'
], "dépenses Innovation")

# Fonction optimisée pour calculer les montants nets bénéficiaires
def calculer_benef_optimise(df, colonne_source, nom_calcul=""):
    """Calcule les montants bénéficiaires de manière optimisée"""
    if colonne_source is None or colonne_source not in df.columns:
        print(f"  {nom_calcul}: Colonne non trouvée, initialisation à 0")
        return pd.Series(0, index=df.index, dtype='float64')
    
    print(f"  {nom_calcul}: Calcul en cours...")
    
    # Créer un DataFrame temporaire avec les données essentielles
    df_temp = df[['SIREN_DEPOSANT_STD', 'IS_MATCH', colonne_source]].copy()
    df_temp[colonne_source] = df_temp[colonne_source].fillna(0)
    
    # Calculer la somme totale par déposant
    sommes = df_temp.groupby('SIREN_DEPOSANT_STD')[colonne_source].sum().to_dict()
    
    # Créer une série pour stocker les résultats
    resultats = pd.Series(0, index=df.index, dtype='float64')
    
    # Compteurs pour le rapport
    count_match = 0
    count_no_match = 0
    
    # Pour chaque SIREN déposant unique
    for siren, montant in sommes.items():
        if montant == 0:
            continue
            
        # Identifier les lignes correspondant à ce SIREN
        mask_siren = df['SIREN_DEPOSANT_STD'] == siren
        
        # Identifier les lignes où il y a correspondance (déclarant = déposant)
        mask_match_siren = mask_siren & df['IS_MATCH']
        nb_match = mask_match_siren.sum()
        
        if nb_match > 0:
            # Distribuer le montant également entre les lignes correspondantes
            resultats[mask_match_siren] = montant / nb_match
            count_match += nb_match
        else:
            # Attribuer à la première ligne du SIREN s'il n'y a pas de correspondance
            if mask_siren.any():
                idx = df[mask_siren].index[0]
                resultats[idx] = montant
                count_no_match += 1
    
    total_benef = resultats.sum()
    total_gen = df_temp[colonne_source].sum()
    print(f"    Total généré: {total_gen:,.2f} € | Total bénéficiaire: {total_benef:,.2f} € | Écart: {total_gen-total_benef:,.2f} €")
    print(f"    Répartition: {count_match} avec correspondance, {count_no_match} sans correspondance")
    
    return resultats

# Calculer les montants nets des dépenses bénéficiaires
print("Calcul des montants nets des dépenses bénéficiaires:")
df['deprd_benef'] = calculer_benef_optimise(df, colonne_depenses_rd, "Dépenses R&D")
df['depcoll_benef'] = calculer_benef_optimise(df, col_dep_coll, "Dépenses Collection")
df['depinno_benef'] = calculer_benef_optimise(df, col_dep_inno, "Dépenses Innovation")

# Calculer le montant total des dépenses bénéficiaires
df['deptot_benef'] = df['deprd_benef'] + df['depcoll_benef'] + df['depinno_benef']
print(f"Montant total des dépenses bénéficiaires: {df['deptot_benef'].sum():,.2f} €")


print("\n=== CRÉATION DES INDICATRICES ===")

# i_dep_rd : "Oui" si dépenses R&D > 0
if colonne_depenses_rd and colonne_depenses_rd in df.columns:
    df['i_dep_rd'] = np.where(df[colonne_depenses_rd] > 0, "Oui", "Non")
else:
    df['i_dep_rd'] = "Non"

# i_dep_inno : "Oui" si dépenses inno > 0
if col_dep_inno and col_dep_inno in df.columns:
    df['i_dep_inno'] = np.where(df[col_dep_inno] > 0, "Oui", "Non")
else:
    df['i_dep_inno'] = "Non"

# i_dep_coll : "Oui" si dépenses collection > 0
if col_dep_coll and col_dep_coll in df.columns:
    df['i_dep_coll'] = np.where(df[col_dep_coll] > 0, "Oui", "Non")
else:
    df['i_dep_coll'] = "Non"

# i_bef : "Oui" si bénéficiaire du CIR total
df['i_bef'] = np.where(df['cirtot_benef'] > 0, "Oui", "Non")

# i_bef_rech : "Oui" si bénéficiaire du CIR recherche
df['i_bef_rech'] = np.where(df['cirrech_benef'] > 0, "Oui", "Non")

# i_bef_inno : "Oui" si bénéficiaire du CII
df['i_bef_inno'] = np.where(df['cii_benef'] > 0, "Oui", "Non")

# i_bef_crc : "Oui" si bénéficiaire du CRC
df['i_bef_crc'] = np.where(df['crecrc_benef'] > 0, "Oui", "Non")

# i_bef_coll : "Oui" si bénéficiaire du CI collection
df['i_bef_coll'] = np.where(df['cic_benef'] > 0, "Oui", "Non")

print("Indicatrices créées avec succès")



print("\n" + "="*60)
print("RAPPORT DÉTAILLÉ DE VÉRIFICATION DES DONNÉES")
print("="*60)

# Vérifier les totaux des montants nets de dépenses
print("\n1. MONTANTS NETS DES DÉPENSES BÉNÉFICIAIRES:")
print(f"   • Dépenses R&D bénéficiaires: {df['deprd_benef'].sum():,.2f} €")
print(f"   • Dépenses Collection bénéficiaires: {df['depcoll_benef'].sum():,.2f} €")
print(f"   • Dépenses Innovation bénéficiaires: {df['depinno_benef'].sum():,.2f} €")
print(f"   • TOTAL dépenses bénéficiaires: {df['deptot_benef'].sum():,.2f} €")

# Vérifier les totaux des crédits DOM-TOM
print("\n2. CRÉDITS D'IMPÔT DOM-TOM:")
print(f"   • CIR Recherche DOM: {df['crerd_genom'].sum():,.2f} €")
print(f"   • CIR Collection DOM: {df['crecoll_genom'].sum():,.2f} €")
print(f"   • CIR Innovation DOM: {df['creinno_genom'].sum():,.2f} €")
print(f"   • TOTAL CIR DOM: {df['cretot_genom'].sum():,.2f} €")

# Vérifier les comptes des indicatrices
print("\n3. RÉPARTITION DES INDICATRICES:")
indicatrices = ['i_dep_rd', 'i_dep_inno', 'i_dep_coll', 'i_bef', 'i_bef_rech', 'i_bef_inno', 'i_bef_crc', 'i_bef_coll']

for indicatrice in indicatrices:
    if indicatrice in df.columns:
        count_oui = (df[indicatrice] == "Oui").sum()
        count_non = (df[indicatrice] == "Non").sum()
        pct_oui = count_oui/len(df)*100
        pct_non = count_non/len(df)*100
        print(f"   • {indicatrice:<12}: {count_oui:>6,} Oui ({pct_oui:>5.1f}%) | {count_non:>6,} Non ({pct_non:>5.1f}%)")

# Vérification de cohérence des bénéficiaires
print("\n4. VÉRIFICATIONS DE COHÉRENCE:")

# Cohérence entre i_bef et les autres indicatrices bénéficiaires
nb_benef_total = (df['i_bef'] == "Oui").sum()
nb_benef_rech = (df['i_bef_rech'] == "Oui").sum()
nb_benef_coll = (df['i_bef_coll'] == "Oui").sum()
nb_benef_inno = (df['i_bef_inno'] == "Oui").sum()
nb_benef_crc = (df['i_bef_crc'] == "Oui").sum()

print(f"   • Bénéficiaires totaux: {nb_benef_total:,}")
print(f"   • Détail: Recherche {nb_benef_rech:,} | Collection {nb_benef_coll:,} | Innovation {nb_benef_inno:,} | CRC {nb_benef_crc:,}")

# Vérifier la cohérence entre dépenses et bénéfices
coherence_rd = ((df['i_dep_rd'] == "Oui") & (df['i_bef_rech'] == "Non")).sum()
coherence_coll = ((df['i_dep_coll'] == "Oui") & (df['i_bef_coll'] == "Non")).sum()
coherence_inno = ((df['i_dep_inno'] == "Oui") & (df['i_bef_inno'] == "Non")).sum()

print(f"   • Entreprises avec dépenses R&D mais sans bénéfice recherche: {coherence_rd:,}")
print(f"   • Entreprises avec dépenses Collection mais sans bénéfice collection: {coherence_coll:,}")
print(f"   • Entreprises avec dépenses Innovation mais sans bénéfice innovation: {coherence_inno:,}")

# ---------------------------------------------------
# 5. FINALISATION ET SAUVEGARDE
# ---------------------------------------------------

# Supprimer les colonnes temporaires
colonnes_temp = ['SIREN_DEPOSANT_STD', 'SIREN_DECLARANT_STD', 'IS_MATCH']
for col in colonnes_temp:
    if col in df.columns:
        df = df.drop(col, axis=1)

# Ajouter une colonne de contrôle avec la date de traitement
from datetime import datetime
df['date_traitement'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Sauvegarder le fichier avec les nouvelles colonnes
output_file = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_2021_complet.csv"
df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)

print(f"\n{'='*60}")
print(f"TRAITEMENT TERMINÉ AVEC SUCCÈS")
print(f"{'='*60}")
print(f"Fichier sauvegardé: {output_file}")
print(f"Nombre total de lignes: {len(df):,}")
print(f"Nombre total de colonnes: {len(df.columns):,}")
print(f"Taille du fichier final: {len(df) * len(df.columns):,} cellules")

# Résumé des nouvelles colonnes créées
nouvelles_colonnes = [
    'crerd_genom', 'crecoll_genom', 'creinno_genom', 'cretot_genom',
    'deprd_benef', 'depcoll_benef', 'depinno_benef', 'deptot_benef',
    'i_dep_rd', 'i_dep_inno', 'i_dep_coll', 'i_bef', 'i_bef_rech', 
    'i_bef_inno', 'i_bef_crc', 'i_bef_coll', 'date_traitement'
]

print(f"\nNOUVELLES COLONNES CRÉÉES ({len(nouvelles_colonnes)}):")
for i, col in enumerate(nouvelles_colonnes, 1):
    print(f"  {i:2d}. {col}")

print(f"\nFichier prêt ")

Nombre total de lignes: 4543
Colonnes disponibles: 181

=== CALCUL DES CRÉDITS DOM-TOM ===
Colonnes DOM R&D trouvées: ['L31B_MONTANT_NET_DEPENSES_DOM_DECLARE', 'L31B_MONTANT_NET_DEPENSES_DOM_CALCULE', 'L43B_CREDIT_IMPOT_RECHERCHE_DOM', 'L51B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM', 'L52B_DEPENSES_RECHERCHE_DOM_LIMITE', 'L58B_CREDIT_IMPOT_RECHERCHE_DOM_PLUS_100M', 'L69B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM_PLUS_100M']
Colonnes DOM Collection trouvées: ['L38B_MONTANT_NET_COLLECTION_DOM_DECLARE', 'L38B_MONTANT_NET_COLLECTION_DOM_CALCULE', 'L51B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM', 'L59B_MONTANT_NET_COLLECTION_DOM_PLUS_100M', 'L69B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM_PLUS_100M']
Colonnes DOM Innovation trouvées: ['L82B_MONTANT_NET_INNOVATION_DOM', 'L85B_CREDIT_IMPOT_INNOVATION_DOM']
Pas de données DOM pour la recherche trouvées
Pas de données DOM pour la collection trouvées
CIR Innovation DOM+Corse calculé: 0.00 €
CIR Total DOM calculé: 0.00 €

=== CALCUL DES MONTA

In [ ]:
# -*- coding: utf-8 -*-
"""
Ce script :
1. Charge et prépare un fichier de référence des unités légales (StockUniteLegale).
2. Charge une base de données principale à enrichir ("base_a_sireniser").
3. Joint ces deux bases sur le numéro SIREN.
4. Charge une nomenclature d'activités (Nomenclature Sittar).
5. Joint le résultat précédent avec cette nomenclature sur le code APE.

librairie pandas installée requis: pip install pandas openpyxl
(openpyxl est nécessaire pour lire les fichiers Excel .xlsx)
"""

import pandas as pd # Librairie pour la manipulation de données
import re           # Librairie pour les expressions régulières (nettoyage des noms de colonnes)
# La librairie 'os' n'est plus nécessaire car on ne crée plus de dossier/fichiers de test.

# --- Fonction Auxiliaire pour Nettoyer les Noms de Colonnes ---
def nettoyer_noms_colonnes(df_a_nettoyer):
    """
    Nettoie les noms de colonnes d'un DataFrame :
    1. Convertit en minuscules.
    2. Remplace les espaces et caractères spéciaux par des tirets bas ('_').
    3. Supprime les tirets bas en début ou fin de nom.
    Retourne un nouveau DataFrame avec les noms de colonnes nettoyés.
    """
    df = df_a_nettoyer.copy() # Travailler sur une copie
    nouveaux_noms = {}
    for col in df.columns:
        nouveau_nom = str(col).lower()
        nouveau_nom = re.sub(r'\s+', '_', nouveau_nom)
        nouveau_nom = re.sub(r'[^a-z0-9_]', '', nouveau_nom)
        nouveau_nom = re.sub(r'_+', '_', nouveau_nom)
        nouveau_nom = nouveau_nom.strip('_')
        nouveaux_noms[col] = nouveau_nom
    df.rename(columns=nouveaux_noms, inplace=True)
    return df

# --- 1. Chargement et Préparation des Données 'StockUniteLegale' ---
def charger_et_preparer_stock_ul(chemin_fichier_stock_ul):
    """
    Charge et prépare les données du fichier 'StockUniteLegale'.
    """
    print(f"Chargement et préparation de '{chemin_fichier_stock_ul}'...")
    colonnes_a_garder = [
        'siren', 'nicSiegeUniteLegale', 'denominationUniteLegale',
        'denominationUsuelle1UniteLegale', 'denominationUsuelle2UniteLegale',
        'denominationUsuelle3UniteLegale', 'dateCreationUniteLegale',
        'activitePrincipaleUniteLegale', 'categorieEntreprise',
        'trancheEffectifsUniteLegale'
    ]
    try:
        stock_ul_df = pd.read_csv(chemin_fichier_stock_ul, 
                                  usecols=colonnes_a_garder, 
                                  dtype=str, 
                                  encoding='utf-8')
    except FileNotFoundError:
        print(f"ERREUR : Le fichier '{chemin_fichier_stock_ul}' est introuvable.")
        raise
    except ValueError as e:
        print(f"ERREUR : Vérifiez que les colonnes {colonnes_a_garder} existent bien dans '{chemin_fichier_stock_ul}'. Détail : {e}")
        raise

    stock_ul_df['nic'] = stock_ul_df['nicSiegeUniteLegale'].str.zfill(5)
    stock_ul_df['siren'] = stock_ul_df['siren'].str.zfill(9)
    stock_ul_df['siret'] = stock_ul_df['siren'] + stock_ul_df['nic']
    stock_ul_df.rename(columns={'siren': 'SIREN_DECLARANT'}, inplace=True)
    print(f"Colonnes de stock_ul_extract après préparation : {stock_ul_df.columns.tolist()}")
    return stock_ul_df

# --- Fonction pour Charger la 'base_a_sireniser' ---
def charger_base_a_sireniser(chemin_fichier_base, nom_colonne_siren='SIREN_DECLARANT'):
    """
    Charge les données de la 'base_a_sireniser'.
    'nom_colonne_siren' doit être le nom de la colonne SIREN dans ce fichier.
    """
    print(f"Chargement de la base à siréniser depuis '{chemin_fichier_base}'...")
    try:
        base_df = pd.read_csv(chemin_fichier_base,
                      dtype={nom_colonne_siren: str}, 
                      encoding='utf-8',
                      sep=';') # Ajoutez sep=';' ou le séparateur correct
    except FileNotFoundError:
        print(f"ERREUR : Le fichier '{chemin_fichier_base}' est introuvable.")
        raise
    except KeyError:
        print(f"ERREUR : La colonne '{nom_colonne_siren}' (spécifiée pour le SIREN) est introuvable dans '{chemin_fichier_base}'.")
        raise
        
    if nom_colonne_siren in base_df.columns:
        base_df[nom_colonne_siren] = base_df[nom_colonne_siren].str.zfill(9)
    print(f"Colonnes de base_a_sireniser : {base_df.columns.tolist()}")
    return base_df

# --- 3. Chargement et Préparation des Données de la 'Nomenclature Sittar' ---
def charger_et_preparer_nomenclature(chemin_fichier_nomenclature, nom_original_col_ape='code_ape_entreprise'):
    """
    Charge et prépare les données du fichier 'Nomenclature_secteur_sittar.xlsx'.
    'nom_original_col_ape' est le nom original de la colonne du code APE dans le fichier Excel.
    """
    print(f"Chargement et préparation de la nomenclature '{chemin_fichier_nomenclature}'...")
    try:
        nomenclature_df = pd.read_excel(chemin_fichier_nomenclature)
    except FileNotFoundError:
        print(f"ERREUR : Le fichier '{chemin_fichier_nomenclature}' est introuvable.")
        raise

    nomenclature_df = nettoyer_noms_colonnes(nomenclature_df)
    col_ape_nettoyee_candidate = nettoyer_noms_colonnes(pd.DataFrame(columns=[nom_original_col_ape])).columns[0]
    nom_cible_col_ape_pour_jointure = 'activiteprincipaleunitelegale'

    if col_ape_nettoyee_candidate in nomenclature_df.columns:
        nomenclature_df.rename(columns={col_ape_nettoyee_candidate: nom_cible_col_ape_pour_jointure}, inplace=True)
    elif nom_original_col_ape.lower() in nomenclature_df.columns and nom_original_col_ape.lower() != nom_cible_col_ape_pour_jointure :
         nomenclature_df.rename(columns={nom_original_col_ape.lower(): nom_cible_col_ape_pour_jointure}, inplace=True)
    elif nom_cible_col_ape_pour_jointure not in nomenclature_df.columns:
        print(f"Attention : Colonne APE ('{col_ape_nettoyee_candidate}' ou '{nom_original_col_ape.lower()}') non trouvée "
              f"pour renommage en '{nom_cible_col_ape_pour_jointure}'. Colonnes dispo : {nomenclature_df.columns.tolist()}")

    if nom_cible_col_ape_pour_jointure in nomenclature_df.columns:
        cols = [nom_cible_col_ape_pour_jointure] + [col for col in nomenclature_df.columns if col != nom_cible_col_ape_pour_jointure]
        nomenclature_df = nomenclature_df[cols]
    print(f"Colonnes de nomenclature_sitar après préparation : {nomenclature_df.columns.tolist()}")
    return nomenclature_df

# --- Processus Principal de Sirénisation ---
def processus_principal_sirenisation():
    """
    Fonction principale orchestrant le processus de sirénisation.
    """
    # --- Configuration : Définissez vos chemins de fichiers ici ---
    chemin_stock_ul = "C://Users//msamb//Documents//StockUniteLegale_utf8.csv"
    chemin_base_a_sireniser = "M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_2021_complet.csv"
    nom_col_siren_dans_base = "SIREN_DECLARANT" 
    chemin_nomenclature_sittar = "C://Users//msamb//Documents//Nomenclature_secteur_sittar.xlsx"
    nom_original_col_ape_nomenclature = "Code APE Entreprise"
    # --- Fin de la Configuration ---

    try:
        # 1. Charger et préparer 'stock_ul_extract'
        stock_ul_extrait_df = charger_et_preparer_stock_ul(chemin_stock_ul)

        # 2. Charger 'base_a_sireniser'
        base_a_sireniser_df = charger_base_a_sireniser(chemin_base_a_sireniser, nom_colonne_siren=nom_col_siren_dans_base)

        # 3. Première jointure (sirénisation)
        print(f"\nJonction de 'base_a_sireniser' avec 'stock_ul_extrait' sur la colonne '{nom_col_siren_dans_base}'...")
        base_finale_def_sirenise = pd.merge(
            left=base_a_sireniser_df,
            right=stock_ul_extrait_df,
            left_on=nom_col_siren_dans_base,
            right_on='SIREN_DECLARANT',
            how='left'
        )
        print(f"Nombre de lignes après la première jonction : {len(base_finale_def_sirenise)}")

        # 4. Charger et préparer 'nomenclature_sitar'
        nomenclature_sitar_df = charger_et_preparer_nomenclature(
            chemin_nomenclature_sittar, 
            nom_original_col_ape=nom_original_col_ape_nomenclature
        )

        # 5. Seconde jointure avec la nomenclature
        cle_jointure_ape_base = 'activitePrincipaleUniteLegale'
        cle_jointure_ape_nomenclature = 'activiteprincipaleunitelegale'

        if cle_jointure_ape_base not in base_finale_def_sirenise.columns:
            print(f"ERREUR : Colonne '{cle_jointure_ape_base}' absente de la base après première jointure.")
            return
        if cle_jointure_ape_nomenclature not in nomenclature_sitar_df.columns:
            print(f"ERREUR : Colonne '{cle_jointure_ape_nomenclature}' absente de la nomenclature préparée.")
            return

        # Harmonisation des clés de jointure APE pour robustesse
        base_finale_def_sirenise[cle_jointure_ape_base + '_pour_jointure'] = base_finale_def_sirenise[cle_jointure_ape_base].astype(str).str.strip().str.lower()
        nomenclature_sitar_df[cle_jointure_ape_nomenclature + '_pour_jointure'] = nomenclature_sitar_df[cle_jointure_ape_nomenclature].astype(str).str.strip().str.lower()
        
        print(f"\nJonction avec 'nomenclature_sitar' sur la colonne APE (harmonisées)...")
        base_finale_df = pd.merge(
            base_finale_def_sirenise,
            nomenclature_sitar_df,
            left_on=cle_jointure_ape_base + '_pour_jointure',
            right_on=cle_jointure_ape_nomenclature + '_pour_jointure',
            how='left',
            suffixes=('_base', '_nomenclature')
        )
        
        # Optionnel: supprimer les colonnes temporaires de jointure APE
        base_finale_df.drop(columns=[cle_jointure_ape_base + '_pour_jointure', cle_jointure_ape_nomenclature + '_pour_jointure'], inplace=True, errors='ignore')
        
        print(f"Nombre de lignes après la seconde jonction : {len(base_finale_df)}")
        print(f"Colonnes finales : {base_finale_df.columns.tolist()}")

        print("\n✓ Processus de sirénisation terminé.")
        print("\nAperçu du DataFrame final (base_finale_df) :")
        print(base_finale_df.head())
        base_finale_df.to_csv("C://Users//msamb//Documents//base_finale_sirenise_2021.csv", sep=';', encoding='utf-8-sig', index=False)


    except FileNotFoundError:
        print("ERREUR CRITIQUE : Un ou plusieurs fichiers d'entrée sont introuvables. Vérifiez les chemins dans la 'Configuration'.")
    except KeyError as e:
        print(f"ERREUR DE COLONNE (KeyError) : {e}. Vérifiez les noms de colonnes dans vos fichiers et la configuration.")
    except Exception as e:
        print(f"Une erreur inattendue est survenue : {e}")
        import traceback
        traceback.print_exc()

# --- Point d'Entrée du Script ---
if __name__ == "__main__":
    # Ce code est exécuté lorsque vous lancez le script directement.
    
    # Exécuter le processus principal de sirénisation.
    # Assurez-vous d'avoir bien configuré les chemins et noms de colonnes
    # dans la section 'Configuration' de la fonction 'processus_principal_sirenisation'.
    processus_principal_sirenisation()

Chargement et préparation de 'C://Users//msamb//Documents//StockUniteLegale_utf8.csv'...
Colonnes de stock_ul_extract après préparation : ['SIREN_DECLARANT', 'dateCreationUniteLegale', 'trancheEffectifsUniteLegale', 'categorieEntreprise', 'denominationUniteLegale', 'denominationUsuelle1UniteLegale', 'denominationUsuelle2UniteLegale', 'denominationUsuelle3UniteLegale', 'activitePrincipaleUniteLegale', 'nicSiegeUniteLegale', 'nic', 'siret']
Chargement de la base à siréniser depuis 'M://str-dgri-gecir-donnees-fiscales//x-pour MF-SAMB//scripts//Calcul_Creance_CIR_format_2021_complet.csv'...
Colonnes de base_a_sireniser : ['SIREN_DECLARANT', 'SIREN_DEPOSANT', 'DESIGNATION', 'L1_DOTATION_AMORT_IMMO', 'L2_DOTATION_AMORT_SINISTR', 'L3_DEPENSES_PERSONNEL_CHERCHEURS', 'L4_REMUNERATION_INVENTEURS', 'L5_DEPENSES_JEUNES_DOCTEURS', 'L6_AUTRES_DEP_FONCT_DECLARE', 'L6_AUTRES_DEP_FONCT_CALCULE', 'L6_ECART', 'L7_TOTAL_DEP_FONCT_DECLARE', 'L7_TOTAL_DEP_FONCT_CALCULE', 'L7_ECART', 'L8_FRAIS_BREVETS_COV', 

In [4]:
import pandas as pd

# Charger les deux fichiers
df_base_final = pd.read_csv("C://Users//msamb//Documents//base_finale_sirenise_2021.csv", sep=';', encoding='utf-8-sig')
df_cir2021prov = pd.read_excel("M://str-dgri-gecir-donnees-fiscales//z_Statistiques Séries Demandes//2021//0_BASE_CIR2021prov_extr202306.xlsx")

# Standardiser les SIREN dans les deux fichiers (en string, 9 chiffres)
df_base_final['SIREN_DECLARANT_STD'] = df_base_final['SIREN_DECLARANT'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(9)
df_cir2021prov['SIREN_DECLARANT_STD'] = df_cir2021prov['siren'].astype(str).str.replace(r'\D', '', regex=True).str.zfill(9)

# Vérifier la présence des SIREN de base_final dans cir2021prov
siren_cir2021prov = set(df_cir2021prov['SIREN_DECLARANT_STD'])
df_base_final['present_dans_cir2021prov'] = df_base_final['SIREN_DECLARANT_STD'].isin(siren_cir2021prov)

# Faire un left join pour récupérer les valeurs de cretot_gen du fichier cir2021prov
df_merged = df_base_final.merge(
    df_cir2021prov[['SIREN_DECLARANT_STD', 'cretot_gen']], 
    on='SIREN_DECLARANT_STD', 
    how='left', 
    suffixes=('', '_cir2021prov')
)

# Calculer la différence cretot_gen (base 2021) - cretot_gen (cir2021prov)
# Seulement pour les SIREN présents dans les deux fichiers
df_merged['difference_cretot_gen'] = None  # Initialiser avec None
mask_present = df_merged['present_dans_cir2021prov'] == True

# Calculer la différence seulement pour les lignes où les deux valeurs existent
df_merged.loc[mask_present, 'difference_cretot_gen'] = (
    df_merged.loc[mask_present, 'cretot_gen'] - 
    df_merged.loc[mask_present, 'cretot_gen_cir2021prov']
)

# Statistiques
nb_total = len(df_merged)
nb_present = df_merged['present_dans_cir2021prov'].sum()
nb_absent = nb_total - nb_present

print(f"Nombre total de SIREN dans base_finale_sirenise_2021.csv : {nb_total}")
print(f"Nombre de SIREN présents dans cir2021prov : {nb_present}")
print(f"Nombre de SIREN absents dans cir2021prov : {nb_absent}")

# Statistiques sur les différences
if nb_present > 0:
    differences = df_merged.loc[mask_present, 'difference_cretot_gen'].dropna()
    if len(differences) > 0:
        print(f"\nStatistiques des différences cretot_gen (base_2021 - cir2021prov) :")
        print(f"Nombre de différences calculées : {len(differences)}")
        print(f"Différence moyenne : {differences.mean():.2f}")
        print(f"Différence médiane : {differences.median():.2f}")
        print(f"Différence min : {differences.min():.2f}")
        print(f"Différence max : {differences.max():.2f}")
        print(f"Nombre de différences nulles (identiques) : {(differences == 0).sum()}")
        print(f"Nombre de différences positives : {(differences > 0).sum()}")
        print(f"Nombre de différences négatives : {(differences < 0).sum()}")

# Afficher quelques exemples absents
if nb_absent > 0:
    print("\nExemples de SIREN absents dans cir2021prov :")
    print(df_merged.loc[~df_merged['present_dans_cir2021prov'], 'SIREN_DECLARANT_STD'].head())

# Afficher quelques exemples de différences
if nb_present > 0:
    print("\nExemples de différences calculées :")
    exemples = df_merged.loc[mask_present, ['SIREN_DECLARANT_STD', 'cretot_gen', 'cretot_gen_cir2021prov', 'difference_cretot_gen']].head()
    print(exemples)

# Sauvegarder le résultat avec la nouvelle colonne
df_merged.to_csv("C://Users//msamb//Documents//base_finale_sirenise_2021_verif_cir2021prov_avec_diff.csv", sep=';', encoding='utf-8-sig', index=False)

print(f"\nFichier sauvegardé avec la colonne 'difference_cretot_gen'")

Nombre total de SIREN dans base_finale_sirenise_2021.csv : 4543
Nombre de SIREN présents dans cir2021prov : 485
Nombre de SIREN absents dans cir2021prov : 4058

Statistiques des différences cretot_gen (base_2021 - cir2021prov) :
Nombre de différences calculées : 485
Différence moyenne : -123942.93
Différence médiane : 11622.66
Différence min : -10216618.00
Différence max : 9607203.76
Nombre de différences nulles (identiques) : 28
Nombre de différences positives : 273
Nombre de différences négatives : 184

Exemples de SIREN absents dans cir2021prov :
0    006720049
2    340012392
3    343266854
4    409108115
5    383960135
Name: SIREN_DECLARANT_STD, dtype: object

Exemples de différences calculées :
   SIREN_DECLARANT_STD  cretot_gen  cretot_gen_cir2021prov  \
1            356000000   585721.16               1107030.0   
7            421100645   183715.43                129644.0   
9            424335693   224474.48                250784.0   
44           320217144  1223501.90         

## Transformation

In [17]:
import pandas as pd
import numpy as np
import os

def transformer_vers_format_cible(fichier_principal, fichier_complement=None, fichier_sortie="fichier_format_cible.csv"):
    """
    Transforme le fichier CIR vers le format cible exact
    """
    
    print("TRANSFORMATION VERS FORMAT CIBLE")
    print("=" * 50)
    
    # 1. Ordre exact des colonnes demande
    COLONNES_CIBLE = [
        'siren', 'nom', 'siren_g', 'nom_g', 'type', 'caht', 'trannee', 'treff_benef', 
        'effectif', 'ch_tech', 'nb_jd', 'categorie', 'cat_jur', 'ape', 'sa_decl', 
        'ind_serv', 'cp', 'commune', 'dpt', 'old_region', 'region', 'pays_siege', 
        'deprd_decl', 'deprd_declom', 'depcoll_decl', 'depcoll_declom', 'depinno_decl', 
        'depinno_declom', 'deptot_decl', 'deptot_declom', 'deprd_benef', 'depcoll_benef', 
        'depinno_benef', 'deptot_benef', 'crerd_gen', 'crerd_genom', 'crecoll_gen', 
        'crecoll_genom', 'creinno_gen', 'creinno_genom', 'cretot_gen', 'cretot_genom', 
        'cirrech_benef', 'cic_benef', 'cii_benef', 'cirtot_benef', '_1', '_2', '_3', 
        '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15a', 
        '_15b', '_16a', '_16b', '_17', '_18a', '_18b', '_19a', '_19b', '_20', '_21', 
        '_22', '_23', '_24', '_25', '_26', '_27', '_28a', '_28b', '_29', '_30', '_31a', 
        '_31b', '_32', '_33', '_34', '_35', '_36', '_37', '_38a', '_38b', '_39a', 
        '_39b', '_40a', '_40b', '_41', '_42', '_43a', '_43b', '_44a', '_44b', '_45', 
        '_46', '_47a', '_47b', '_48', '_49', '_50a', '_50b', '_51a', '_51b', '_52a', 
        '_52b', '_53', '_54', '_55', '_56', '_57', '_58a', '_58b', '_59a', '_59b', 
        '_60', '_61', '_62', '_63', '_64', '_65', '_66', '_67', '_68a', '_68b', 
        '_69a', '_69b', '_70', '_71', '_72', '_73', '_74', '_75', '_76', '_77', 
        '_78', '_79', '_80', '_81', '_82a', '_82b', '_82c', '_82d', '_83', '_84', 
        '_85a', '_85b', '_85c', '_86a', '_86b'
    ]
    
    # 2. Chargement du fichier principal
    print(f"Chargement : {fichier_principal}")
    df = pd.read_csv(fichier_principal, sep=';', encoding='utf-8-sig')
    print(f"   {len(df):,} lignes chargees")
    
    # 3. Jointure avec fichier complement (si fourni)
    if fichier_complement:
        print(f"Chargement complement : {fichier_complement}")
        try:
            df_comp = pd.read_excel(fichier_complement)
            # Standardiser les SIREN pour la jointure
            df_comp['SIREN_STD'] = df_comp['siren_declarant'].astype(str).str.zfill(9)
            df['SIREN_STD'] = df['SIREN_DECLARANT'].astype(str).str.zfill(9)
            
            # Selectionner les colonnes utiles du complement
            colonnes_utiles = ['SIREN_STD', 'CAHT', 'NB_JD', 'NBR_CHERCH_TECH', 'NATUR_ESE']
            colonnes_disponibles = [col for col in colonnes_utiles if col in df_comp.columns]
            
            df = df.merge(df_comp[colonnes_disponibles], on='SIREN_STD', how='left', suffixes=('', '_comp'))
            print(f"   Jointure effectuee - {len(colonnes_disponibles)-1} colonnes recuperees")
        except Exception as e:
            print(f"   Erreur jointure: {e}, continuation sans complement")
    
    # 4. Fonctions utilitaires
    def get_col_calcule(base_name):
        """Retourne la colonne CALCULE si elle existe, sinon DECLARE, sinon NaN"""
        calcule = f"{base_name}_CALCULE"
        declare = f"{base_name}_DECLARE"
        if calcule in df.columns:
            return df[calcule]
        elif declare in df.columns:
            return df[declare]
        else:
            return np.nan
    
    def get_col_safe(col_name):
        """Retourne la colonne si elle existe, sinon NaN"""
        return df[col_name] if col_name in df.columns else np.nan
    
    # 5. Mapping des colonnes
    print("Mapping des colonnes...")
    
    # Mapping pour toutes les colonnes
    mapping_complet = {
        # Colonnes principales
        'siren': get_col_safe('SIREN_DECLARANT'),
        'nom': get_col_safe('DESIGNATION'),
        'siren_g': get_col_safe('SIREN_DEPOSANT'),
        'nom_g': get_col_safe('denominationUniteLegale'),
        'type': get_col_safe('type_final'),
        'caht': get_col_safe('CAHT'),  # Depuis le fichier complement
        'trannee': get_col_safe('dateCreationUniteLegale'),
        'treff_benef': get_col_safe('treff_benef'),
        'effectif': get_col_safe('trancheEffectifsUniteLegale'),
        'ch_tech': get_col_safe('NBR_CHERCH_TECH'),  # Depuis le fichier complement
        'nb_jd': get_col_safe('NB_JD'),  # Depuis le fichier complement
        'categorie': get_col_safe('categorieEntreprise'),
        'cat_jur': get_col_safe('NATUR_ESE'),  # Depuis le fichier complement
        'ape': get_col_safe('activitePrincipaleUniteLegale'),
        'sa_decl': get_col_safe('secteur_dactivit'),
        'ind_serv': np.nan,
        'cp': np.nan,
        'commune': np.nan,
        'dpt': np.nan,
        'old_region': np.nan,
        'region': np.nan,
        'pays_siege': 'FR',
        'deprd_decl': np.nan,
        'deprd_declom': np.nan,
        'depcoll_decl': np.nan,
        'depcoll_declom': np.nan,
        'depinno_decl': np.nan,
        'depinno_declom': np.nan,
        'deptot_decl': np.nan,
        'deptot_declom': np.nan,
        'deprd_benef': get_col_safe('deprd_benef'),
        'depcoll_benef': get_col_safe('depcoll_benef'),
        'depinno_benef': get_col_safe('depinno_benef'),
        'deptot_benef': get_col_safe('deptot_benef'),
        'crerd_gen': get_col_safe('crerd_gen'),
        'crerd_genom': get_col_safe('crerd_genom'),
        'crecoll_gen': get_col_safe('crecoll_gen'),
        'crecoll_genom': get_col_safe('crecoll_genom'),
        'creinno_gen': get_col_safe('creinno_gen'),
        'creinno_genom': get_col_safe('creinno_genom'),
        'cretot_gen': get_col_safe('cretot_gen'),
        'cretot_genom': get_col_safe('cretot_genom'),
        'cirrech_benef': get_col_safe('cirrech_benef'),
        'cic_benef': get_col_safe('cic_benef'),
        'cii_benef': get_col_safe('cii_benef'),
        'cirtot_benef': get_col_safe('cirtot_benef'),
        
        # Lignes formulaire - Priorite aux colonnes CALCULE
        '_1': get_col_safe('L1_DOTATION_AMORT_IMMO'),
        '_2': get_col_safe('L2_DOTATION_AMORT_SINISTR'),
        '_3': get_col_safe('L3_DEPENSES_PERSONNEL_CHERCHEURS'),
        '_4': get_col_safe('L4_REMUNERATION_INVENTEURS'),
        '_5': get_col_safe('L5_DEPENSES_JEUNES_DOCTEURS'),
        '_6': get_col_calcule('L6_AUTRES_DEP_FONCT'),
        '_7': get_col_calcule('L7_TOTAL_DEP_FONCT'),
        '_8': get_col_safe('L8_FRAIS_BREVETS_COV'),
        '_9': get_col_safe('L9_DEPENSES_DEFENSE_BREVETS'),
        '_10': get_col_safe('L10_DOTATION_AMORT_BREVETS'),
        '_11': get_col_safe('L11_DEPENSES_NORMALISATION'),
        '_12': get_col_safe('L12_PRIMES_COTISATIONS_BRUT'),
        '_13': get_col_safe('L13_VEILLE_TECHNO_BRUT'),
        '_14': get_col_calcule('L14_TOTAL_DEPENSES_INTERNES'),
        '_15a': get_col_safe('L15A_ST_PUB_LIES_FR'),
        '_15b': get_col_safe('L15B_ST_PUB_LIES_ETR'),
        '_16a': get_col_safe('L16A_ST_PUB_NON_LIES_FR'),
        '_16b': get_col_safe('L16B_ST_PUB_NON_LIES_ETR'),
        '_17': get_col_calcule('L17_TOTAL_ST_PUBLIQUE'),
        '_18a': get_col_safe('L18A_ST_PRIV_LIES_FR'),
        '_18b': get_col_safe('L18B_ST_PRIV_LIES_ETR'),
        '_19a': get_col_safe('L19A_ST_PRIV_NON_LIES_FR'),
        '_19b': get_col_safe('L19B_ST_PRIV_NON_LIES_ETR'),
        '_20': get_col_calcule('L20_TOTAL_ST_PRIVEE'),
        '_21': get_col_calcule('L21_PLAFOND_ST_PRIVEE'),
        '_22': get_col_calcule('L22_TOTAL_ST'),
        '_23': get_col_calcule('L23_PLAFOND_ORGANISMES_LIES'),
        '_24': get_col_calcule('L24_PLAFOND_ORG_NON_LIES'),
        '_25': get_col_calcule('L25_PLAFOND_GENERAL'),
        '_26': get_col_calcule('L26_TOTAL_ST_PLAFONNE'),
        '_27': get_col_calcule('L27_TOTAL_DEPENSES_RECHERCHE'),
        '_28a': get_col_safe('L28A_SUBVENTIONS'),
        '_28b': get_col_safe('L28B_SOMMES_ENCAISSEES_TIERS'),
        '_29': get_col_safe('L29_DEPENSES_CONSEIL_CIR'),
        '_30': get_col_safe('L30_REMBOURSEMENTS_SUBVENTIONS'),
        '_31a': get_col_calcule('L31A_MONTANT_NET_DEPENSES'),
        '_31b': get_col_calcule('L31B_MONTANT_NET_DEPENSES_DOM'),
        '_32': get_col_safe('L32_FRAIS_COLLECTION'),
        '_33': get_col_safe('L33_FRAIS_DEFENSE_DESSINS_BRUT'),
        '_34': get_col_calcule('L34_TOTAL_DEPENSES_COLLECTION'),
        '_35': get_col_safe('L35_SUBVENTIONS_COLLECTION'),
        '_36': get_col_safe('L36_DEPENSES_CONSEIL_COLLECTION'),
        '_37': get_col_safe('L37_REMBOURSEMENTS_SUBVENTIONS_COLL'),
        '_38a': get_col_calcule('L38A_MONTANT_NET_COLLECTION'),
        '_38b': get_col_calcule('L38B_MONTANT_NET_COLLECTION_DOM'),
        '_39a': get_col_safe('L39A_MONTANT_NET_TOTAL_RD_COLL'),
        '_39b': get_col_safe('L39B_MONTANT_NET_TOTAL_RD_COLL_DOM'),
        '_40a': get_col_calcule('L31A_MONTANT_NET_DEPENSES'),
        '_40b': get_col_calcule('L31B_MONTANT_NET_DEPENSES_DOM'),
        '_41': get_col_safe('L41_CREDIT_IMPOT_RECHERCHE_MOINS_100M'),
        '_42': get_col_safe('L42_QUOTE_PART_RECHERCHE_SOC_PERSONNES'),
        '_43a': get_col_calcule('L43A_CREDIT_IMPOT_RECHERCHE_TOTAL'),
        '_43b': get_col_safe('L43B_CREDIT_IMPOT_RECHERCHE_DOM'),
        '_44a': get_col_calcule('L38A_MONTANT_NET_COLLECTION'),
        '_44b': get_col_calcule('L38B_MONTANT_NET_COLLECTION_DOM'),
        '_45': get_col_safe('L45_CREDIT_IMPOT_COLLECTION_MOINS_100M'),
        '_46': get_col_safe('L46_QUOTE_PART_COLLECTION_SOC_PERSONNES'),
        '_47a': get_col_calcule('L47A_CREDIT_IMPOT_COLL_AVANT_PLAF'),
        '_47b': get_col_safe('L47B_CREDIT_IMPOT_COLL_DOM_AVANT_PLAF'),
        '_48': get_col_safe('L48_AIDES_MINIMIS'),
        '_49': get_col_safe('L49_CUMUL_CREDIT_IMPOT_ET_AIDES'),
        '_50a': get_col_calcule('L50A_CREDIT_IMPOT_COLL_APRES_PLAF'),
        '_50b': get_col_safe('L50B_CREDIT_IMPOT_COLL_DOM_APRES_PLAF'),
        '_51a': get_col_safe('L51A_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION'),
        '_51b': get_col_safe('L51B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM'),
        '_52a': get_col_safe('L52A_DEPENSES_RECHERCHE_LIMITE_100M'),
        '_52b': get_col_safe('L52B_DEPENSES_RECHERCHE_DOM_LIMITE'),
        '_53': get_col_safe('L53_CIR_RECHERCHE_PREMIERE_TRANCHE'),
        '_54': get_col_safe('L54_DEPENSES_RECHERCHE_SUP_100M'),
        '_55': get_col_safe('L55_CIR_RECHERCHE_DEUXIEME_TRANCHE'),
        '_56': get_col_safe('L56_CIR_RECHERCHE_PLUS_100M'),
        '_57': get_col_safe('L57_QUOTE_PART_RECHERCHE_SOC_PERSONNES_PLUS_100M'),
        '_58a': get_col_calcule('L58A_CREDIT_IMPOT_RECHERCHE_TOTAL_PLUS_100M'),
        '_58b': get_col_safe('L58B_CREDIT_IMPOT_RECHERCHE_DOM_PLUS_100M'),
        '_59a': get_col_safe('L59A_MONTANT_NET_COLLECTION_PLUS_100M'),
        '_59b': get_col_safe('L59B_MONTANT_NET_COLLECTION_DOM_PLUS_100M'),
        '_60': get_col_safe('L60_PLAFOND_DISPO_COLLECTION'),
        '_61': get_col_safe('L61_CIR_COLLECTION_PREMIERE_TRANCHE'),
        '_62': get_col_safe('L62_CIR_COLLECTION_DEUXIEME_TRANCHE'),
        '_63': get_col_safe('L63_CIR_COLLECTION_PLUS_100M'),
        '_64': get_col_safe('L64_QUOTE_PART_COLLECTION_SOC_PERSONNES_PLUS_100M'),
        '_65': get_col_safe('L65_CREDIT_IMPOT_COLL_AVANT_PLAF_PLUS_100M'),
        '_66': get_col_safe('L66_AIDES_MINIMIS_PLUS_100M'),
        '_67': get_col_safe('L67_CUMUL_CREDIT_IMPOT_ET_AIDES_PLUS_100M'),
        '_68a': get_col_calcule('L68A_CREDIT_IMPOT_COLL_APRES_PLAF_PLUS_100M'),
        '_68b': get_col_safe('L68B_CREDIT_IMPOT_COLL_DOM_APRES_PLAF_PLUS_100M'),
        '_69a': get_col_safe('L69A_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_PLUS_100M'),
        '_69b': get_col_safe('L69B_MONTANT_TOTAL_CIR_RECHERCHE_COLLECTION_DOM_PLUS_100M'),
        '_70': get_col_safe('L70_DOTATION_AMORT_IMMO_INNOVATION'),
        '_71': get_col_safe('L71_DEPENSES_PERSONNEL_INNOVATION'),
        '_72': get_col_safe('L72_AUTRES_DEPENSES_FONCT_INNOVATION'),
        '_73': get_col_safe('L73_FRAIS_BREVETS_INNOVATION'),
        '_74': get_col_safe('L74_FRAIS_DEFENSE_BREVETS_INNOVATION'),
        '_75': get_col_safe('L75_OPERATIONS_CONFIEES_INNOVATION'),
        '_76': get_col_calcule('L76_TOTAL_DEPENSES_INNOVATION'),
        '_77': get_col_calcule('L77_DEPENSES_INNOVATION_PLAFONNEES'),
        '_78': get_col_safe('L78_SUBVENTIONS_INNOVATION'),
        '_79': get_col_safe('L79_PRESTATIONS_INNOVATION'),
        '_80': get_col_safe('L80_DEPENSES_CONSEIL_INNOVATION'),
        '_81': get_col_safe('L81_REMBOURSEMENTS_SUBVENTIONS_INNO'),
        '_82a': get_col_calcule('L82A_MONTANT_NET_INNOVATION'),
        '_82b': get_col_safe('L82B_MONTANT_NET_INNOVATION_DOM'),
        '_82c': get_col_safe('L82C_MONTANT_NET_INNOVATION_CORSE_MPE'),
        '_82d': get_col_safe('L82D_MONTANT_NET_INNOVATION_CORSE_ME'),
        '_83': get_col_safe('L83_CREDIT_IMPOT_INNOVATION'),
        '_84': get_col_safe('L84_QUOTE_PART_INNOVATION_SOC_PERSONNES'),
        '_85a': get_col_calcule('L85A_TOTAL_CREDIT_IMPOT_INNOVATION'),
        '_85b': get_col_safe('L85B_CREDIT_IMPOT_INNOVATION_DOM'),
        '_85c': get_col_safe('L85C_CREDIT_IMPOT_INNOVATION_CORSE'),
        '_86a': get_col_safe('L86A_TOTAL_CIR_RECH_COLL_INNO'),
        '_86b': get_col_safe('L86B_TOTAL_CIR_RECH_COLL_INNO_DOM')
    }
    
    # 6. Creation du dataframe final dans l'ordre exact
    print("Creation du dataframe final...")
    df_final = pd.DataFrame()
    
    for colonne in COLONNES_CIBLE:
        df_final[colonne] = mapping_complet[colonne]
    
    # 7. Sauvegarde
    print(f"Sauvegarde : {fichier_sortie}")
    df_final.to_csv(fichier_sortie, sep=';', index=False, encoding='utf-8-sig')
    
    # 8. Statistiques
    total_cols = len(df_final.columns)
    cols_avec_donnees = df_final.notna().any().sum()
    
    print(f"\nRESULTATS:")
    print(f"   Lignes creees: {len(df_final):,}")
    print(f"   Colonnes totales: {total_cols}")
    print(f"   Colonnes avec donnees: {cols_avec_donnees}/{total_cols} ({cols_avec_donnees/total_cols_100:.1f}%)")
    print(f"   Colonnes vides: {total_cols - cols_avec_donnees}")
    
    # Detail des colonnes recuperees du complement
    if fichier_complement:
        colonnes_complement = ['caht', 'nb_jd', 'ch_tech', 'cat_jur']
        cols_complement_remplies = sum(1 for col in colonnes_complement if df_final[col].notna().any())
        print(f"   Colonnes complement remplies: {cols_complement_remplies}/4")
    
    print(f"\nTRANSFORMATION TERMINEE")
    print(f"Fichier cree: {fichier_sortie}")
    
    return df_final

# Script principal
if __name__ == "__main__":
    
    # Parametres a modifier
    FICHIER_PRINCIPAL = "C://Users//msamb//Documents//base_finale_sirenise_2021.csv"  # Remplacez par votre fichier
    FICHIER_COMPLEMENT = "C://Users//msamb//Documents//new_millesime_CIR_2021-suite_corrected_sans_doublon.xlsx"  # Fichier complement
    FICHIER_SORTIE = "C://Users//msamb//Documents//base_final_2021_suite_final.csv"  # Nom du fichier de sortie
    
    # Verification
    if not os.path.exists(FICHIER_PRINCIPAL):
        print(f"ERREUR: Fichier principal non trouve: {FICHIER_PRINCIPAL}")
        print("Modifiez la variable FICHIER_PRINCIPAL avec le bon chemin")
        exit(1)
    
    if FICHIER_COMPLEMENT and not os.path.exists(FICHIER_COMPLEMENT):
        print(f"ATTENTION: Fichier complement non trouve: {FICHIER_COMPLEMENT}")
        print("Les colonnes caht, nb_jd, ch_tech, cat_jur resteront vides")
        FICHIER_COMPLEMENT = None
    
    # Transformation
    try:
        resultat = transformer_vers_format_cible(
            fichier_principal=FICHIER_PRINCIPAL,
            fichier_complement=FICHIER_COMPLEMENT,
            fichier_sortie=FICHIER_SORTIE
        )
        
        # Verification de l'ordre des colonnes
        print(f"\nVERIFICATION:")
        print(f"Nombre de colonnes: {len(resultat.columns)}")
        print(f"Premieres colonnes: {list(resultat.columns[:10])}")
        print(f"Dernieres colonnes: {list(resultat.columns[-5:])}")
        
    except Exception as e:
        print(f"ERREUR: {e}")
        print("Verifiez le chemin et le format de votre fichier")

TRANSFORMATION VERS FORMAT CIBLE
Chargement : C://Users//msamb//Documents//base_finale_sirenise_2021.csv
   4,543 lignes chargees
Chargement complement : C://Users//msamb//Documents//new_millesime_CIR_2021-suite_corrected_sans_doublon.xlsx
   Jointure effectuee - 4 colonnes recuperees
Mapping des colonnes...
Creation du dataframe final...
Sauvegarde : C://Users//msamb//Documents//base_final_2021_suite_final.csv


C:\Users\msamb\AppData\Local\Temp\ipykernel_16792\49172490.py:247: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[colonne] = mapping_complet[colonne]
C:\Users\msamb\AppData\Local\Temp\ipykernel_16792\49172490.py:247: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[colonne] = mapping_complet[colonne]
C:\Users\msamb\AppData\Local\Temp\ipykernel_16792\49172490.py:247: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons


RESULTATS:
   Lignes creees: 4,619
   Colonnes totales: 157
ERREUR: name 'total_cols_100' is not defined
Verifiez le chemin et le format de votre fichier


In [18]:
import pandas as pd
import numpy as np
import os

# Dictionnaire des régions par département
REGIONS = {
    # Auvergne-Rhône-Alpes
    '01': ('RHONE-ALPES', 'AUVERGNE-RHONE-ALPES'),
    '03': ('AUVERGNE', 'AUVERGNE-RHONE-ALPES'),
    '07': ('RHONE-ALPES', 'AUVERGNE-RHONE-ALPES'),
    '15': ('AUVERGNE', 'AUVERGNE-RHONE-ALPES'),
    '26': ('RHONE-ALPES', 'AUVERGNE-RHONE-ALPES'),
    '38': ('RHONE-ALPES', 'AUVERGNE-RHONE-ALPES'),
    '42': ('RHONE-ALPES', 'AUVERGNE-RHONE-ALPES'),
    '43': ('AUVERGNE', 'AUVERGNE-RHONE-ALPES'),
    '63': ('AUVERGNE', 'AUVERGNE-RHONE-ALPES'),
    '69': ('RHONE-ALPES', 'AUVERGNE-RHONE-ALPES'),
    '73': ('RHONE-ALPES', 'AUVERGNE-RHONE-ALPES'),
    '74': ('RHONE-ALPES', 'AUVERGNE-RHONE-ALPES'),
    
    # Bourgogne-Franche-Comté
    '21': ('BOURGOGNE', 'BOURGOGNE-FRANCHE-COMTE'),
    '25': ('FRANCHE-COMTE', 'BOURGOGNE-FRANCHE-COMTE'),
    '39': ('FRANCHE-COMTE', 'BOURGOGNE-FRANCHE-COMTE'),
    '58': ('BOURGOGNE', 'BOURGOGNE-FRANCHE-COMTE'),
    '70': ('FRANCHE-COMTE', 'BOURGOGNE-FRANCHE-COMTE'),
    '71': ('BOURGOGNE', 'BOURGOGNE-FRANCHE-COMTE'),
    '89': ('BOURGOGNE', 'BOURGOGNE-FRANCHE-COMTE'),
    '90': ('FRANCHE-COMTE', 'BOURGOGNE-FRANCHE-COMTE'),
    
    # Bretagne
    '22': ('BRETAGNE', 'BRETAGNE'),
    '29': ('BRETAGNE', 'BRETAGNE'),
    '35': ('BRETAGNE', 'BRETAGNE'),
    '56': ('BRETAGNE', 'BRETAGNE'),
    
    # Centre-Val de Loire
    '18': ('CENTRE', 'CENTRE-VAL DE LOIRE'),
    '28': ('CENTRE', 'CENTRE-VAL DE LOIRE'),
    '36': ('CENTRE', 'CENTRE-VAL DE LOIRE'),
    '37': ('CENTRE', 'CENTRE-VAL DE LOIRE'),
    '41': ('CENTRE', 'CENTRE-VAL DE LOIRE'),
    '45': ('CENTRE', 'CENTRE-VAL DE LOIRE'),
    
    # Corse
    '2A': ('CORSE', 'CORSE'),
    '2B': ('CORSE', 'CORSE'),
    '20': ('CORSE', 'CORSE'),  # Cas général pour la Corse
    
    # Grand Est
    '08': ('CHAMPAGNE-ARDENNE', 'GRAND EST'),
    '10': ('CHAMPAGNE-ARDENNE', 'GRAND EST'),
    '51': ('CHAMPAGNE-ARDENNE', 'GRAND EST'),
    '52': ('CHAMPAGNE-ARDENNE', 'GRAND EST'),
    '54': ('LORRAINE', 'GRAND EST'),
    '55': ('LORRAINE', 'GRAND EST'),
    '57': ('LORRAINE', 'GRAND EST'),
    '67': ('ALSACE', 'GRAND EST'),
    '68': ('ALSACE', 'GRAND EST'),
    '88': ('LORRAINE', 'GRAND EST'),
    
    # Hauts-de-France
    '02': ('PICARDIE', 'HAUTS-DE-FRANCE'),
    '59': ('NORD-PAS-DE-CALAIS', 'HAUTS-DE-FRANCE'),
    '60': ('PICARDIE', 'HAUTS-DE-FRANCE'),
    '62': ('NORD-PAS-DE-CALAIS', 'HAUTS-DE-FRANCE'),
    '80': ('PICARDIE', 'HAUTS-DE-FRANCE'),
    
    # Île-de-France
    '75': ('ILE-DE-FRANCE', 'ILE-DE-FRANCE'),
    '77': ('ILE-DE-FRANCE', 'ILE-DE-FRANCE'),
    '78': ('ILE-DE-FRANCE', 'ILE-DE-FRANCE'),
    '91': ('ILE-DE-FRANCE', 'ILE-DE-FRANCE'),
    '92': ('ILE-DE-FRANCE', 'ILE-DE-FRANCE'),
    '93': ('ILE-DE-FRANCE', 'ILE-DE-FRANCE'),
    '94': ('ILE-DE-FRANCE', 'ILE-DE-FRANCE'),
    '95': ('ILE-DE-FRANCE', 'ILE-DE-FRANCE'),
    
    # Normandie
    '14': ('BASSE-NORMANDIE', 'NORMANDIE'),
    '27': ('HAUTE-NORMANDIE', 'NORMANDIE'),
    '50': ('BASSE-NORMANDIE', 'NORMANDIE'),
    '61': ('BASSE-NORMANDIE', 'NORMANDIE'),
    '76': ('HAUTE-NORMANDIE', 'NORMANDIE'),
    
    # Nouvelle-Aquitaine
    '16': ('POITOU-CHARENTES', 'NOUVELLE-AQUITAINE'),
    '17': ('POITOU-CHARENTES', 'NOUVELLE-AQUITAINE'),
    '19': ('LIMOUSIN', 'NOUVELLE-AQUITAINE'),
    '23': ('LIMOUSIN', 'NOUVELLE-AQUITAINE'),
    '24': ('AQUITAINE', 'NOUVELLE-AQUITAINE'),
    '33': ('AQUITAINE', 'NOUVELLE-AQUITAINE'),
    '40': ('AQUITAINE', 'NOUVELLE-AQUITAINE'),
    '47': ('AQUITAINE', 'NOUVELLE-AQUITAINE'),
    '64': ('AQUITAINE', 'NOUVELLE-AQUITAINE'),
    '79': ('POITOU-CHARENTES', 'NOUVELLE-AQUITAINE'),
    '86': ('POITOU-CHARENTES', 'NOUVELLE-AQUITAINE'),
    '87': ('LIMOUSIN', 'NOUVELLE-AQUITAINE'),
    
    # Occitanie
    '09': ('MIDI-PYRENEES', 'OCCITANIE'),
    '11': ('LANGUEDOC-ROUSSILLON', 'OCCITANIE'),
    '12': ('MIDI-PYRENEES', 'OCCITANIE'),
    '30': ('LANGUEDOC-ROUSSILLON', 'OCCITANIE'),
    '31': ('MIDI-PYRENEES', 'OCCITANIE'),
    '32': ('MIDI-PYRENEES', 'OCCITANIE'),
    '34': ('LANGUEDOC-ROUSSILLON', 'OCCITANIE'),
    '46': ('MIDI-PYRENEES', 'OCCITANIE'),
    '48': ('LANGUEDOC-ROUSSILLON', 'OCCITANIE'),
    '65': ('MIDI-PYRENEES', 'OCCITANIE'),
    '66': ('LANGUEDOC-ROUSSILLON', 'OCCITANIE'),
    '81': ('MIDI-PYRENEES', 'OCCITANIE'),
    '82': ('MIDI-PYRENEES', 'OCCITANIE'),
    
    # Pays de la Loire
    '44': ('PAYS DE LA LOIRE', 'PAYS DE LA LOIRE'),
    '49': ('PAYS DE LA LOIRE', 'PAYS DE LA LOIRE'),
    '53': ('PAYS DE LA LOIRE', 'PAYS DE LA LOIRE'),
    '72': ('PAYS DE LA LOIRE', 'PAYS DE LA LOIRE'),
    '85': ('PAYS DE LA LOIRE', 'PAYS DE LA LOIRE'),
    
    # Provence-Alpes-Côte d'Azur
    '04': ('PROVENCE-ALPES-COTE D\'AZUR', 'PROVENCE-ALPES-COTE D\'AZUR'),
    '05': ('PROVENCE-ALPES-COTE D\'AZUR', 'PROVENCE-ALPES-COTE D\'AZUR'),
    '06': ('PROVENCE-ALPES-COTE D\'AZUR', 'PROVENCE-ALPES-COTE D\'AZUR'),
    '13': ('PROVENCE-ALPES-COTE D\'AZUR', 'PROVENCE-ALPES-COTE D\'AZUR'),
    '83': ('PROVENCE-ALPES-COTE D\'AZUR', 'PROVENCE-ALPES-COTE D\'AZUR'),
    '84': ('PROVENCE-ALPES-COTE D\'AZUR', 'PROVENCE-ALPES-COTE D\'AZUR'),
    
    # DOM-TOM
    '971': ('GUADELOUPE', 'GUADELOUPE'),
    '972': ('MARTINIQUE', 'MARTINIQUE'),
    '973': ('GUYANE', 'GUYANE'),
    '974': ('LA REUNION', 'LA REUNION'),
    '975': ('SAINT-PIERRE-ET-MIQUELON', 'SAINT-PIERRE-ET-MIQUELON'),
    '976': ('MAYOTTE', 'MAYOTTE'),
    '977': ('SAINT-BARTHELEMY', 'SAINT-BARTHELEMY'),
    '978': ('SAINT-MARTIN', 'SAINT-MARTIN'),
    '986': ('WALLIS-ET-FUTUNA', 'WALLIS-ET-FUTUNA'),
    '987': ('POLYNESIE FRANCAISE', 'POLYNESIE FRANCAISE'),
    '988': ('NOUVELLE-CALEDONIE', 'NOUVELLE-CALEDONIE')
}

# Dictionnaire des tranches d'effectifs
TRANCHES_EFFECTIFS = {
    '00': '0 salarié',
    '01': '1 ou 2 salariés',
    '02': '3 à 5 salariés',
    '03': '6 à 9 salariés',
    '11': '10 à 19 salariés',
    '12': '20 à 49 salariés',
    '21': '50 à 99 salariés',
    '22': '100 à 199 salariés',
    '31': '200 à 249 salariés',
    '32': '250 à 499 salariés',
    '41': '500 à 999 salariés',
    '42': '1000 à 1999 salariés',
    '51': '2000 à 4999 salariés',
    '52': '5000 à 9999 salariés',
    '53': '10000 salariés et plus',
    'NN': 'Unités non employeuses',
    # Ajout de codes alternatifs parfois utilisés
    '0': '0 salarié',
    '1': '1 ou 2 salariés',
    '2': '3 à 5 salariés',
    '3': '6 à 9 salariés',
    '10': '10 à 19 salariés',
    '20': '20 à 49 salariés',
    '50': '50 à 99 salariés',
    '100': '100 à 199 salariés',
    '200': '200 à 249 salariés',
    '250': '250 à 499 salariés',
    '500': '500 à 999 salariés',
    '1000': '1000 à 1999 salariés',
    '2000': '2000 à 4999 salariés',
    '5000': '5000 à 9999 salariés',
    '10000': '10000 salariés et plus'
}

# Dictionnaire des départements français
DEPARTEMENTS = {
    '01': 'Ain', '02': 'Aisne', '03': 'Allier', '04': 'Alpes-de-Haute-Provence',
    '05': 'Hautes-Alpes', '06': 'Alpes-Maritimes', '07': 'Ardèche', '08': 'Ardennes',
    '09': 'Ariège', '10': 'Aube', '11': 'Aude', '12': 'Aveyron',
    '13': 'Bouches-du-Rhône', '14': 'Calvados', '15': 'Cantal', '16': 'Charente',
    '17': 'Charente-Maritime', '18': 'Cher', '19': 'Corrèze', '2A': 'Corse-du-Sud',
    '2B': 'Haute-Corse', '21': 'Côte-d\'Or', '22': 'Côtes-d\'Armor', '23': 'Creuse',
    '24': 'Dordogne', '25': 'Doubs', '26': 'Drôme', '27': 'Eure',
    '28': 'Eure-et-Loir', '29': 'Finistère', '30': 'Gard', '31': 'Haute-Garonne',
    '32': 'Gers', '33': 'Gironde', '34': 'Hérault', '35': 'Ille-et-Vilaine',
    '36': 'Indre', '37': 'Indre-et-Loire', '38': 'Isère', '39': 'Jura',
    '40': 'Landes', '41': 'Loir-et-Cher', '42': 'Loire', '43': 'Haute-Loire',
    '44': 'Loire-Atlantique', '45': 'Loiret', '46': 'Lot', '47': 'Lot-et-Garonne',
    '48': 'Lozère', '49': 'Maine-et-Loire', '50': 'Manche', '51': 'Marne',
    '52': 'Haute-Marne', '53': 'Mayenne', '54': 'Meurthe-et-Moselle', '55': 'Meuse',
    '56': 'Morbihan', '57': 'Moselle', '58': 'Nièvre', '59': 'Nord',
    '60': 'Oise', '61': 'Orne', '62': 'Pas-de-Calais', '63': 'Puy-de-Dôme',
    '64': 'Pyrénées-Atlantiques', '65': 'Hautes-Pyrénées', '66': 'Pyrénées-Orientales', '67': 'Bas-Rhin',
    '68': 'Haut-Rhin', '69': 'Rhône', '70': 'Haute-Saône', '71': 'Saône-et-Loire',
    '72': 'Sarthe', '73': 'Savoie', '74': 'Haute-Savoie', '75': 'Paris',
    '76': 'Seine-Maritime', '77': 'Seine-et-Marne', '78': 'Yvelines', '79': 'Deux-Sèvres',
    '80': 'Somme', '81': 'Tarn', '82': 'Tarn-et-Garonne', '83': 'Var',
    '84': 'Vaucluse', '85': 'Vendée', '86': 'Vienne', '87': 'Haute-Vienne',
    '88': 'Vosges', '89': 'Yonne', '90': 'Territoire de Belfort', '91': 'Essonne',
    '92': 'Hauts-de-Seine', '93': 'Seine-Saint-Denis', '94': 'Val-de-Marne', '95': 'Val-d\'Oise',
    # DOM-TOM
    '971': 'Guadeloupe', '972': 'Martinique', '973': 'Guyane', '974': 'La Réunion',
    '975': 'Saint-Pierre-et-Miquelon', '976': 'Mayotte', '977': 'Saint-Barthélemy',
    '978': 'Saint-Martin', '986': 'Wallis-et-Futuna', '987': 'Polynésie française',
    '988': 'Nouvelle-Calédonie'
}

# Dictionnaire des pays en code ISO vers nom complet
PAYS = {
    'FR': 'FRANCE',
    'BE': 'BELGIQUE',
    'DE': 'ALLEMAGNE',
    'IT': 'ITALIE',
    'ES': 'ESPAGNE',
    'GB': 'ROYAUME-UNI',
    'CH': 'SUISSE',
    'LU': 'LUXEMBOURG',
    'NL': 'PAYS-BAS',
    'PT': 'PORTUGAL',
    'US': 'ÉTATS-UNIS',
    'CA': 'CANADA',
    'CN': 'CHINE',
    'JP': 'JAPON',
    'AU': 'AUSTRALIE',
    'BR': 'BRÉSIL',
    'IN': 'INDE',
    'MX': 'MEXIQUE',
    'RU': 'RUSSIE',
    'KR': 'CORÉE DU SUD'
}

# Dictionnaire des catégories juridiques
CATEGORIES_JURIDIQUES = {
    '1000': 'Entrepreneur individuel',
    '2110': 'Indivision',
    '2120': 'Société créée de fait',
    '2210': 'GIE',
    '2220': 'GEIE',
    '2310': 'Société en participation',
    '2320': 'Société créée de fait',
    '2385': 'Société d\'exercice libéral',
    '2400': 'Fiducie',
    '2700': 'Paroisse hors zone concordataire',
    '2900': 'Autre groupement de droit privé non doté de la personnalité morale',
    '3110': 'Représentation ou agence commerciale d\'état ou organisme public étranger',
    '3120': 'Société commerciale étrangère',
    '3205': 'Organisation internationale',
    '3210': 'État collectivité ou établissement public étranger',
    '3220': 'Société étrangère non immatriculée au RCS',
    '3290': 'Autre personne morale de droit étranger',
    '4110': 'Établissement public national à caractère industriel ou commercial',
    '4120': 'Établissement public national à caractère administratif',
    '4130': 'Exploitant public',
    '4140': 'Établissement public local à caractère industriel ou commercial',
    '4150': 'Régie d\'une collectivité locale à caractère industriel ou commercial',
    '4160': 'Institution Banque de France',
    '5195': 'Caisse de crédit agricole mutuel',
    '5196': 'Caisse de crédit mutuel',
    '5202': 'Société européenne',
    '5203': 'SCIC',
    '5306': 'Société en commandite simple',
    '5307': 'Société en commandite par actions',
    '5308': 'Société en nom collectif',
    '5309': 'Fiducie',
    '5310': 'SCOP',
    '5370': 'Société par actions simplifiée unipersonnelle',
    '5385': 'Société d\'exercice libéral',
    '5410': 'SARL unipersonnelle',
    '5415': 'SARL d\'économie mixte',
    '5422': 'SARL immobilière',
    '5426': 'SARL de famille',
    '5430': 'SARL d\'attribution',
    '5431': 'SARL mixte d\'intérêt agricole',
    '5432': 'SARL d\'intérêt collectif agricole',
    '5442': 'SARL d\'attribution',
    '5443': 'SARL coopérative de construction',
    '5451': 'SARL coopérative de consommation',
    '5453': 'SARL coopérative artisanale',
    '5454': 'SARL coopérative d\'intérêt maritime',
    '5455': 'SARL coopérative de transport',
    '5458': 'SARL coopérative ouvrière de production',
    '5459': 'SARL union de sociétés coopératives',
    '5460': 'Autre SARL coopérative',
    '5485': 'Société d\'exercice libéral à responsabilité limitée',
    '5498': 'SARL unipersonnelle',
    '5499': 'SARL',
    '5505': 'SA à participation ouvrière à conseil d\'administration',
    '5510': 'SA nationale à conseil d\'administration',
    '5515': 'SA d\'économie mixte à conseil d\'administration',
    '5520': 'SICAV',
    '5522': 'SA immobilière d\'investissement',
    '5525': 'SA immobilière de gestion',
    '5530': 'SA d\'aménagement foncier et d\'équipement rural',
    '5531': 'SA mixte d\'intérêt agricole',
    '5532': 'SAFER',
    '5542': 'SA d\'attribution',
    '5543': 'SA coopérative de construction',
    '5546': 'SA de HLM',
    '5547': 'Société anonyme de crédit immobilier',
    '5548': 'SA de crédit-bail',
    '5551': 'SA coopérative de consommation',
    '5552': 'SA coopérative de commerçants-détaillants',
    '5553': 'SA coopérative artisanale',
    '5554': 'SA coopérative d\'intérêt maritime',
    '5555': 'SA coopérative de transport',
    '5558': 'SA coopérative ouvrière de production',
    '5559': 'SA union de sociétés coopératives',
    '5560': 'Autre SA coopérative',
    '5570': 'SCIC',
    '5585': 'Société d\'exercice libéral',
    '5599': 'SA à conseil d\'administration',
    '5605': 'SA à participation ouvrière à directoire',
    '5610': 'SA nationale à directoire',
    '5615': 'SA d\'économie mixte à directoire',
    '5620': 'SICAV',
    '5622': 'SA immobilière d\'investissement',
    '5625': 'SA immobilière de gestion',
    '5630': 'SAFER',
    '5631': 'SA mixte d\'intérêt agricole',
    '5632': 'SA d\'intérêt collectif agricole',
    '5642': 'SA d\'attribution',
    '5643': 'SA coopérative de construction',
    '5646': 'SA de HLM',
    '5647': 'SA de crédit immobilier',
    '5648': 'SA de crédit-bail',
    '5651': 'SA coopérative de consommation',
    '5652': 'SA coopérative de commerçants-détaillants',
    '5653': 'SA coopérative artisanale',
    '5654': 'SA coopérative d\'intérêt maritime',
    '5655': 'SA coopérative de transport',
    '5658': 'SA coopérative ouvrière de production',
    '5659': 'SA union de sociétés coopératives',
    '5660': 'Autre SA coopérative',
    '5670': 'SCIC',
    '5685': 'Société d\'exercice libéral',
    '5699': 'SA à directoire',
    '5710': 'SAS',
    '5720': 'SASU',
    '5785': 'Société d\'exercice libéral par actions simplifiée',
    '5800': 'Société européenne',
    '6100': 'Caisse d\'épargne et de prévoyance',
    '6210': 'GEIE',
    '6220': 'GIE',
    '6316': 'EPCI',
    '6317': 'Secteur de commune',
    '6318': 'ASA',
    '6411': 'Commune et commune nouvelle',
    '6412': 'Commune associée et commune déléguée',
    '6413': 'Section de commune',
    '6414': 'Ensemble urbain',
    '6421': 'Région',
    '6422': 'Département',
    '6431': 'Métropole',
    '6432': 'District urbain',
    '6433': 'Communauté urbaine',
    '6434': 'Métropole',
    '6435': 'Syndicat communautaire',
    '6436': 'Communauté de communes',
    '6441': 'Commune et commune nouvelle',
    '6442': 'Commune associée et commune déléguée',
    '6443': 'Section de commune',
    '6444': 'Ensemble urbain',
    '6451': 'Syndicat intercommunal à vocation unique',
    '6452': 'Syndicat intercommunal à vocation multiple',
    '6453': 'Syndicat mixte fermé',
    '6454': 'Syndicat mixte ouvert',
    '6455': 'Autre établissement public local',
    '6460': 'Régie personnalisée',
    '6461': 'Institution interdépartementale ou entente',
    '6462': 'Pôle métropolitain',
    '6463': 'Pôle d\'équilibre territorial et rural',
    '6464': 'Syndicat mixte fermé',
    '6465': 'Syndicat mixte ouvert',
    '6466': 'Commune et commune nouvelle',
    '6467': 'Communauté de villes',
    '6468': 'Autre établissement public local',
    '6469': 'Syndicat intercommunal ou mixte',
    '6470': 'Autre établissement public local',
    '6490': 'Autre collectivité territoriale',
    '6499': 'Autre collectivité territoriale',
    '6511': 'Syndicat de propriétaires',
    '6512': 'ASL',
    '6513': 'AFUL',
    '6514': 'Syndicat de communes',
    '6515': 'Pôle d\'équilibre territorial et rural',
    '6516': 'Syndicat mixte',
    '6517': 'Commission syndicale',
    '6518': 'Pôle métropolitain',
    '6519': 'Société d\'économie mixte',
    '6520': 'Société publique locale',
    '6521': 'AFU',
    '6522': 'ASA',
    '6523': 'ASCO',
    '6524': 'Commission syndicale',
    '6525': 'Association syndicale',
    '6529': 'Autre groupement',
    '6531': 'Commission de gestion des services publics',
    '6532': 'Commission de gestion urbaine',
    '6533': 'ASL',
    '6534': 'AFUL',
    '6535': 'ASCO',
    '6536': 'Commission syndicale de gestion',
    '6537': 'Centre communal d\'action sociale',
    '6538': 'Centre intercommunal d\'action sociale',
    '6539': 'Centre d\'action sociale',
    '6540': 'Autre établissement public administratif',
    '6541': 'CROUS',
    '6542': 'Caisse de crédit municipal',
    '6543': 'Établissement d\'hospitalisation',
    '6544': 'Syndicat inter hospitalier',
    '6545': 'Établissement public médico-social et social communal',
    '6546': 'Établissement public médico-social et social intercommunal',
    '6547': 'Établissement public médico-social et social départemental',
    '6548': 'Établissement public médico-social et social interdépartemental',
    '6549': 'Établissement public médico-social et social national',
    '6551': 'Écoles',
    '6552': 'Université',
    '6553': 'Établissement public à caractère scientifique culturel et professionnel',
    '6554': 'Autre établissement public d\'enseignement',
    '6555': 'OPH communal',
    '6556': 'OPH intercommunal',
    '6557': 'OPH départemental',
    '6558': 'OPH interdépartemental',
    '6559': 'OPH régional',
    '6560': 'OPH national',
    '6561': 'Autre établissement public administratif',
    '6562': 'EPA',
    '6563': 'DEL',
    '6564': 'EPIC',
    '6565': 'EPIC',
    '6566': 'EPA',
    '6567': 'EPA',
    '6568': 'EPA',
    '6569': 'EPA',
    '6571': 'Service déconcentré de l\'État à compétence territoriale',
    '6572': 'Service déconcentré de l\'État à compétence nationale',
    '6573': 'Service central d\'un ministère',
    '6574': 'Service ministériel déconcentré à compétence régionale',
    '6575': 'Service ministériel déconcentré à compétence départementale',
    '6576': 'Service ministériel déconcentré à compétence infra-départementale',
    '6577': 'Service ministériel déconcentré à compétence territoriale',
    '6578': 'Service ministériel déconcentré à compétence nationale',
    '6579': 'Autre service administratif',
    '6580': 'Autre service administratif',
    '6581': 'Organisme consulaire',
    '6582': 'Établissement public des cultes d\'Alsace-Moselle',
    '6583': 'Établissement public administratif',
    '6584': 'Groupement d\'intérêt public',
    '6585': 'Autre établissement public national administratif',
    '6589': 'Groupement de coopération sanitaire',
    '6595': 'Association syndicale autorisée',
    '6596': 'Association foncière urbaine',
    '6597': 'Association foncière de remembrement',
    '6598': 'Association syndicale de propriétaires',
    '6599': 'Autre personne morale de droit administratif',
    '7111': 'Autorité constitutionnelle',
    '7112': 'Autorité administrative indépendante',
    '7113': 'Ministère',
    '7120': 'Service central d\'un ministère',
    '7150': 'Service du ministère de la Défense',
    '7160': 'Service déconcentré à compétence régionale',
    '7171': 'Service déconcentré de l\'État à compétence régionale',
    '7172': 'Service déconcentré de l\'État à compétence départementale',
    '7179': 'Autre service déconcentré de l\'État',
    '7190': 'Échelon de contrôle',
    '7210': 'Commune et commune nouvelle',
    '7220': 'Département',
    '7225': 'Collectivité et territoire d\'Outre-Mer',
    '7229': 'Collectivité territoriale',
    '7230': 'Région',
    '7312': 'Commune associée et commune déléguée',
    '7313': 'Section de commune',
    '7314': 'Ensemble urbain',
    '7321': 'ASA',
    '7322': 'AFUL',
    '7323': 'AFU',
    '7331': 'Établissement public local d\'enseignement',
    '7340': 'Pôle d\'équilibre territorial et rural',
    '7341': 'Secteur de commune',
    '7342': 'District urbain',
    '7343': 'Communauté urbaine',
    '7344': 'Métropole',
    '7345': 'Syndicat communautaire',
    '7346': 'Communauté de communes',
    '7347': 'Communauté de villes',
    '7348': 'Communauté d\'agglomération',
    '7349': 'Autre établissement public local',
    '7351': 'Institution interdépartementale ou entente',
    '7352': 'Établissement public interdépartemental',
    '7353': 'Syndicat interdépartemental',
    '7354': 'Institution interdépartementale ou entente',
    '7355': 'Institution interdépartementale ou entente',
    '7356': 'Syndicat intercommunal à vocation unique',
    '7357': 'Syndicat intercommunal à vocation multiple',
    '7361': 'Centre régional de la propriété forestière',
    '7362': 'Établissement public foncier',
    '7363': 'Établissement public d\'aménagement',
    '7364': 'Établissement public d\'aménagement et de restructuration',
    '7365': 'Autre établissement public local de développement',
    '7366': 'Syndicat mixte fermé',
    '7367': 'Syndicat mixte ouvert',
    '7371': 'OPH communal',
    '7372': 'OPH intercommunal',
    '7373': 'OPH départemental',
    '7378': 'Régie d\'une collectivité locale à caractère administratif',
    '7379': 'Régie',
    '7381': 'Organisme consulaire',
    '7382': 'Établissement public des cultes d\'Alsace-Moselle',
    '7383': 'Établissement public local',
    '7384': 'Établissement public local culturel',
    '7385': 'Établissement public local social et médico-social',
    '7389': 'Établissement public',
    '7410': 'Groupement d\'intérêt public',
    '7420': 'Groupement d\'intérêt public',
    '7430': 'Établissement public',
    '7440': 'Groupement de coopération sanitaire',
    '7450': 'Établissement public',
    '7460': 'Groupement d\'intérêt public',
    '7470': 'Groupement de coopération',
    '7480': 'Autre personne morale de droit public',
    '7490': 'Autre personne morale de droit public',
    '8110': 'Régime général de la Sécurité sociale',
    '8120': 'Régime spécial de Sécurité sociale',
    '8130': 'Institution de retraite complémentaire',
    '8140': 'Mutualité sociale agricole',
    '8150': 'Régime maladie des non salariés',
    '8160': 'Régime vieillesse des non salariés',
    '8170': 'Régime d\'assurance chômage',
    '8190': 'Autre organisme de Sécurité sociale',
    '8210': 'Mutuelle',
    '8220': 'Société mutualiste',
    '8290': 'Autre organisme mutualiste',
    '8310': 'Comité d\'entreprise',
    '8311': 'Comité central d\'entreprise',
    '8410': 'Syndicat de salariés',
    '8420': 'Syndicat patronal',
    '8450': 'Ordre professionnel',
    '8470': 'Centre technique industriel et comité professionnel du développement économique',
    '8490': 'Autre organisme professionnel',
    '8510': 'Institution de prévoyance',
    '8520': 'Institution de retraite supplémentaire',
    '9110': 'Syndicat de copropriété',
    '9150': 'ASL',
    '9210': 'Association non déclarée',
    '9220': 'Association déclarée',
    '9221': 'Association déclarée',
    '9222': 'Association d\'insertion par l\'économique',
    '9223': 'Association intermédiaire',
    '9224': 'Groupement d\'employeurs',
    '9230': 'Association reconnue d\'utilité publique',
    '9240': 'Congrégation',
    '9250': 'Association de droit local',
    '9260': 'Association agréée',
    '9300': 'Fondation',
    '9900': 'Autre organisme privé',
    '9970': 'Groupement de droit privé'
}

def convertir_date_en_tranche(date_creation):
    """
    Convertit une date de création en tranche d'années simplifiée
    Par exemple: 2005 -> "de 2000 à 2009"
    """
    try:
        if pd.isna(date_creation) or date_creation == '':
            return np.nan
        
        # Extraire l'année de la date
        annee = int(str(date_creation)[:4])
        
        # Déterminer la tranche simplifiée
        if annee < 1980:
            return "avant 1980"
        elif annee < 1990:
            return "de 1980 à 1989"
        elif annee < 2000:
            return "de 1990 à 1999"
        elif annee < 2010:
            return "de 2000 à 2009"
        elif annee < 2020:
            return "de 2010 à 2019"
        else:
            return "de 2020 à aujourd'hui"
    except:
        return np.nan

def mapper_tranche_effectifs(code_tranche):
    """
    Convertit le code tranche effectifs en libellé
    """
    if pd.isna(code_tranche) or code_tranche == '':
        return np.nan
    
    code_str = str(code_tranche).strip().upper()
    
    if code_str in TRANCHES_EFFECTIFS:
        return TRANCHES_EFFECTIFS[code_str]
    
    # Si pas trouvé, essayer de deviner
    try:
        code_num = int(code_str)
        if code_num == 0:
            return '0 salarié'
        elif code_num <= 2:
            return '1 ou 2 salariés'
        elif code_num <= 5:
            return '3 à 5 salariés'
        elif code_num <= 9:
            return '6 à 9 salariés'
        elif code_num <= 19:
            return '10 à 19 salariés'
        elif code_num <= 49:
            return '20 à 49 salariés'
        elif code_num <= 99:
            return '50 à 99 salariés'
        elif code_num <= 199:
            return '100 à 199 salariés'
        elif code_num <= 249:
            return '200 à 249 salariés'
        elif code_num <= 499:
            return '250 à 499 salariés'
        elif code_num <= 999:
            return '500 à 999 salariés'
        elif code_num <= 1999:
            return '1000 à 1999 salariés'
        elif code_num <= 4999:
            return '2000 à 4999 salariés'
        elif code_num <= 9999:
            return '5000 à 9999 salariés'
        else:
            return '10000 salariés et plus'
    except:
        return code_str  # Retourner tel quel si impossible à convertir

def mapper_regions(code_dept):
    """
    Retourne old_region et region à partir du code département
    """
    if pd.isna(code_dept) or code_dept == '':
        return (np.nan, np.nan)
    
    code_str = str(code_dept).strip()
    
    # Gérer les codes à 3 chiffres pour les DOM-TOM
    if len(code_str) == 3:
        if code_str in REGIONS:
            return REGIONS[code_str]
    # Gérer les codes à 2 chiffres
    elif len(code_str) == 2 or (len(code_str) == 1 and code_str.isdigit()):
        code_str = code_str.zfill(2)
        if code_str in REGIONS:
            return REGIONS[code_str]
    
    return (np.nan, np.nan)

def mapper_categorie_juridique(code_categorie):
    """
    Convertit le code catégorie juridique en libellé
    """
    if pd.isna(code_categorie) or code_categorie == '':
        return np.nan
    
    code_str = str(code_categorie).zfill(4)
    
    # Si le code exact existe
    if code_str in CATEGORIES_JURIDIQUES:
        return CATEGORIES_JURIDIQUES[code_str]
    
    # Sinon, chercher avec les premiers chiffres
    for i in range(3, 0, -1):
        code_partiel = code_str[:i]
        if code_partiel + '00' in CATEGORIES_JURIDIQUES:
            return CATEGORIES_JURIDIQUES[code_partiel + '00']
    
    # Si toujours pas trouvé, essayer de mapper les codes les plus courants
    if code_str.startswith('54'):
        return 'SARL'
    elif code_str.startswith('55'):
        return 'Société anonyme'
    elif code_str.startswith('57'):
        return 'Société par actions simplifiée'
    
    return f"Forme juridique non identifiée (code: {code_str})"

def mapper_departement(code_dept):
    """
    Convertit le code département en nom complet
    """
    if pd.isna(code_dept) or code_dept == '':
        return np.nan
    
    code_str = str(code_dept).strip()
    
    # Gérer les codes à 3 chiffres pour les DOM-TOM
    if len(code_str) == 3:
        if code_str in DEPARTEMENTS:
            return DEPARTEMENTS[code_str]
    # Gérer les codes à 2 chiffres
    elif len(code_str) == 2 or (len(code_str) == 1 and code_str.isdigit()):
        code_str = code_str.zfill(2)
        if code_str in DEPARTEMENTS:
            return DEPARTEMENTS[code_str]
    
    return code_str  # Retourner le code si pas trouvé

def mapper_pays(code_pays):
    """
    Convertit le code pays en nom complet
    """
    if pd.isna(code_pays) or code_pays == '':
        return 'FRANCE'  # Par défaut FRANCE
    
    code_str = str(code_pays).strip().upper()
    
    # Si c'est déjà un nom complet connu
    if code_str in ['FRANCE', 'BELGIQUE', 'ALLEMAGNE', 'ITALIE', 'ESPAGNE']:
        return code_str
    
    # Si c'est le code FR, retourner FRANCE
    if code_str == 'FR':
        return 'FRANCE'
    
    # Si c'est un code ISO
    if code_str in PAYS:
        return PAYS[code_str]
    
    # Sinon retourner tel quel
    return code_str

def extraire_code_dept_du_cp(code_postal):
    """
    Extrait le code département du code postal
    """
    if pd.isna(code_postal) or code_postal == '':
        return np.nan
    
    cp_str = str(code_postal).strip().zfill(5)
    
    # Cas spéciaux pour les DOM-TOM
    if cp_str.startswith('971'):
        return '971'
    elif cp_str.startswith('972'):
        return '972'
    elif cp_str.startswith('973'):
        return '973'
    elif cp_str.startswith('974'):
        return '974'
    elif cp_str.startswith('975'):
        return '975'
    elif cp_str.startswith('976'):
        return '976'
    
    # Cas spécial pour la Corse
    if cp_str.startswith('20'):
        # 20000-20199 et 20900-20999 = Corse du Sud (2A)
        # 20200-20899 (sauf 20900-20999) = Haute-Corse (2B)
        cp_num = int(cp_str)
        if (20000 <= cp_num <= 20199) or (20900 <= cp_num <= 20999):
            return '2A'
        elif 20200 <= cp_num <= 20899:
            return '2B'
    
    # Cas général : les 2 premiers chiffres
    return cp_str[:2]

def charger_stock_unite_legale(chemin_fichier_stock):
    """
    Charge et prepare les donnees du fichier StockUniteLegale
    """
    print(f"Chargement du fichier StockUniteLegale : {chemin_fichier_stock}")
    
    colonnes_a_garder = [
        'siren', 'denominationUniteLegale', 'dateCreationUniteLegale',
        'categorieEntreprise', 'trancheEffectifsUniteLegale',
        'categorieJuridiqueUniteLegale'  # Ajout de cette colonne
    ]
    
    try:
        stock_df = pd.read_csv(
            chemin_fichier_stock,
            usecols=colonnes_a_garder,
            dtype=str,
            encoding='utf-8'
        )
        
        # Standardiser le SIREN
        stock_df['siren'] = stock_df['siren'].str.zfill(9)
        
        print(f"   {len(stock_df):,} unites legales chargees")
        return stock_df
        
    except FileNotFoundError:
        print(f"ERREUR: Fichier StockUniteLegale non trouve : {chemin_fichier_stock}")
        return None
    except Exception as e:
        print(f"ERREUR lors du chargement : {e}")
        return None

def charger_stock_etablissement(chemin_fichier_stock_etab, siren_list=None):
    """
    Charge les donnees géographiques du fichier StockEtablissement
    """
    print(f"Chargement du fichier StockEtablissement : {chemin_fichier_stock_etab}")
    
    colonnes_geo = [
        'siren', 'siret', 'codePostalEtablissement', 
        'libelleCommuneEtablissement', 'codePaysEtrangerEtablissement',
        'etablissementSiege'
    ]
    
    try:
        # Si on a une liste de SIREN, on peut filtrer pour optimiser
        if siren_list is not None:
            # Charger par chunks pour les gros fichiers
            chunks = []
            for chunk in pd.read_csv(
                chemin_fichier_stock_etab,
                usecols=colonnes_geo,
                dtype=str,
                encoding='utf-8',
                chunksize=100000
            ):
                # Filtrer seulement les sièges sociaux
                chunk = chunk[chunk['etablissementSiege'] == 'true']
                chunk['siren'] = chunk['siren'].str.zfill(9)
                chunk_filtre = chunk[chunk['siren'].isin(siren_list)]
                if len(chunk_filtre) > 0:
                    chunks.append(chunk_filtre)
            
            if chunks:
                stock_etab_df = pd.concat(chunks, ignore_index=True)
            else:
                return None
        else:
            stock_etab_df = pd.read_csv(
                chemin_fichier_stock_etab,
                usecols=colonnes_geo,
                dtype=str,
                encoding='utf-8'
            )
            # Garder seulement les sièges sociaux
            stock_etab_df = stock_etab_df[stock_etab_df['etablissementSiege'] == 'true']
            stock_etab_df['siren'] = stock_etab_df['siren'].str.zfill(9)
        
        print(f"   {len(stock_etab_df):,} etablissements sieges charges")
        return stock_etab_df
        
    except FileNotFoundError:
        print(f"ATTENTION: Fichier StockEtablissement non trouve : {chemin_fichier_stock_etab}")
        print("Les donnees geographiques ne seront pas completees")
        return None
    except Exception as e:
        print(f"ERREUR lors du chargement : {e}")
        return None

def completer_fichier_cible_avec_stock(fichier_cible, chemin_stock, fichier_sortie=None, chemin_stock_etab=None):
    """
    Complete le fichier cible avec les donnees du StockUniteLegale et StockEtablissement
    """
    
    print("COMPLETION DU FICHIER CIBLE AVEC STOCK SIRENE")
    print("=" * 60)
    
    # 1. Chargement du fichier cible
    print(f"Chargement du fichier cible : {fichier_cible}")
    try:
        df_cible = pd.read_csv(fichier_cible, sep=';', encoding='utf-8-sig')
        print(f"   {len(df_cible):,} lignes chargees")
    except Exception as e:
        print(f"ERREUR: Impossible de charger le fichier cible : {e}")
        return None
    
    # 2. Chargement du StockUniteLegale
    stock_df = charger_stock_unite_legale(chemin_stock)
    if stock_df is None:
        return None
    
    # 3. Preparation des donnees pour la jointure
    print("Preparation des donnees pour jointure...")
    
    # Standardiser les SIREN dans le fichier cible
    df_cible['siren_std'] = df_cible['siren'].astype(str).str.zfill(9)
    df_cible['siren_g_std'] = df_cible['siren_g'].astype(str).str.zfill(9)
    
    # 4. Jointure pour les SIREN principaux
    print("Jointure pour les SIREN principaux...")
    df_avec_siren = df_cible.merge(
        stock_df,
        left_on='siren_std',
        right_on='siren',
        how='left',
        suffixes=('', '_stock_principal')
    )
    
    # 5. Jointure pour les SIREN groupes (nom_g)
    print("Jointure pour les SIREN groupes...")
    stock_groupe = stock_df[['siren', 'denominationUniteLegale']].copy()
    stock_groupe.rename(columns={'denominationUniteLegale': 'nom_g_stock'}, inplace=True)
    
    df_final = df_avec_siren.merge(
        stock_groupe,
        left_on='siren_g_std',
        right_on='siren',
        how='left',
        suffixes=('', '_groupe')
    )
    
    # 6. Chargement et jointure avec StockEtablissement si disponible
    if chemin_stock_etab and os.path.exists(chemin_stock_etab):
        print("\nChargement des donnees geographiques...")
        # Obtenir la liste des SIREN pour optimiser le chargement
        siren_uniques = pd.concat([df_final['siren_std'], df_final['siren_g_std']]).unique()
        
        stock_etab_df = charger_stock_etablissement(chemin_stock_etab, siren_uniques)
        
        if stock_etab_df is not None:
            # Jointure avec les données géographiques
            df_final = df_final.merge(
                stock_etab_df[['siren', 'codePostalEtablissement', 'libelleCommuneEtablissement', 'codePaysEtrangerEtablissement']],
                left_on='siren_std',
                right_on='siren',
                how='left',
                suffixes=('', '_etab')
            )
    
    # 7. Completion et transformation des colonnes
    print("Completion et transformation des colonnes...")
    
    def completer_si_vide(colonne_cible, colonne_source):
        """Complete la colonne cible seulement si elle est vide"""
        if colonne_cible not in df_final.columns:
            df_final[colonne_cible] = np.nan
        
        mask_vide = (
            df_final[colonne_cible].isna() | 
            (df_final[colonne_cible] == '') | 
            (df_final[colonne_cible] == 'nan')
        )
        df_final.loc[mask_vide, colonne_cible] = df_final.loc[mask_vide, colonne_source]
    
    # Completion des noms (denomination) pour les SIREN principaux
    completer_si_vide('nom', 'denominationUniteLegale')
    
    # Transformation et completion tranche effectifs
    print("   Transformation des tranches d'effectifs...")
    df_final['treff_transforme'] = df_final['trancheEffectifsUniteLegale'].apply(mapper_tranche_effectifs)
    completer_si_vide('treff_benef', 'treff_transforme')
    
    # Transformation de la date de création en tranche d'années
    print("   Transformation des dates de création en tranches d'années...")
    df_final['trannee_transforme'] = df_final['dateCreationUniteLegale'].apply(convertir_date_en_tranche)
    completer_si_vide('trannee', 'trannee_transforme')
    
    # Transformation de la catégorie juridique
    print("   Transformation des catégories juridiques...")
    df_final['cat_jur_transforme'] = df_final['categorieJuridiqueUniteLegale'].apply(mapper_categorie_juridique)
    completer_si_vide('cat_jur', 'cat_jur_transforme')
    
    # Completion nom_g (toujours remplacer comme demande)
    mask_siren_g_valide = df_final['nom_g_stock'].notna()
    df_final.loc[mask_siren_g_valide, 'nom_g'] = df_final.loc[mask_siren_g_valide, 'nom_g_stock']
    
    # Completion des données géographiques si disponibles
    if 'codePostalEtablissement' in df_final.columns:
        print("   Completion des donnees geographiques...")
        
        # Code postal
        completer_si_vide('cp', 'codePostalEtablissement')
        
        # Commune
        completer_si_vide('commune', 'libelleCommuneEtablissement')
        
        # Département - extraire du code postal et mapper
        df_final['dpt_extrait'] = df_final['cp'].apply(extraire_code_dept_du_cp)
        df_final['dpt_nom'] = df_final['dpt_extrait'].apply(mapper_departement)
        completer_si_vide('dpt', 'dpt_nom')
        
        # Régions (old_region et region) basées sur le département
        print("   Ajout des régions (anciennes et nouvelles)...")
        df_final[['old_region_calc', 'region_calc']] = df_final['dpt_extrait'].apply(
            lambda x: pd.Series(mapper_regions(x))
        )
        completer_si_vide('old_region', 'old_region_calc')
        completer_si_vide('region', 'region_calc')
        
        # Pays - toujours FRANCE et non FR
        df_final['pays_code'] = df_final['codePaysEtrangerEtablissement'].fillna('FR')
        df_final['pays_nom'] = df_final['pays_code'].apply(lambda x: 'FRANCE' if x == 'FR' else mapper_pays(x))
        completer_si_vide('pays_siege', 'pays_nom')
    else:
        print("   ATTENTION: Donnees geographiques non disponibles (fichier StockEtablissement manquant)")
        # Si pas de données géographiques mais qu'on a des codes postaux existants
        if 'cp' in df_final.columns:
            # Essayer d'extraire le département du code postal existant
            df_final['dpt_extrait'] = df_final['cp'].apply(extraire_code_dept_du_cp)
            df_final['dpt_nom'] = df_final['dpt_extrait'].apply(mapper_departement)
            completer_si_vide('dpt', 'dpt_nom')
            
            # Régions basées sur le département existant
            df_final[['old_region_calc', 'region_calc']] = df_final['dpt_extrait'].apply(
                lambda x: pd.Series(mapper_regions(x))
            )
            completer_si_vide('old_region', 'old_region_calc')
            completer_si_vide('region', 'region_calc')
        
        # Si on n'a pas de pays_siege, mettre FRANCE par défaut
        if 'pays_siege' not in df_final.columns:
            df_final['pays_siege'] = 'FRANCE'
        else:
            mask_pays_vide = df_final['pays_siege'].isna() | (df_final['pays_siege'] == '')
            df_final.loc[mask_pays_vide, 'pays_siege'] = 'FRANCE'
    
    # 8. Nettoyage des colonnes temporaires
    colonnes_a_supprimer = [
        'siren_std', 'siren_g_std', 'siren_stock_principal', 'siren_groupe', 'siren_etab',
        'denominationUniteLegale', 'dateCreationUniteLegale', 
        'categorieEntreprise', 'trancheEffectifsUniteLegale', 'nom_g_stock',
        'categorieJuridiqueUniteLegale', 'trannee_transforme', 'cat_jur_transforme',
        'treff_transforme', 'codePostalEtablissement', 'libelleCommuneEtablissement', 
        'codePaysEtrangerEtablissement', 'dpt_extrait', 'dpt_nom', 'pays_code', 
        'pays_nom', 'old_region_calc', 'region_calc'
    ]
    
    for col in colonnes_a_supprimer:
        if col in df_final.columns:
            df_final.drop(columns=[col], inplace=True)
    
    # 9. Statistiques de completion
    print("\nStatistiques de completion:")
    
    colonnes_completees = ['nom', 'treff_benef', 'trannee', 'cat_jur', 'nom_g', 'cp', 'commune', 'dpt', 'old_region', 'region', 'pays_siege']
    for colonne in colonnes_completees:
        if colonne in df_final.columns:
            avant = df_cible[colonne].notna().sum() if colonne in df_cible.columns else 0
            apres = df_final[colonne].notna().sum()
            ajoutes = apres - avant
            print(f"   {colonne}: {avant} -> {apres} (+{ajoutes})")
    
    # 10. Sauvegarde
    if fichier_sortie is None:
        fichier_sortie = fichier_cible.replace('.csv', '_complete_stock.csv')
    
    print(f"\nSauvegarde : {fichier_sortie}")
    df_final.to_csv(fichier_sortie, sep=';', index=False, encoding='utf-8-sig')
    
    print(f"\nCOMPLETION TERMINEE")
    print(f"Fichier sauvegarde : {fichier_sortie}")
    
    return df_final

def verifier_qualite_completion(df_resultat):
    """
    Verifie la qualite de la completion
    """
    print("\nVERIFICATION QUALITE:")
    print("-" * 30)
    
    total_lignes = len(df_resultat)
    colonnes_a_verifier = ['nom', 'treff_benef', 'trannee', 'cat_jur', 'nom_g', 'cp', 'commune', 'dpt', 'old_region', 'region', 'pays_siege']
    
    for col in colonnes_a_verifier:
        if col in df_resultat.columns:
            remplies = df_resultat[col].notna().sum()
            pourcentage = (remplies / total_lignes) * 100
            print(f"   {col}: {remplies}/{total_lignes} ({pourcentage:.1f}%)")
    
    # Afficher quelques exemples de transformations
    print("\nExemples de transformations:")
    print("-" * 30)
    
    if 'trannee' in df_resultat.columns:
        print("\nTranches d'années (simplifiées):")
        print(df_resultat['trannee'].value_counts().sort_index())
    
    if 'treff_benef' in df_resultat.columns:
        print("\nTranches d'effectifs:")
        print(df_resultat['treff_benef'].value_counts().head(10))
    
    if 'cat_jur' in df_resultat.columns:
        print("\nCatégories juridiques (Top 10):")
        print(df_resultat['cat_jur'].value_counts().head(10))
    
    if 'region' in df_resultat.columns:
        print("\nRégions (nouvelles) (Top 10):")
        print(df_resultat['region'].value_counts().head(10))
    
    if 'dpt' in df_resultat.columns:
        print("\nDépartements (Top 10):")
        print(df_resultat['dpt'].value_counts().head(10))
    
    if 'pays_siege' in df_resultat.columns:
        print("\nPays:")
        pays_counts = df_resultat['pays_siege'].value_counts()
        print(pays_counts.head(10) if len(pays_counts) > 10 else pays_counts)
    
    # Verification des SIREN non trouves
    siren_non_trouves = df_resultat[df_resultat['treff_benef'].isna()]
    if len(siren_non_trouves) > 0:
        print(f"\nSIREN non trouves dans StockUniteLegale: {len(siren_non_trouves)}")
        print("Exemples:")
        colonnes_exemple = [col for col in ['siren', 'nom'] if col in df_resultat.columns]
        if colonnes_exemple:
            print(siren_non_trouves[colonnes_exemple].head())

# Script principal
if __name__ == "__main__":
    
    # PARAMETRES A MODIFIER
    FICHIER_CIBLE = "C://Users//msamb//Documents//base_final_2021_suite_final.csv"
    FICHIER_STOCK = "C://Users//msamb//Documents//StockUniteLegale_utf8.csv"
    FICHIER_STOCK_ETAB = "C://Users//msamb//Documents//StockEtablissement_utf8.csv"  # Optionnel
    FICHIER_SORTIE = "C://Users//msamb//Documents//base_final_2021_suite_final_complet_stock.csv"
    
    print("DEMARRAGE DU PROCESSUS DE COMPLETION")
    print("=" * 50)
    print("Transformations qui seront appliquées:")
    print("- trannee: dates -> tranches simplifiées (ex: 'avant 1980', 'de 1980 à 1989'...)")
    print("- treff_benef: codes -> libellés (ex: '41' -> '500 à 999 salariés')")
    print("- cat_jur: codes -> libellés (ex: 5710 -> 'SAS')")
    print("- dpt: codes -> noms (ex: 75 -> 'Paris')")
    print("- old_region: ancienne région administrative")
    print("- region: nouvelle région (réforme 2016)")
    print("- pays_siege: 'FRANCE' par défaut")
    print("=" * 50)
    
    # Verification de l'existence des fichiers
    if not os.path.exists(FICHIER_CIBLE):
        print(f"ERREUR: Fichier cible non trouve : {FICHIER_CIBLE}")
        exit(1)
    
    if not os.path.exists(FICHIER_STOCK):
        print(f"ERREUR: Fichier StockUniteLegale non trouve : {FICHIER_STOCK}")
        print("Telechargez le fichier depuis : https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/")
        exit(1)
    
    # Vérifier si le fichier StockEtablissement existe
    if not os.path.exists(FICHIER_STOCK_ETAB):
        print(f"ATTENTION: Fichier StockEtablissement non trouve : {FICHIER_STOCK_ETAB}")
        print("Les donnees geographiques (cp, commune) ne pourront pas etre completees depuis SIRENE")
        print("Mais les departements seront deduits des codes postaux existants")
        FICHIER_STOCK_ETAB = None
    
    # Lancement de la completion
    try:
        df_complete = completer_fichier_cible_avec_stock(
            fichier_cible=FICHIER_CIBLE,
            chemin_stock=FICHIER_STOCK,
            fichier_sortie=FICHIER_SORTIE,
            chemin_stock_etab=FICHIER_STOCK_ETAB
        )
        
        if df_complete is not None:
            # Verification de la qualite
            verifier_qualite_completion(df_complete)
            
            # Apercu du resultat
            print(f"\nAPERCU DU RESULTAT:")
            print("-" * 30)
            colonnes_affichage = ['siren', 'nom', 'siren_g', 'nom_g', 'treff_benef', 'trannee', 'cat_jur', 'cp', 'commune', 'dpt', 'old_region', 'region', 'pays_siege']
            colonnes_disponibles = [col for col in colonnes_affichage if col in df_complete.columns]
            
            if colonnes_disponibles:
                # Afficher seulement quelques colonnes à la fois pour la lisibilité
                print("\nDonnées principales:")
                colonnes_principales = ['siren', 'nom', 'treff_benef', 'trannee', 'cat_jur']
                colonnes_principales_dispo = [col for col in colonnes_principales if col in df_complete.columns]
                if colonnes_principales_dispo:
                    print(df_complete[colonnes_principales_dispo].head(5).to_string())
                
                print("\nDonnées géographiques:")
                colonnes_geo = ['siren', 'cp', 'commune', 'dpt', 'old_region', 'region', 'pays_siege']
                colonnes_geo_dispo = [col for col in colonnes_geo if col in df_complete.columns]
                if colonnes_geo_dispo:
                    print(df_complete[colonnes_geo_dispo].head(5).to_string())
            
            print(f"\nProcessus termine avec succes.")
        
    except Exception as e:
        print(f"ERREUR: {e}")
        import traceback
        traceback.print_exc()

DEMARRAGE DU PROCESSUS DE COMPLETION
Transformations qui seront appliquées:
- trannee: dates -> tranches simplifiées (ex: 'avant 1980', 'de 1980 à 1989'...)
- treff_benef: codes -> libellés (ex: '41' -> '500 à 999 salariés')
- cat_jur: codes -> libellés (ex: 5710 -> 'SAS')
- dpt: codes -> noms (ex: 75 -> 'Paris')
- old_region: ancienne région administrative
- region: nouvelle région (réforme 2016)
- pays_siege: 'FRANCE' par défaut
COMPLETION DU FICHIER CIBLE AVEC STOCK SIRENE
Chargement du fichier cible : C://Users//msamb//Documents//base_final_2021_suite_final.csv
   4,619 lignes chargees
Chargement du fichier StockUniteLegale : C://Users//msamb//Documents//StockUniteLegale_utf8.csv
   28,236,992 unites legales chargees
Preparation des donnees pour jointure...
Jointure pour les SIREN principaux...
Jointure pour les SIREN groupes...

Chargement des donnees geographiques...
Chargement du fichier StockEtablissement : C://Users//msamb//Documents//StockEtablissement_utf8.csv
   1,937 etabl

C:\Users\msamb\AppData\Local\Temp\ipykernel_16792\631115534.py:944: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['20 à 49 salariés' '10000 salariés et plus' '200 à 249 salariés' ...
 '50 à 99 salariés' '1 ou 2 salariés' nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[mask_vide, colonne_cible] = df_final.loc[mask_vide, colonne_source]
C:\Users\msamb\AppData\Local\Temp\ipykernel_16792\631115534.py:944: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['60370' '75015' '92130' ... '75009' nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[mask_vide, colonne_cible] = df_final.loc[mask_vide, colonne_source]
C:\Users\msamb\AppData\Local\Temp\ipykernel_16792\631115534.py:944: FutureWarning: Setting an item of


Statistiques de completion:
   nom: 3161 -> 4510 (+1349)
   treff_benef: 0 -> 3020 (+3020)
   trannee: 4619 -> 4619 (+0)
   cat_jur: 396 -> 4619 (+4223)
   nom_g: 4470 -> 4470 (+0)
   cp: 0 -> 1982 (+1982)
   commune: 0 -> 1982 (+1982)
   dpt: 0 -> 1982 (+1982)
   old_region: 0 -> 1972 (+1972)
   region: 0 -> 1972 (+1972)
   pays_siege: 4619 -> 4619 (+0)

Sauvegarde : C://Users//msamb//Documents//base_final_2021_suite_final_complet_stock.csv

COMPLETION TERMINEE
Fichier sauvegarde : C://Users//msamb//Documents//base_final_2021_suite_final_complet_stock.csv

VERIFICATION QUALITE:
------------------------------
   nom: 4510/4619 (97.6%)
   treff_benef: 3020/4619 (65.4%)
   trannee: 4619/4619 (100.0%)
   cat_jur: 4619/4619 (100.0%)
   nom_g: 4470/4619 (96.8%)
   cp: 1982/4619 (42.9%)
   commune: 1982/4619 (42.9%)
   dpt: 1982/4619 (42.9%)
   old_region: 1972/4619 (42.7%)
   region: 1972/4619 (42.7%)
   pays_siege: 4619/4619 (100.0%)

Exemples de transformations:
-------------------------

In [14]:
!pip install pandas openpyxl numpy

In [5]:
import pandas as pd
import numpy as np
import os
import openpyxl
from datetime import datetime

def convertir_date_en_tranche(date_creation):
    """
    Convertit une date de création en tranche d'années simplifiée
    """
    try:
        if pd.isna(date_creation) or date_creation == '':
            return np.nan
        
        # Si c'est déjà une tranche, la retourner
        date_str = str(date_creation)
        if 'avant' in date_str or 'de ' in date_str:
            return date_creation
        
        # Extraire l'année de la date
        if len(date_str) >= 4:
            annee = int(date_str[:4])
        else:
            return np.nan
        
        # Déterminer la tranche simplifiée
        if annee < 1980:
            return "avant 1980"
        elif annee < 1990:
            return "de 1980 à 1989"
        elif annee < 2000:
            return "de 1990 à 1999"
        elif annee < 2010:
            return "de 2000 à 2009"
        elif annee < 2020:
            return "de 2010 à 2019"
        else:
            return "de 2020 à aujourd'hui"
    except:
        return np.nan

def charger_fichier_auto(chemin_fichier, nom_fichier=""):
    """
    Charge un fichier CSV ou Excel en détectant automatiquement le séparateur ou le format.
    """
    print(f"\nChargement du fichier {nom_fichier}: {chemin_fichier}")
    try:
        ext = os.path.splitext(chemin_fichier)[-1].lower()
        if ext in [".xls", ".xlsx"]:
            df = pd.read_excel(chemin_fichier)
            print(f"   Format détecté: Excel ({ext})")
        elif ext == ".csv":
            df = pd.read_csv(chemin_fichier, sep=';', encoding='utf-8-sig', nrows=5)
            if len(df.columns) > 1:
                df = pd.read_csv(chemin_fichier, sep=';', encoding='utf-8-sig')
                print(f"   Séparateur détecté: point-virgule (;)")
            else:
                df = pd.read_csv(chemin_fichier, sep=',', encoding='utf-8-sig')
                print(f"   Séparateur détecté: virgule (,)")
        else:
            print(f"   ERREUR: Format de fichier non supporté ({ext})")
            return None
        print(f"   {len(df):,} lignes chargées")
        print(f"   {len(df.columns)} colonnes: {', '.join(df.columns[:5])}...")
        return df
    except Exception as e:
        print(f"ERREUR lors du chargement: {e}")
        return None

def remplacer_donnees_excel_modele(fichier_modele, fichier_nouveau, df):
    """
    Copie le fichier Excel modèle, et remplace les données par celles de df,
    en conservant les styles, formats, filtres, etc.
    """
    wb = openpyxl.load_workbook(fichier_modele)
    ws = wb.active

    # Efface toutes les lignes de données sauf l'en-tête
    ws.delete_rows(2, ws.max_row - 1)

    # Insère les nouvelles lignes de données
    for r_idx, row in enumerate(df.itertuples(index=False, name=None), start=2):
        for c_idx, value in enumerate(row, start=1):
            ws.cell(row=r_idx, column=c_idx, value=value)

    # Enregistre le nouveau fichier
    wb.save(fichier_nouveau)
    print(f"✓ Fichier sauvegardé avec formatage : {fichier_nouveau}")

def fusionner_fichiers_cir(fichier_cir_ancien, fichier_base_complete, fichier_sortie=None):
    """
    Fusionne le fichier CIR ancien avec le fichier base complété.
    Pour les mixtes : prend tout de la base sauf le type, et ajoute les montants du prov21
    """
    print("\n" + "="*70)
    print("FUSION DES FICHIERS CIR 2021 - VERSION FINALE")
    print("="*70)

    # 1. Chargement des fichiers
    df_cir = charger_fichier_auto(fichier_cir_ancien, "CIR 2021 prov (ancien)")
    if df_cir is None:
        return None

    df_base = charger_fichier_auto(fichier_base_complete, "Base complétée")
    if df_base is None:
        return None

    # 2. Identification des colonnes SIREN
    print("\nIdentification des colonnes SIREN...")
    siren_col_cir = None
    siren_col_base = None

    for col in df_cir.columns:
        if col.lower() in ['siren', 'siren_benef', 'num_siren']:
            siren_col_cir = col
            break

    for col in df_base.columns:
        if col.lower() in ['siren', 'siren_benef', 'num_siren']:
            siren_col_base = col
            break

    if not siren_col_cir or not siren_col_base:
        print("ERREUR: Colonne SIREN non trouvée")
        return None

    print(f"   Colonne SIREN dans CIR: '{siren_col_cir}'")
    print(f"   Colonne SIREN dans Base: '{siren_col_base}'")

    # Renommer la colonne SIREN de la base si nécessaire
    if siren_col_base != siren_col_cir:
        df_base = df_base.rename(columns={siren_col_base: siren_col_cir})

    # 3. Standardiser les SIREN
    print("\nStandardisation des SIREN...")
    df_cir['SIREN_STD'] = df_cir[siren_col_cir].astype(str).str.zfill(9)
    df_base['SIREN_STD'] = df_base[siren_col_cir].astype(str).str.zfill(9)

    # 4. Transformer les dates en tranches dans la base
    print("\nTransformation des dates en tranches d'années...")
    if 'trannee' in df_base.columns:
        df_base['trannee'] = df_base['trannee'].apply(convertir_date_en_tranche)
        print("   Échantillon après transformation:")
        print(f"   {df_base['trannee'].dropna().head(5).tolist()}")

    # 5. Vérifier et éliminer les doublons
    print("\nVérification de l'unicité des SIREN...")
    doublons_cir = df_cir[df_cir['SIREN_STD'].duplicated()]['SIREN_STD'].nunique()
    doublons_base = df_base[df_base['SIREN_STD'].duplicated()]['SIREN_STD'].nunique()
    
    if doublons_cir > 0:
        print(f"   ATTENTION: {doublons_cir} SIREN en double dans le CIR - suppression...")
        df_cir = df_cir.drop_duplicates(subset=['SIREN_STD'], keep='first')
        
    if doublons_base > 0:
        print(f"   ATTENTION: {doublons_base} SIREN en double dans la base - suppression...")
        df_base = df_base.drop_duplicates(subset=['SIREN_STD'], keep='first')

    # 6. Analyse des données
    print("\nAnalyse des données:")
    print(f"   Lignes dans CIR (après dédoublonnage): {len(df_cir):,}")
    print(f"   Lignes dans Base (après dédoublonnage): {len(df_base):,}")

    # Identifier les groupes de SIREN
    siren_cir = set(df_cir['SIREN_STD'])
    siren_base = set(df_base['SIREN_STD'])
    
    siren_communs = siren_cir & siren_base
    siren_cir_seuls = siren_cir - siren_base
    siren_base_seuls = siren_base - siren_cir
    
    print(f"\n   SIREN présents dans les deux fichiers: {len(siren_communs):,}")
    print(f"   SIREN uniquement dans CIR: {len(siren_cir_seuls):,}")
    print(f"   SIREN uniquement dans Base: {len(siren_base_seuls):,}")

    # 7. Création des DataFrames pour chaque groupe
    print("\nCréation des DataFrames par groupe...")
    
    # A. SIREN uniquement dans CIR (on garde tel quel)
    df_cir_seuls = df_cir[df_cir['SIREN_STD'].isin(siren_cir_seuls)].copy()
    df_cir_seuls['source'] = 'prov2021'
    df_cir_seuls['montant_cretot_gen_prov21'] = np.nan
    df_cir_seuls['montant_crerd_gen_prov21'] = np.nan
    df_cir_seuls['montant_crecoll_gen_prov21'] = np.nan
    df_cir_seuls['montant_creinno_gen_prov21'] = np.nan
    
    # B. SIREN uniquement dans la base (on garde tel quel)
    df_base_seuls = df_base[df_base['SIREN_STD'].isin(siren_base_seuls)].copy()
    df_base_seuls['source'] = 'base_2021_suite'
    df_base_seuls['montant_cretot_gen_prov21'] = np.nan
    df_base_seuls['montant_crerd_gen_prov21'] = np.nan
    df_base_seuls['montant_crecoll_gen_prov21'] = np.nan
    df_base_seuls['montant_creinno_gen_prov21'] = np.nan
    
    # C. SIREN communs (mixtes)
    print(f"\nTraitement des {len(siren_communs):,} SIREN mixtes...")
    df_mixtes = df_base[df_base['SIREN_STD'].isin(siren_communs)].copy()
    
    # Pour les mixtes, on prend la base MAIS on remplace le type par celui du CIR
    # et on ajoute les montants du CIR
    df_cir_pour_mixtes = df_cir[df_cir['SIREN_STD'].isin(siren_communs)].set_index('SIREN_STD')
    df_mixtes = df_mixtes.set_index('SIREN_STD')
    
    # Remplacer le type par celui du CIR
    if 'type' in df_cir_pour_mixtes.columns and 'type' in df_mixtes.columns:
        df_mixtes['type'] = df_cir_pour_mixtes['type']
        print("   ✓ Colonne 'type' remplacée par celle du CIR")
    
    # Ajouter les montants du prov21
    if 'cretot_gen' in df_cir_pour_mixtes.columns:
        df_mixtes['montant_cretot_gen_prov21'] = df_cir_pour_mixtes['cretot_gen']
        print("   ✓ Montant cretot_gen du prov21 ajouté")
    else:
        df_mixtes['montant_cretot_gen_prov21'] = np.nan
        print("   ! Colonne cretot_gen non trouvée dans prov21")
    
    if 'crerd_gen' in df_cir_pour_mixtes.columns:
        df_mixtes['montant_crerd_gen_prov21'] = df_cir_pour_mixtes['crerd_gen']
        print("   ✓ Montant crerd_gen du prov21 ajouté")
    else:
        df_mixtes['montant_crerd_gen_prov21'] = np.nan
        print("   ! Colonne crerd_gen non trouvée dans prov21")
    
    if 'crecoll_gen' in df_cir_pour_mixtes.columns:
        df_mixtes['montant_crecoll_gen_prov21'] = df_cir_pour_mixtes['crecoll_gen']
        print("   ✓ Montant crecoll_gen du prov21 ajouté")
    else:
        df_mixtes['montant_crecoll_gen_prov21'] = np.nan
        print("   ! Colonne crecoll_gen non trouvée dans prov21")
    
    if 'creinno_gen' in df_cir_pour_mixtes.columns:
        df_mixtes['montant_creinno_gen_prov21'] = df_cir_pour_mixtes['creinno_gen']
        print("   ✓ Montant creinno_gen du prov21 ajouté")
    else:
        df_mixtes['montant_creinno_gen_prov21'] = np.nan
        print("   ! Colonne creinno_gen non trouvée dans prov21")
    
    df_mixtes['source'] = 'mixte_prov2021_base2021'
    df_mixtes = df_mixtes.reset_index()
    
    # 8. Alignement des colonnes
    print("\nAlignement des colonnes...")
    
    # Obtenir toutes les colonnes uniques
    toutes_colonnes = set()
    for df in [df_cir_seuls, df_base_seuls, df_mixtes]:
        toutes_colonnes.update(df.columns)
    
    # Supprimer SIREN_STD de la liste (sera remplacé par la colonne SIREN originale)
    toutes_colonnes.discard('SIREN_STD')
    
    # Ajouter les colonnes manquantes
    for df in [df_cir_seuls, df_base_seuls, df_mixtes]:
        for col in toutes_colonnes:
            if col not in df.columns:
                df[col] = np.nan
    
    # 9. Fusion finale
    print("\nFusion finale des trois groupes...")
    
    # Supprimer SIREN_STD avant la concatenation
    for df in [df_cir_seuls, df_base_seuls, df_mixtes]:
        if 'SIREN_STD' in df.columns:
            df.drop(columns=['SIREN_STD'], inplace=True)
    
    df_final = pd.concat([df_mixtes, df_cir_seuls, df_base_seuls], ignore_index=True)
    
    # 10. Vérifications finales
    print(f"\nVérifications finales:")
    print(f"   Nombre total de lignes: {len(df_final):,}")
    print(f"   Nombre de SIREN uniques: {df_final[siren_col_cir].nunique():,}")
    
    if len(df_final) != df_final[siren_col_cir].nunique():
        print("   ATTENTION: Il reste des doublons!")
    else:
        print("   ✓ Pas de doublons - Une seule ligne par SIREN")
    
    print("\nDistribution des sources:")
    print(df_final['source'].value_counts())
    
    # Vérifier trannee
    if 'trannee' in df_final.columns:
        print("\nVérification des tranches d'années:")
        print(df_final['trannee'].value_counts().head())
    
    # Statistiques sur les montants ajoutés
    print("\nStatistiques sur les montants prov21 ajoutés:")
    montants_cretot = df_final[df_final['source'] == 'mixte_prov2021_base2021']['montant_cretot_gen_prov21'].notna().sum()
    montants_crerd = df_final[df_final['source'] == 'mixte_prov2021_base2021']['montant_crerd_gen_prov21'].notna().sum()
    montants_crecoll = df_final[df_final['source'] == 'mixte_prov2021_base2021']['montant_crecoll_gen_prov21'].notna().sum()
    montants_creinno = df_final[df_final['source'] == 'mixte_prov2021_base2021']['montant_creinno_gen_prov21'].notna().sum()
    print(f"   Montants cretot_gen renseignés: {montants_cretot:,}")
    print(f"   Montants crerd_gen renseignés: {montants_crerd:,}")
    print(f"   Montants crecoll_gen renseignés: {montants_crecoll:,}")
    print(f"   Montants creinno_gen renseignés: {montants_creinno:,}")
    
    # 11. Réorganiser les colonnes
    # Mettre source et les montants prov21 à la fin
    colonnes_fin = ['source', 'montant_cretot_gen_prov21', 'montant_crerd_gen_prov21', 
                    'montant_crecoll_gen_prov21', 'montant_creinno_gen_prov21']
    colonnes_debut = [col for col in df_final.columns if col not in colonnes_fin]
    df_final = df_final[colonnes_debut + colonnes_fin]
    
    # 12. Sauvegarde
    if fichier_sortie is None:
        fichier_sortie = "cir_2021_prov_fusion_finale.xlsx"
    else:
        fichier_sortie = os.path.splitext(fichier_sortie)[0] + ".xlsx"

    print(f"\nSauvegarde du fichier fusionné : {fichier_sortie}")
    remplacer_donnees_excel_modele(fichier_cir_ancien, fichier_sortie, df_final)
    
    print(f"\n✓ FUSION TERMINÉE AVEC SUCCÈS")
    print(f"✓ Une seule ligne par SIREN")
    print(f"✓ Pour les mixtes: données de la base + type du CIR + 4 montants du prov21")
    
    return df_final

def comparer_fichiers_avant_apres(df_cir_original, df_final):
    """
    Compare les fichiers avant et après fusion
    """
    print("\n" + "="*50)
    print("COMPARAISON AVANT/APRÈS")
    print("="*50)
    
    print(f"\nNombre de lignes:")
    print(f"   CIR original: {len(df_cir_original):,}")
    print(f"   Fichier fusionné: {len(df_final):,}")
    print(f"   Différence: {len(df_final) - len(df_cir_original):+,}")
    
    colonnes_a_comparer = ['nom', 'cat_jur', 'trannee', 'commune', 'dpt', 'cp', 'treff_benef', 'old_region', 'region', 'pays_siege']
    
    print("\nTaux de remplissage des colonnes:")
    for col in colonnes_a_comparer:
        if col in df_cir_original.columns and col in df_final.columns:
            avant = df_cir_original[col].notna().sum()
            apres = df_final[col].notna().sum()
            taux_avant = (avant / len(df_cir_original)) * 100
            taux_apres = (apres / len(df_final)) * 100
            
            print(f"\n{col}:")
            print(f"   Avant: {avant:,}/{len(df_cir_original):,} ({taux_avant:.1f}%)")
            print(f"   Après: {apres:,}/{len(df_final):,} ({taux_apres:.1f}%)")
            print(f"   Amélioration: {taux_apres - taux_avant:+.1f} points")

# Script principal
if __name__ == "__main__":
    FICHIER_CIR_ANCIEN = "M://str-dgri-gecir-donnees-fiscales//z_Statistiques Séries Demandes//2021//0_BASE_CIR2021prov_extr202306.xlsx"
    FICHIER_BASE_COMPLETE = "C://Users//msamb//Documents//base_final_2021_suite_final_complet_stock.csv"
    FICHIER_SORTIE = "C://Users//msamb//Documents//cir_2021_prov_fusion_finale.xlsx"

    print("SCRIPT DE FUSION CIR 2021 - VERSION FINALE")
    print("="*50)
    print("Ce script va:")
    print("1. Pour les SIREN uniquement dans CIR: garder tel quel")
    print("2. Pour les SIREN uniquement dans la base: garder tel quel")
    print("3. Pour les SIREN mixtes:")
    print("   - Prendre TOUTES les colonnes de la base")
    print("   - SAUF 'type' qui vient du CIR")
    print("   - Ajouter les 4 montants du prov21:")
    print("     • cretot_gen")
    print("     • crerd_gen")
    print("     • crecoll_gen")
    print("     • creinno_gen")
    print("4. Ajouter une colonne 'source' pour la traçabilité")
    print("="*50)

    if not os.path.exists(FICHIER_CIR_ANCIEN):
        print(f"\nERREUR: Fichier CIR ancien non trouvé: {FICHIER_CIR_ANCIEN}")
        exit(1)
    if not os.path.exists(FICHIER_BASE_COMPLETE):
        print(f"\nERREUR: Fichier base complété non trouvé: {FICHIER_BASE_COMPLETE}")
        exit(1)

    try:
        print("\nChargement du fichier CIR original pour comparaison...")
        df_cir_original = charger_fichier_auto(FICHIER_CIR_ANCIEN, "CIR original")

        df_fusion = fusionner_fichiers_cir(
            fichier_cir_ancien=FICHIER_CIR_ANCIEN,
            fichier_base_complete=FICHIER_BASE_COMPLETE,
            fichier_sortie=FICHIER_SORTIE
        )

        if df_fusion is not None and df_cir_original is not None:
            comparer_fichiers_avant_apres(df_cir_original, df_fusion)
            print("\n✓ Processus terminé avec succès!")
            print(f"✓ {len(df_fusion):,} lignes au total")
            print(f"✓ Les 4 colonnes de montants prov21 ont été ajoutées:")
            print(f"  - montant_cretot_gen_prov21")
            print(f"  - montant_crerd_gen_prov21")
            print(f"  - montant_crecoll_gen_prov21")
            print(f"  - montant_creinno_gen_prov21")

    except Exception as e:
        print(f"\nERREUR: {e}")
        import traceback
        traceback.print_exc()

SCRIPT DE FUSION CIR 2021 - VERSION FINALE
Ce script va:
1. Pour les SIREN uniquement dans CIR: garder tel quel
2. Pour les SIREN uniquement dans la base: garder tel quel
3. Pour les SIREN mixtes:
   - Prendre TOUTES les colonnes de la base
   - SAUF 'type' qui vient du CIR
   - Ajouter les 4 montants du prov21:
     • cretot_gen
     • crerd_gen
     • crecoll_gen
     • creinno_gen
4. Ajouter une colonne 'source' pour la traçabilité

Chargement du fichier CIR original pour comparaison...

Chargement du fichier CIR original: M://str-dgri-gecir-donnees-fiscales//z_Statistiques Séries Demandes//2021//0_BASE_CIR2021prov_extr202306.xlsx
   Format détecté: Excel (.xlsx)
   28,810 lignes chargées
   157 colonnes: siren, nom, siren_g, nom_g, type...

FUSION DES FICHIERS CIR 2021 - VERSION FINALE

Chargement du fichier CIR 2021 prov (ancien): M://str-dgri-gecir-donnees-fiscales//z_Statistiques Séries Demandes//2021//0_BASE_CIR2021prov_extr202306.xlsx
   Format détecté: Excel (.xlsx)
   28,810

C:\Users\msamb\AppData\Local\Temp\ipykernel_2340\1033389304.py:55: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(chemin_fichier, sep=';', encoding='utf-8-sig')


   ✓ Colonne 'type' remplacée par celle du CIR
   ✓ Montant cretot_gen du prov21 ajouté
   ✓ Montant crerd_gen du prov21 ajouté
   ✓ Montant crecoll_gen du prov21 ajouté
   ✓ Montant creinno_gen du prov21 ajouté

Alignement des colonnes...

Fusion finale des trois groupes...

Vérifications finales:
   Nombre total de lignes: 32,854
   Nombre de SIREN uniques: 32,854
   ✓ Pas de doublons - Une seule ligne par SIREN

Distribution des sources:
source
prov2021                   28331
base_2021_suite             4044
mixte_prov2021_base2021      479
Name: count, dtype: int64

Vérification des tranches d'années:
trannee
de 2010 à 2019    12032
de 2000 à 2009     6419
de 1990 à 1999     3666
de 1980 à 1989     2317
avant 1980         2256
Name: count, dtype: int64

Statistiques sur les montants prov21 ajoutés:
   Montants cretot_gen renseignés: 479
   Montants crerd_gen renseignés: 479
   Montants crecoll_gen renseignés: 479
   Montants creinno_gen renseignés: 479

Sauvegarde du fichier fusio

In [3]:
import openpyxl
import pandas as pd

# Chargement du fichier Excel
df = pd.read_excel("C://Users//msamb//Documents//cir_2021_prov_fusion_finale.xlsx")

# Liste des colonnes de crédits
cre_list = ["cretot_gen", "crerd_gen", "crecoll_gen", "creinno_gen"]

# Mapping vers les colonnes prov21 correspondantes
prov_map = {
    "cretot_gen": "montant_cretot_gen_prov21",
    "crerd_gen": "montant_crerd_gen_prov21",
    "crecoll_gen": "montant_crecoll_gen_prov21",
    "creinno_gen": "montant_creinno_gen_prov21",
}

# 1. Somme "normale"
somme_normale = df[cre_list].sum()

# 2. Somme corrigée avec prov21 pour les "mixte"
somme_corrigee = pd.Series(dtype=float)
for col in cre_list:
    prov_col = prov_map[col]
    valeurs = df.apply(
        lambda row: row[prov_col] if row["source"] == "mixte_prov2021_base2021" and pd.notnull(row[prov_col]) else row[col],
        axis=1
    )
    somme_corrigee[col] = valeurs.sum()

# 3. Écart entre les deux
ecart = somme_corrigee - somme_normale

# 4. Mise en forme dans un DataFrame
result = pd.DataFrame({
    "Somme_normale": somme_normale,
    "Somme_corrigée": somme_corrigee,
    "Différence": ecart
})

# 5. Formatage pour affichage lisible
def format_nombre(x):
    return "{:,.0f}".format(x).replace(",", " ").replace(".0", "") if pd.notnull(x) else ""

# Affichage formaté
print("\nComparaison des montants (valeurs en euros) :\n")
for index, row in result.iterrows():
    normal = format_nombre(row["Somme_normale"])
    corrigee = format_nombre(row["Somme_corrigée"])
    diff = format_nombre(row["Différence"])
    print(f"{index:<12} | Somme normale : {normal:>12} € | Corrigée : {corrigee:>12} € | Différence : {diff:>12} €")



Comparaison des montants (valeurs en euros) :

cretot_gen   | Somme normale : 7 265 494 072 € | Corrigée : 7 320 514 209 € | Différence :   55 020 137 €
crerd_gen    | Somme normale : 6 842 255 020 € | Corrigée : 6 907 150 378 € | Différence :   64 895 357 €
crecoll_gen  | Somme normale :   30 483 111 € | Corrigée :   30 244 757 € | Différence :     -238 353 €
creinno_gen  | Somme normale :  392 755 910 € | Corrigée :  383 118 887 € | Différence :   -9 637 023 €
